In [ ]:
# 原跟车训练测试过程
from torch import nn, optim
import torch
import torch.nn as nnn
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import partial


ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# car_following_data = HighD_data
# the data shape format (number of car following event, 4 dimension data, lenth of each dimension)数据形状格式（跟车事件的车辆数、4 维数据、各维数据的长度）
# 4 dimension data= [spacing, subject_vehicle_speed, relative_speed, leading_vehicle_speed]4 维数据= [车距、后车车速、相对车速、前车车速］
# print(car_following_data)
# print(car_following_data.shape)   # HighD(12541, 4, 375)

# split the date into train, validation, test 数据集划分
def split_train(data,test_ratio,val_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    # 根据输入百分比（test_ratio）计算测试集大小
    test_set_size=int(len(data)*test_ratio)
    # 根据输入百分比（val_ratio）计算验证集大小
    val_number=int(len(data)*(test_ratio+val_ratio))
    # 根据索引划分数据集
    test_indices =shuffled_indices[:test_set_size] # 测试 (1881)
    val_indices=shuffled_indices[test_set_size:val_number] # 验证 (1881)
    train_indices=shuffled_indices[val_number:] # 训练 (8779)
    return data[train_indices],data[test_indices],data[val_indices]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# Define NN and lstm models
# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150

class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, split: str, max_len = max_len):
        if split == 'train':
            self.data = train_data
        elif split == 'test':
            self.data = test_data
        elif split == 'validation':
            self.data = val_data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts  # (max_len - 1)
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}

# Define data-driven car-following models
class nn_model(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 256):
        super(nn_model, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Tanh(),
            )

    def forward(self, x):
        out = ACC_LIMIT*self.encoder(x)
        return out

class lstm_model(nn.Module):
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size=3, hidden_size=16, lstm_layers=1, dropout=0.1):
        super(lstm_model, self).__init__()
        # LSTM编码器，接受输入序列并输出隐藏状态
        self.encoder = nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=False, dropout=dropout)
        # 线性层，将LSTM的输出映射到1维
        # self.linear = nn.Linear(hidden_size, 1)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 1)
        self.relu = nnn.ReLU(inplace=True)
        # 初始化线性层的权重和偏置
        nn.init.normal_(self.linear1.weight, 0, 0.02)
        nn.init.constant_(self.linear1.bias, 0.0)
        nn.init.normal_(self.linear2.weight, 0, 0.02)
        nn.init.constant_(self.linear2.bias, 0.0)
    # LSTM 模型由编码器和线性层组成。编码器接收输入数据并计算编码表示和最终层的隐藏状态。
    # 随后，线性层层将隐藏状态映射到输出值，输出值经过 tanh 激活函数并乘以代表加速度极限的常数
    def forward(self, src):
        # 将输入序列通过LSTM编码器得到隐藏状态  src(10,32,3)
        # enc_x(10,32,16)是模型在每个时间步的输出，(h_n, c_n)((1,32,16), (1,32,16))) 是模型的隐藏状态和记忆单元状态
        enc_x, (h_n, c_n) = self.encoder(src)
        # 如果隐藏状态的维度为3，则取最后一层的隐藏状态 即(batch_size, hidden_size)
        if len(h_n.shape) == 3:
            h_n = h_n[-1] # (32,16)
        # 通过线性层和激活函数得到最终输出
        out_0 = self.linear1(h_n) # (32,16)
        out = self.linear2(self.relu(out_0)) # (32, 1)
        # out = self.linear(h_n)
        # out = torch.tanh(out) * ACC_LIMIT
        return out


# Train
# 获取划分数据集
# (8779,4,375)，(1881,4,375)，(1881,4,375)
# train_data, test_data, val_data = split_train(car_following_data,0.15,0.15)
# train_data, test_data, val_data = split_train(car_following_data,0.8,0.1)
# train_data = Lyft_train
# test_data = Lyft_test
# val_data = Lyft_val
train_data = NGSIM_train
test_data = NGSIM_test
val_data = NGSIM_val
print(train_data.shape, test_data.shape, val_data.shape)
dataset = 'NGSIM'
model_type = 'lstm'
batch_size = 32
total_epochs = 30
# 创建训练集 DataLoader
train_dataset = ImitationCarFolData(split = 'train', max_len = max_len)
train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建验证集 DataLoader
validation_dataset = ImitationCarFolData(split = 'validation', max_len = max_len)
validation_loader = DataLoader(
        validation_dataset ,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建测试集 DataLoader
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=1,
        drop_last=True)

# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

his_horizon = 10 # number of time steps as history data
lr = 1e-3 # learning rate
# lr = 0.01
save = f'{model_type}_{dataset}.pt' # 保存模型文件
# 根据名称定义模型
if model_type == 'nn':
    model = nn_model(input_size = his_horizon*3).to(device)
elif model_type == 'lstm':
    model = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm
model_state = list(model.parameters())
print(model_state)
# 定义优化器和损失函数
model_optim = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失
best_epoch = None # 最佳验证损失时的轮次

print("----")
# 训练过程
for epoch in tqdm(range(total_epochs)):
    train_losses = [] # 记录每个epoch的训练损失
    validation_losses = [] # 记录每个epoch的验证损失
    model.train()
    # 遍历数据集
    for i, item in enumerate(train_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(32,374)连接，转换成(3,32,374)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(374,32,3)，y->(374,32)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            # 根据his_horizon个数据预测加速度
            acc_pre = model(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        model_optim.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
        model_optim.step()

        train_losses.append(loss.item())
    # 计算本轮平均损失
    train_loss = np.mean(train_losses)

    train_loss_his.append(train_loss)
    print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

    # 验证过程
    model.eval()
    error_list = []
    for i, item in enumerate(validation_loader):
        x_data, y_data = item['inputs'], item['label']
        # Put T into the first dimension, B, T, d -> T, B, d
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        # lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # save a copy of x_data
        x_data_orig = x_data.clone().detach()
        y_label = y_data[his_horizon:]
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data

            acc_pre = model(x).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        validation_loss = criterion(y_pre, y_label)
        model_optim.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        model_optim.step()
        validation_losses.append(validation_loss.item())
    # 计算平均验证误差
    mean_validation_error = np.mean(validation_losses)
    train_val_error = (mean_validation_error + train_loss)/2
    if best_validation_loss is None or best_validation_loss > train_val_error:
        best_epoch = epoch + 1
        best_validation_loss = train_val_error
        # save the best model
        with open(save, 'wb') as f:
            torch.save(model, f)

    validation_error_his.append(mean_validation_error)
    print("Epoch: {0}| Validation error: {1:.7f}".format(epoch + 1, mean_validation_error))

print("Epoch:{0}| Best Train+Val Loss：{1:.7f}".format(best_epoch, best_validation_loss))
plt.plot(train_loss_his, label = 'train_loss')
plt.plot(validation_error_his, label = 'val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss (MSE for acceleration prediction)')
plt.show()


# 用该数据集训练得到的模型去对其他数据集进行测试
# Ts = 0.04
# max_len = 375
# # car_following_data = NGSIM_data
# # print(car_following_data.shape)
# # train_data, test_data, val_data = split_train(car_following_data,0.15,0.15)
# test_data = HighD_test
# # 创建测试集 DataLoader
# test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
# test_loader = DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=1,
#         drop_last=True)


# Testing, closed-loop prediction
# Load the best model saved
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device) # x(374,32,3)  y(374,32)
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测的车距
    spacing_pre = x_data[..., 0] # (374,32)
    # 真实的车距
    spacing_obs = x_data_orig[..., 0] # (374,32)

    error = criterion(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
# Ts = 0.04
# max_len = 375
batch_size = 1
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=1,
        drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
jerk_set = [] 
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
#             MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_) # 正常情况(139)
    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)  # 大小为测试数据总数减去发生了碰撞的事件

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = criterion(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)

    jerk_set.append(jerk)
# 输出指标
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)

In [ ]:
# 只含有test和指标计算代码部分，用于使用在不同数据集下训练的模型用在其他数据集上测试
from torch import nn, optim
import torch
import torch.nn as nnn
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import partial


ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
# car_following_data = NGSIM_data
# the data shape format (number of car following event, 4 dimension data, lenth of each dimension)数据形状格式（跟车事件的车辆数、4 维数据、各维数据的长度）
# 4 dimension data= [spacing, subject_vehicle_speed, relative_speed, leading_vehicle_speed]4 维数据= [车距、后车车速、相对车速、前车车速］
# print(car_following_data)
# print(car_following_data.shape)   # (12541, 4, 375)

# # 除HighD的数据集需要进行连接
# def car_stack(data):
#     # 创建一个空的目标数组，形状为 (1930, 4, 600)
#     target_shape = (data.shape[0], data.shape[1], data[0, 0].shape[0])
#     target_array = np.empty(target_shape)

#     # 遍历原始数组，将每个元素转换为 (4, 600) 的子数组，并放入目标数组
#     for i in range(data.shape[0]):
#         for j in range(data.shape[1]):
#             target_array[i, j, :] = data[i, j]
#     # 最终 target_array 的形状为 (1930, 4, 600)
#     return target_array


# split the date into train, validation, test 数据集划分
def split_train(data,test_ratio,val_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    # 根据输入百分比（test_ratio）计算测试集大小
    test_set_size=int(len(data)*test_ratio)
    # 根据输入百分比（val_ratio）计算验证集大小
    val_number=int(len(data)*(test_ratio+val_ratio))
    # 根据索引划分数据集
    test_indices =shuffled_indices[:test_set_size] # 测试 (1881)
    val_indices=shuffled_indices[test_set_size:val_number] # 验证 (1881)
    train_indices=shuffled_indices[val_number:] # 训练 (8779)
    return data[train_indices],data[test_indices],data[val_indices]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# Define NN and lstm models
# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150

class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, split: str, max_len = max_len):
        if split == 'train':
            self.data = train_data
        elif split == 'test':
            self.data = test_data
        elif split == 'validation':
            self.data = val_data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的除最后一项的前 max_len-1 个时间步
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}

# Define data-driven car-following models
class nn_model(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 256):
        super(nn_model, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Tanh(),
            )

    def forward(self, x):
        out = ACC_LIMIT*self.encoder(x)
        return out

class lstm_model(nn.Module):
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size=3, hidden_size=16, lstm_layers=1, dropout=0.1):
        super(lstm_model, self).__init__()
        # LSTM编码器，接受输入序列并输出隐藏状态
        self.encoder = nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=False, dropout=dropout)
        # 线性层，将LSTM的输出映射到1维
        # self.linear = nn.Linear(hidden_size, 1)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 1)
        self.relu = nnn.ReLU(inplace=True)
        # 初始化线性层的权重和偏置
        nn.init.normal_(self.linear1.weight, 0, 0.02)
        nn.init.constant_(self.linear1.bias, 0.0)
        nn.init.normal_(self.linear2.weight, 0, 0.02)
        nn.init.constant_(self.linear2.bias, 0.0)
    # LSTM 模型由编码器和线性层组成。编码器接收输入数据并计算编码表示和最终层的隐藏状态。
    # 随后，线性层层将隐藏状态映射到输出值，输出值经过 tanh 激活函数并乘以代表加速度极限的常数
    def forward(self, src):
        # 将输入序列通过LSTM编码器得到隐藏状态  src(10,32,3)
        # enc_x(10,32,16)是模型在每个时间步的输出，(h_n, c_n)((1,32,16), (1,32,16))) 是模型的隐藏状态和记忆单元状态
        enc_x, (h_n, c_n) = self.encoder(src)
        # 如果隐藏状态的维度为3，则取最后一层的隐藏状态 即(batch_size, hidden_size)
        if len(h_n.shape) == 3:
            h_n = h_n[-1] # (32,16)
        # 通过线性层和激活函数得到最终输出
        out_0 = self.linear1(h_n) # (32,16)
        out = self.linear2(self.relu(out_0)) # (32, 1)
        # out = self.linear(h_n)
        # out = torch.tanh(out) * ACC_LIMIT
        return out

HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)

# 获取划分数据集
# (8779,4,375)，(1881,4,375)，(1881,4,375)
# train_data, test_data, val_data = split_train(car_following_data,0.15,0.15)
# train_data, test_data, val_data = Lyft_train, Lyft_test, Lyft_val
train_data = NGSIM_train
test_data = NGSIM_test
val_data = NGSIM_val
print(test_data.shape)
dataset = 'NGSIM'
model_type = 'lstm'
batch_size = 32
total_epochs = 20
# 创建训练集 DataLoader
train_dataset = ImitationCarFolData(split = 'train', max_len = max_len)
train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建验证集 DataLoader
validation_dataset = ImitationCarFolData(split = 'validation', max_len = max_len)
validation_loader = DataLoader(
        validation_dataset ,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建测试集 DataLoader
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=1,
        drop_last=True)

# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

his_horizon = 10 # number of time steps as history data
lr = 1e-3 # learning rate
save = f'{model_type}_{dataset}.pt' # 保存模型文件
# 根据名称定义模型
if model_type == 'nn':
    model = nn_model(input_size = his_horizon*3).to(device)
elif model_type == 'lstm':
    model = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm
# 定义优化器和损失函数
model_optim = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失

# Testing, closed-loop prediction
# Load the best model saved
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device) # x(374,32,3)  y(374,32)
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测的车距
    spacing_pre = x_data[..., 0] # (374,32)
    # 真实的车距
    spacing_obs = x_data_orig[..., 0] # (374,32)

    error = criterion(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
batch_size = 1
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=1,
        drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
jerk_set = [] 
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
            MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=MyDevice)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_)

    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = criterion(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)

    jerk_set.append(jerk)
# 输出指标
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)

In [34]:
# 求一些值用
from torch import nn, optim
import torch
import torch.nn as nnn
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s   15/150s
# max_len = 375 # for HighD dataset is 375 for others are 150
Ts = 0.1
max_len = 150

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 获取每个跟车事件的时间步长
def DataMin(dataset):
    data_min = []
    for i in range(dataset.shape[0]):
        data_min.append(dataset[i][0].size)
    return data_min

datamin = DataMin(NGSIM_data)
# print(datamin)
print("最小时间步：", np.min(datamin))

# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len):
        self.data = data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}

# 求原测试集中的平均jerk
Ts = 0.1
max_len = 150
dataset_test = Lyft_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=1,
            shuffle=False,
            num_workers=1,
            drop_last=True)
# 记录跟车事件数和发生碰撞事件数
his_horizon = 10
count = 0
col = 0
jerk_set = []
# TTC_set = []
minimum_ttc_set = []
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
#     x_data, y_data = x_data.transpose(0, 2).float(), y_data.transpose(0, 1).float()

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B = y_data.shape # (total steps, batch_size, d) as the shape of the data
#     print(T,B)
    a = np.diff(y_data)/Ts
    b = np.abs(a)
    jerk =np.mean(b)
    jerk_set.append(jerk)
# 输出指标
print("jerk", np.mean(jerk_set))

cuda:0
最小时间步： 150


2240it [00:07, 288.08it/s]

jerk 0.7705402415347494


In [ ]:
# MAML-跟车  version1
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
import copy


# ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)

# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 任务数据集划分
def split_train(data,data_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    if data_ratio < 0:
        # 如果小于零，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
        # 根据索引划分数据集
        data_indices =shuffled_indices[:test_set_size] # 测试 (1881)
    else:
        # 如果大于等于零，则data_ratio为获取数据集中的数量
        data_num = int(data_ratio/2)
        data_indices1 = shuffled_indices[:data_num]
        data_indices2 = shuffled_indices[data_num:data_ratio]
    return data[data_indices1], data[data_indices2]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150
his_horizon = 10

# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len):
        self.data = data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


# 定义模型
class nn_model(nn.Module):
    def __init__(self, input_size = 3):
        super(nn_model,self).__init__()
        self.net=nn.Sequential(OrderedDict([
            ('l1',nn.Linear(input_size,256)),
            ('relu1',nn.ReLU()),
            ('l2',nn.Linear(256,256)),
            ('relu2',nn.ReLU()),
            ('l3',nn.Linear(256,1))
        ]))
        self.state = list(self.net.parameters())
    # 用于元学习的内循环中更新权重的方法，用于计算元损失，然后重置下一个元任务的权重。
    def argforward(self,x,weights): 
        x=F.linear(x,weights[0],weights[1])  # (input,weight,bias)
        x=F.relu(x)
        x=F.linear(x,weights[2],weights[3])
        x=F.relu(x)
        x=F.linear(x,weights[4],weights[5])
        return x


# 元学习内循环训练过程
def inner_train(data_loader, net ,temp_weights, criterion):
    # 初始化变量
    train_loss_his = []  # 训练损失
    best_train_loss = None  # 最佳训练损失
    best_epoch = None  # 最佳验证损失时的轮次
    # 训练过程
    train_losses = []  # 记录每个epoch的训练损失
    net.train()
    # 遍历数据集
    for i, item in enumerate(data_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((20,149)，(20,149)，(20,149))  y_data(20,149)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(20,149)连接，转换成(3,20,149)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(149,20,3)，y->(149,20)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
#         print(T,B,d)  # 149  20  3
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,20)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,20)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = net.argforward(x, temp_weights).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        return loss


# 生成不同数据集的任务类
class DataTask():
    def __init__(self,data_range):
        self.data_range = data_range
  # 生成任务数据，默认生成一个数据点，可以通过指定size参数生成多个数据点
    def sample_data(self,size=1):
        if self.data_range == 1:
            meta_data = HighD_data
            Ts = 0.04
            max_len = 375
#             print('--HighD--')
        elif self.data_range == 2:
            meta_data = Lyft_data
            Ts = 0.1
            max_len = 150
#             print('--Lyft---')
        elif self.data_range == 3:
            meta_data = SPMD_DAS1_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS1')
        elif self.data_range == 4:
            meta_data = SPMD_DAS2_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS2')
        elif self.data_range == 5:
            meta_data = Waymo_data
            Ts = 0.1
            max_len = 150
#             print('--Waymo--')
        meta_data_set1, meta_data_set2 = split_train(meta_data, size)
#         print(len(meta_data_set1),len(meta_data_set2))
        dataset_loader1 = ImitationCarFolData(meta_data_set1, max_len = max_len)
        dataset_loader2 = ImitationCarFolData(meta_data_set2, max_len = max_len)
        return dataset_loader1, dataset_loader2


# 任务分布，及选择不同的数据
class DataDistribution():
    def __init__(self, data_range=5, n=3):
        self.data_range = data_range    # 获取数据集的数值
        self.n = n
    # 随机生成一个跟车数据集
    def sample_task(self):
        # 从1到5中随机选择3个不重复的数
        data_choice = np.random.choice(self.data_range, size=self.n, replace=False) + 1
        return data_choice


# MAML元学习训练过程
class MAML():
    def __init__(self, net, alpha, beta, tasks, k):  # (net,alpha=0.01,beta=0.001,tasks=data_tasks,k=100)
        # 初始化 MAML 算法的元参数
        self.net = net  # 元学习使用的神经网络模型
        self.weights = list(net.parameters())  # 获取神经网络的参数
        self.alpha = alpha  # 内循环学习率
        self.beta = beta  # 外循环学习率
        self.tasks = tasks  # 正弦任务的分布
        self.k = k  # 内循环（任务采样）的数据集大小
        self.criterion = nn.MSELoss()  # 均方误差损失函数
        self.meta_optimiser = torch.optim.Adam(self.weights, self.beta)  # 外循环优化器（Adam）
        self.meta_losses = []  # 用于存储元学习的损失历史
        self.plot_every = 1  # 每隔多少次迭代记录一次损失，用于绘图
        self.print_every = 1  # 每隔多少次迭代输出一次损失信息，用于打印
        self.batch_size = int(self.k/2)

    def inner_loop(self, task):
        # 内循环更新参数，用于计算元学习损失
        temp_weights = [w.clone() for w in self.weights]  # 复制当前网络参数作为临时参数
        dataset_loader1, dataset_loader2 = task.sample_data(size=self.k)  # 从任务中采样数据集 D 和 D'
        dataloader1 = DataLoader(
                dataset_loader1,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        # 训练过程
        loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
        grads = torch.autograd.grad(loss, temp_weights)  # 计算损失对参数的梯度
        temp_weights = [w - self.alpha * g for w, g in zip(temp_weights, grads)]  # 临时参数更新 梯度下降
        dataloader2 = DataLoader(
                dataset_loader2,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        metaloss = inner_train(dataloader2, self.net, temp_weights, self.criterion) / self.batch_size
        return metaloss

    def outer_loop(self, num_epochs):  # epoch 500
        total_loss = 0
        for epoch in range(1, num_epochs + 1):
            metaloss_sum = 0
            tasks = self.tasks.sample_task()
            for i in tasks:
                task = DataTask(i)
                metaloss = self.inner_loop(task)  # 内循环更新参数，计算元学习损失
                metaloss_sum += metaloss  # mete_loss求和
            metagrads = torch.autograd.grad(metaloss_sum, self.weights)  # 计算元学习损失对参数的梯度
            # 重要步骤：使用元学习梯度更新网络参数
            for w, g in zip(self.weights, metagrads):
                w.grad = g
            self.meta_optimiser.step()  # 使用外循环优化器更新网络参数
            total_loss += metaloss_sum.item() / len(tasks)
            if epoch % self.print_every == 0:
                print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
            if epoch % self.plot_every == 0:
                self.meta_losses.append(total_loss / self.plot_every)
                total_loss = 0

dataset = 'NGSIM'
model_type = 'nn'
save = f'{model_type}_{dataset}.pt' # 保存模型文件
if model_type == 'nn':
    net = nn_model(input_size = his_horizon*3).to(device)
# elif model_type == 'lstm':
#     net = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm
data_tasks=DataDistribution(data_range=5, n=3)
maml=MAML(net,alpha=0.01,beta=0.0001,tasks=data_tasks,k=400)


maml.outer_loop(num_epochs=10000)
plt.plot(maml.meta_losses)
plt.show()
print('----------------------')


# Train
Ts = 0.1
max_len = 150
og_net = maml.net.net
# 创建一个与原始网络结构相同的虚拟网络
if model_type == 'nn':
    dummy_net = nn.Sequential(OrderedDict([
        ('l1', nn.Linear(his_horizon*3,256)),
        ('relu1', nn.ReLU()),
        ('l2', nn.Linear(256,256)),
        ('relu2', nn.ReLU()),
        ('l3', nn.Linear(256,1))
    ]))
dummy_net=dummy_net.to(device)
# 加载原始网络的权重
dummy_net.load_state_dict(og_net.state_dict())
# 进行迭代，每次更新虚拟网络的参数
num_shots=5
lr = 0.01
loss_fn=nn.MSELoss()
optim=torch.optim.SGD
opt=optim(dummy_net.parameters(),lr=lr)


# 数据集划分
def split_data(data,data_ratio):
    if data_ratio < 0:
        # 如果小于零，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
    else:
        # 如果大于等于零，则data_ratio为获取数据集中的数量
        data_size = data_ratio
    return data[:data_size]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# 获取数据集的数量
K=40
dataset_train = split_data(NGSIM_train, K)
dataset_loader_train = ImitationCarFolData(dataset_train, max_len = max_len)
train_loader = DataLoader(
            dataset_loader_train,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_val = NGSIM_val
dataset_loader_val = ImitationCarFolData(dataset_val, max_len = max_len)
val_loader = DataLoader(
            dataset_loader_val,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=10,
            shuffle=False,
            num_workers=1,
            drop_last=True)


# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失
best_epoch = None # 最佳验证损失时的轮次

# 训练过程
for epoch in tqdm(range(num_shots)):
    train_losses = [] # 记录每个epoch的训练损失
    validation_losses = [] # 记录每个epoch的验证损失
    jerk_val = 0
    dummy_net.train()
    # 遍历数据集
    for i, item in enumerate(train_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(32,374)连接，转换成(3,32,374)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(374,32,3)，y->(374,32)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            # 根据his_horizon个数据预测加速度
            acc_pre = dummy_net(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = loss_fn(y_pre, y_label)
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
        opt.step()

        train_losses.append(loss.item())
    # 计算本轮平均损失
    train_loss = np.mean(train_losses)

    train_loss_his.append(train_loss)
    print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

    # 验证过程
    dummy_net.eval()
    error_list = []
    for i, item in enumerate(val_loader):
        x_data, y_data = item['inputs'], item['label']
        # Put T into the first dimension, B, T, d -> T, B, d
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        # lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # save a copy of x_data
        x_data_orig = x_data.clone().detach()
        y_label = y_data[his_horizon:]
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
            if model_type == 'nn':
              x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            acc_pre = dummy_net(x).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        validation_loss = loss_fn(y_pre, y_label)
        # jerk_val = np.mean(np.abs(np.diff(torch.tensor(y_pre))/Ts))/batch_size
        # print("Val jerk：", jerk_val)
#         opt.zero_grad()
#         torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25)
#         opt.step()
        validation_losses.append(validation_loss.item())
    # 计算平均验证误差
    mean_validation_error = np.mean(validation_losses)
    train_val_error = (mean_validation_error + train_loss)/2
    if best_validation_loss is None or best_validation_loss > train_val_error:
        best_epoch = epoch + 1
        best_validation_loss = train_val_error
        # save the best model
    with open(save, 'wb') as f:
        torch.save(dummy_net, f)

    validation_error_his.append(mean_validation_error)
    print("Epoch: {0}| Validation error: {1:.7f}".format(epoch + 1, mean_validation_error))

# print(list(dummy_net.parameters()))
print("Epoch:{0}| Best Train+Val Loss：{1:.7f}".format(best_epoch, best_validation_loss))
plt.plot(train_loss_his, label = 'train_loss')
plt.plot(validation_error_his, label = 'val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss (MSE for acceleration prediction)')
plt.show()


# Testing, closed-loop prediction
# Load the best model saved
model = dummy_net
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()


error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device) # x(374,32,3)  y(374,32)
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
        if model_type == 'nn':
            x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测的车距
    spacing_pre = x_data[..., 0] # (374,32)
    # 真实的车距
    spacing_obs = x_data_orig[..., 0] # (374,32)

    error = loss_fn(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
# Ts = 0.1
# max_len = 150
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=1,
            shuffle=False,
            num_workers=1,
            drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
jerk_set = []
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
            # MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_) # 正常情况(139)
    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)  # 大小为测试数据总数减去发生了碰撞的事件

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = loss_fn(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)

    jerk_set.append(jerk)
# 输出指标
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)

cpu
1/10000. loss: 0.030780471861362457
2/10000. loss: 0.04516826073328654
3/10000. loss: 0.3769957224527995
4/10000. loss: 0.3960939645767212
5/10000. loss: 0.34749897321065265
6/10000. loss: 0.03875022133191427
7/10000. loss: 0.6340972979863485
8/10000. loss: 1.103097677230835
9/10000. loss: 0.7345333099365234
10/10000. loss: 0.8408072789510092
11/10000. loss: 0.4542378584543864
12/10000. loss: 0.1158681611220042
13/10000. loss: 0.06863734622796376
14/10000. loss: 0.7657466729482015
15/10000. loss: 0.19987616936365762
16/10000. loss: 1.2442971070607503
17/10000. loss: 0.14375762144724527
18/10000. loss: 0.98065718015035
19/10000. loss: 1.2417702674865723
20/10000. loss: 0.20725069443384805
21/10000. loss: 0.11335354050000508
22/10000. loss: 0.15266165137290955
23/10000. loss: 1.0211774508158367
24/10000. loss: 1.0980748335520427
25/10000. loss: 0.8192783196767172
26/10000. loss: 0.6264430284500122
27/10000. loss: 0.504382848739624
28/10000. loss: 0.38451504707336426
29/10000. loss: 0

In [ ]:
# MAML-跟车 增加LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
import copy
import warnings
# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=UserWarning)


SEED = 22
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


# ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 任务数据集划分
def split_train(data,data_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    if data_ratio > 0 and data_ratio <= 1:
        # 如果小于等于1，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
        data_num = int(data_size/2)
        # 根据索引划分数据集
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_size]   # query set
    else:
        # 如果大于1，则data_ratio为获取数据集中的数量
        data_num = int(data_ratio/2)
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_ratio]   # query set
    return data[data_indices1], data[data_indices2]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# Define NN and lstm models
# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150
his_horizon = 10

# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len):
        self.data = data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


# nn模型
class nn_model(nn.Module):
    def __init__(self, input_size = 3):
        super(nn_model,self).__init__()
        self.net=nn.Sequential(OrderedDict([
                ('l1',nn.Linear(input_size,256)),
                ('relu1',nn.ReLU()),
                ('l2',nn.Linear(256,256)),
                ('relu2',nn.ReLU()),
                ('l3',nn.Linear(256,1))
            ]))
        self.state = list(self.net.parameters())
    # 用于元学习的内循环中更新权重的方法，用于计算元损失，然后重置下一个元任务的权重。
    def forward(self,x,weights): 
        x=F.linear(x,weights[0],weights[1])  # (input,weight,bias)
        x=F.relu(x)
        x=F.linear(x,weights[2],weights[3])
        x=F.relu(x)
        x=F.linear(x,weights[4],weights[5])
        return x

# LSTM模型
class lstm_model(nn.Module):
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size=3, hidden_size=16, lstm_layers=1, dropout=0.1):
        super(lstm_model, self).__init__()
        self.net1=nn.Sequential(OrderedDict([
            ('lstm',nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=False, dropout=dropout))
        ]))
        self.net2=nn.Sequential(OrderedDict([
            ('l1',nn.Linear(hidden_size,hidden_size)),
            ('relu',nn.ReLU()),
            ('l2',nn.Linear(hidden_size,1))
        ]))
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
#         # LSTM编码器，接受输入序列并输出隐藏状态
#         self.encoder = nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=False, dropout=dropout)
#         # 线性层，将LSTM的输出映射到1维
#         self.linear1 = nn.Linear(hidden_size, hidden_size)
#         self.relu = nn.ReLU(inplace=True)
#         self.linear2 = nn.Linear(hidden_size, 1)
        # 初始化线性层的权重和偏置
#         nn.init.normal_(self.linear1.weight, 0, 0.02)
#         nn.init.constant_(self.linear1.bias, 0.0)
#         nn.init.normal_(self.linear2.weight, 0, 0.02)
#         nn.init.constant_(self.linear2.bias, 0.0)
    # LSTM 模型由编码器和线性层组成。编码器接收输入数据并计算编码表示和最终层的隐藏状态。
    def forward(self, x, weights):
        # LSTM编码器，接受输入序列并输出隐藏状态
        lstm_encoder = nn.LSTM(self.input_size, self.hidden_size, self.lstm_layers, batch_first=False, dropout=self.dropout)
        # 将已知的权重替换为模型的权重
        lstm_encoder.weight_ih_l0.data = weights[0]
        lstm_encoder.weight_hh_l0.data = weights[1]
        lstm_encoder.bias_ih_l0.data = weights[2]
        lstm_encoder.bias_hh_l0.data = weights[3]
#         # 线性层，将LSTM的输出映射到1维
#         self.linear1 = nn.Linear(self.hidden_size, self.hidden_size)
#         self.relu = nn.ReLU(inplace=True)
#         self.linear2 = nn.Linear(self.hidden_size, 1)
        # 将输入序列通过LSTM编码器得到隐藏状态  src(10,32,3)
        # enc_x(10,32,16)是模型在每个时间步的输出，(h_n, c_n)((1,32,16), (1,32,16))) 是模型的隐藏状态和记忆单元状态
        enc_x, (h_n, c_n) = lstm_encoder(x)
        # 如果隐藏状态的维度为3，则取最后一层的隐藏状态 即(batch_size, hidden_size)
        if len(h_n.shape) == 3:
            h_n = h_n[-1] # (32,16)
        # 通过线性层和激活函数得到最终输出
        out = F.linear(h_n, weights[4], weights[5])
        x = F.relu(out)
        out = F.linear(h_n, weights[6], weights[7])
        return out


class lstm_test(nn.Module):
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size=3, hidden_size=16, lstm_layers=1, dropout=0.1):
        super(lstm_test, self).__init__()
        # LSTM编码器，接受输入序列并输出隐藏状态
        self.encoder = nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=False, dropout=dropout)
        # 线性层，将LSTM的输出映射到1维
        # self.linear = nn.Linear(hidden_size, 1)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU(inplace=True)
        # 初始化线性层的权重和偏置
#         nn.init.normal_(self.linear1.weight, 0, 0.02)
#         nn.init.constant_(self.linear1.bias, 0.0)
#         nn.init.normal_(self.linear2.weight, 0, 0.02)
#         nn.init.constant_(self.linear2.bias, 0.0)
    # LSTM 模型由编码器和线性层组成。编码器接收输入数据并计算编码表示和最终层的隐藏状态。
    # 随后，线性层层将隐藏状态映射到输出值，输出值经过 tanh 激活函数并乘以代表加速度极限的常数
    def forward(self, src):
        # 将输入序列通过LSTM编码器得到隐藏状态  src(10,32,3)
        # enc_x(10,32,16)是模型在每个时间步的输出，(h_n, c_n)((1,32,16), (1,32,16))) 是模型的隐藏状态和记忆单元状态
        enc_x, (h_n, c_n) = self.encoder(src)
        # 如果隐藏状态的维度为3，则取最后一层的隐藏状态 即(batch_size, hidden_size)
        if len(h_n.shape) == 3:
            h_n = h_n[-1] # (32,16)
        # 通过线性层和激活函数得到最终输出
        out_0 = self.linear1(h_n) # (32,16)
        out = self.linear2(self.relu(out_0)) # (32, 1)
        # out = self.linear(h_n)
        # out = torch.tanh(out) * ACC_LIMIT
        return out


# 元学习内循环训练过程
def inner_train(data_loader, net ,temp_weights, criterion):
    # 初始化变量
    train_loss_his = []  # 训练损失
    best_train_loss = None  # 最佳训练损失
    best_epoch = None  # 最佳验证损失时的轮次
    # 训练过程
    train_losses = []  # 记录每个epoch的训练损失
    net.train()
    # 遍历数据集
    for i, item in enumerate(data_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((20,149)，(20,149)，(20,149))  y_data(20,149)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(20,149)连接，转换成(3,20,149)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(149,20,3)，y->(149,20)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
#         print(T,B,d)  # 149  20  3
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,20)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,20)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = net(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        return loss


# 生成不同数据集的任务类
class DataTask():
    def __init__(self,data_range):
        self.data_range = data_range
  # 生成任务数据，默认生成一个数据点，可以通过指定size参数生成多个数据点
    def sample_data(self,size=1):
        if self.data_range == 1:
            meta_data = HighD_data
            Ts = 0.04
            max_len = 375
#             print('--HighD--')
        elif self.data_range == 2:
            meta_data = Lyft_data
            Ts = 0.1
            max_len = 150
#             print('--Lyft---')
        elif self.data_range == 3:
            meta_data = SPMD_DAS1_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS1')
        elif self.data_range == 4:
            meta_data = SPMD_DAS2_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS2')
        elif self.data_range == 5:
            meta_data = Waymo_data
            Ts = 0.1
            max_len = 150
#             print('--Waymo--')
        meta_data_set1, meta_data_set2 = split_train(meta_data, size)
#         print(len(meta_data_set1),len(meta_data_set2))
        dataset_loader1 = ImitationCarFolData(meta_data_set1, max_len = max_len)
        dataset_loader2 = ImitationCarFolData(meta_data_set2, max_len = max_len)
        return dataset_loader1, dataset_loader2


# 任务分布，及选择不同的数据
class DataDistribution():
    def __init__(self, data_range=5, n=3):
        self.data_range = data_range    # 获取数据集的数值
        self.n = n
    # 随机生成一个跟车数据集
    def sample_task(self):
        # 从1到5中随机选择3个不重复的数
        data_choice = np.random.choice(self.data_range, size=self.n, replace=False) + 1
        return data_choice


# MAML元学习训练过程
class MAML():
    def __init__(self, net, alpha, beta, tasks, k):  # (net,alpha=0.01,beta=0.001,tasks=data_tasks,k=100)
        # 初始化 MAML 算法的元参数
        self.net = net  # 元学习使用的神经网络模型
        self.weights = list(net.parameters())  # 获取神经网络的参数
        self.alpha = alpha  # 内循环学习率
        self.beta = beta  # 外循环学习率
        self.tasks = tasks  # 正弦任务的分布
        self.k = k  # 内循环（任务采样）的数据集大小
        self.criterion = nn.MSELoss()  # 均方误差损失函数
        self.meta_optimiser = torch.optim.Adam(self.weights, self.beta)  # 外循环优化器（Adam）
        self.meta_losses = []  # 用于存储元学习的损失历史
        self.plot_every = 1  # 每隔多少次迭代记录一次损失，用于绘图
        self.print_every = 1  # 每隔多少次迭代输出一次损失信息，用于打印
        self.batch_size = int(self.k/2)
        self.models = []

    def inner_loop(self, task):
        learner = lstm_test(input_size = 3, lstm_layers = 1).to(device)
        state_dict = copy.deepcopy(self.net.state_dict())
        learner.load_state_dict(state_dict)
        # 内循环更新参数，用于计算元学习损失
        temp_weights = dict((name, param) for (name, param) in learner.named_parameters())  # 复制当前网络参数作为临时参数
#         weightz = copy.deepcopy(temp_weights)
        dataset_loader1, dataset_loader2 = task.sample_data(size=self.k)  # 从任务中采样数据集 D 和 D'
        dataloader1 = DataLoader(
                dataset_loader1,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        # 训练过程
        loss = inner_train(dataloader1, learner, temp_weights, self.criterion) / self.batch_size
        g = torch.autograd.grad(loss, learner.parameters(), create_graph=True)

        g = tuple(g[i] if i != 0 else torch.zeros(g[0].shape, requires_grad=True) for i in range(len(g)))
        
#         temp_weights = dict((name, param - self.alpha * g) for ((name, param), g) in zip(temp_weights.items(), g))
        
        for ((name, param), g_tuple) in zip(temp_weights.items(), g):
            # 对梯度元组中的每个张量进行设备移动，并计算新的参数值
            new_param = param.to(device) - self.alpha * g_tuple.to(device)
            temp_weights[name] = new_param
        # 测试weight有没有变化
#         for (name, param) in temp_weights.items():
#             print(temp_weights[name] == weightz[name])    
        learner.load_state_dict(temp_weights)
        
        dataloader2 = DataLoader(
                dataset_loader2,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        metaloss = inner_train(dataloader2, learner, temp_weights, self.criterion) / self.batch_size
        return metaloss ,learner

    def outer_loop(self, num_epochs):  # epoch 500
        total_loss = 0
        for epoch in range(1, num_epochs + 1):
            metaloss_sum = 0
            tasks = self.tasks.sample_task()  # 从任务分布中采样一个元任务
            grads = []
            for i in tasks:
                task = DataTask(i)
                metaloss, learner = self.inner_loop(task)  # 内循环更新参数，计算元学习损失
                g = torch.autograd.grad(metaloss, learner.parameters(), create_graph=True)

                #zero the gradients
                g = tuple(g[i] for i in range(len(g)))

                meta_grads = {name:g for ((name, _), g) in zip(learner.named_parameters(), g)}

                #pass the meta-task gradients to the 
                grads.append(meta_grads)
                metaloss_sum += metaloss  # mete_loss求和
            dataset1,dataset2 = task.sample_data(size=20)  # 从任务中采样数据集 D
            dataloader = DataLoader(
                    dataset1,
                    batch_size=10,
                    shuffle=True,
                    num_workers=1,
                    drop_last=False)
            loss = inner_train(dataloader, self.net, grads, self.criterion) / 10
            loss.backward(retain_graph=True)

            # Unpack the gradients from dictionary of meta-gradients
            gradients = {k: sum(d[k] for d in grads) for k in grads[0].keys()}

            hooks = []
            for(k,v) in self.net.named_parameters():
                def get_closure():
                    key = k
                    def replace_grad(grad):
                        return gradients[key]
                    return replace_grad
                hooks.append(v.register_hook(get_closure()))
            torch.optim.Adam(self.net.parameters(), lr=self.beta).zero_grad()
            loss.backward()

            torch.optim.Adam(self.net.parameters(), lr=self.beta).step()

            for h in hooks:
                h.remove()

            self.models.append(copy.deepcopy(self.net.state_dict()))

            total_loss += metaloss_sum.item() / len(tasks)
            if epoch % self.print_every == 0:
                print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
            if epoch % self.plot_every == 0:
                self.meta_losses.append(total_loss / self.plot_every)
                total_loss = 0

dataset = 'NGSIM'
model_type = 'lstm'
save = f'{model_type}_{dataset}.pt' # 保存模型文件
if model_type == 'nn':
    net = nn_model(input_size = his_horizon*3).to(device)
elif model_type == 'lstm':
    net = lstm_test(input_size = 3, lstm_layers = 1).to(device) # single layer lstm
# print(list(net.parameters()))
data_tasks=DataDistribution(data_range=5, n=3)
maml=MAML(net,alpha=0.01,beta=0.0001,tasks=data_tasks,k=300)


maml.outer_loop(num_epochs=5000)
plt.plot(maml.meta_losses)
plt.show()
print('----------------------')
# print(list(net.parameters()))


# Train
Ts = 0.1
max_len = 150
og_net = maml.net
# 创建一个与原始网络结构相同的虚拟网络
if model_type == 'nn':
    dummy_net = nn_model(input_size = his_horizon*3)
elif model_type == 'lstm':
    dummy_net = lstm_test(input_size = 3, lstm_layers = 1)
dummy_net=dummy_net.to(device)
# 加载原始网络的权重
dummy_net.load_state_dict(og_net.state_dict())
# 进行迭代，每次更新虚拟网络的参数
num_shots=5
lr = 0.01
loss_fn=nn.MSELoss()
optim=torch.optim.Adam
opt=optim(dummy_net.parameters(),lr=lr)


# 数据集划分
def split_data(data,data_ratio):
    if data_ratio < 0:
        # 如果小于零，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
    else:
        # 如果大于等于零，则data_ratio为获取数据集中的数量
        data_size = data_ratio
    return data[:data_size]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# 获取数据集的数量
K=40
dataset_train = split_data(NGSIM_train, K)
dataset_loader_train = ImitationCarFolData(dataset_train, max_len = max_len)
train_loader = DataLoader(
            dataset_loader_train,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_val = NGSIM_val
dataset_loader_val = ImitationCarFolData(dataset_val, max_len = max_len)
val_loader = DataLoader(
            dataset_loader_val,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=10,
            shuffle=False,
            num_workers=1,
            drop_last=True)


# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失
best_epoch = None # 最佳验证损失时的轮次

# 训练过程
for epoch in tqdm(range(num_shots)):
    train_losses = [] # 记录每个epoch的训练损失
    validation_losses = [] # 记录每个epoch的验证损失
    jerk_val = 0
    dummy_net.train()
    # 遍历数据集
    for i, item in enumerate(train_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(32,374)连接，转换成(3,32,374)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(374,32,3)，y->(374,32)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            # 根据his_horizon个数据预测加速度
            acc_pre = dummy_net(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = loss_fn(y_pre, y_label)
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
        opt.step()

        train_losses.append(loss.item())
    # 计算本轮平均损失
    train_loss = np.mean(train_losses)

    train_loss_his.append(train_loss)
    print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

    # 验证过程
    dummy_net.eval()
    error_list = []
    for i, item in enumerate(val_loader):
        x_data, y_data = item['inputs'], item['label']
        # Put T into the first dimension, B, T, d -> T, B, d
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        # lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # save a copy of x_data
        x_data_orig = x_data.clone().detach()
        y_label = y_data[his_horizon:]
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
            if model_type == 'nn':
              x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            acc_pre = dummy_net(x).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        validation_loss = loss_fn(y_pre, y_label)
        # jerk_val = np.mean(np.abs(np.diff(torch.tensor(y_pre))/Ts))/batch_size
        # print("Val jerk：", jerk_val)
        opt.zero_grad()
        torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25)
        opt.step()
        validation_losses.append(validation_loss.item())
    # 计算平均验证误差
    mean_validation_error = np.mean(validation_losses)
    train_val_error = (mean_validation_error + train_loss)/2
    if best_validation_loss is None or best_validation_loss > train_val_error:
        best_epoch = epoch + 1
        best_validation_loss = train_val_error
        # save the best model
    with open(save, 'wb') as f:
        torch.save(dummy_net, f)

    validation_error_his.append(mean_validation_error)
    print("Epoch: {0}| Validation error: {1:.7f}".format(epoch + 1, mean_validation_error))

# print(list(dummy_net.parameters()))
print("Epoch:{0}| Best Train+Val Loss：{1:.7f}".format(best_epoch, best_validation_loss))
plt.plot(train_loss_his, label = 'train_loss')
plt.plot(validation_error_his, label = 'val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss (MSE for acceleration prediction)')
plt.show()


# Testing, closed-loop prediction
# Load the best model saved
model = dummy_net
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()


error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device) # x(374,32,3)  y(374,32)
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测的车距
    spacing_pre = x_data[..., 0] # (374,32)
    # 真实的车距
    spacing_obs = x_data_orig[..., 0] # (374,32)

    error = loss_fn(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
# Ts = 0.1
# max_len = 150
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=1,
            shuffle=True,
            num_workers=1,
            drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
jerk_set = []
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
            # MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_) # 正常情况(139)
    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)  # 大小为测试数据总数减去发生了碰撞的事件

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = loss_fn(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)

    jerk_set.append(jerk)
# 输出指标
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)

cuda:0
1/5000. loss: 0.001435397503276666
2/5000. loss: 0.0011475549545139074
3/5000. loss: 0.00138047244399786
4/5000. loss: 0.0013754662747184436
5/5000. loss: 0.002124756264189879
6/5000. loss: 0.002204833241800467
7/5000. loss: 0.002136906608939171
8/5000. loss: 0.0021589193493127823
9/5000. loss: 0.001954034281273683
10/5000. loss: 0.0018329080194234848
11/5000. loss: 0.0020344884445269904
12/5000. loss: 0.002271635147432486
13/5000. loss: 0.001988455497970184
14/5000. loss: 0.002149212329337994
15/5000. loss: 0.0012424499727785587
16/5000. loss: 0.0024165508026878038
17/5000. loss: 0.0012527085685481627
18/5000. loss: 0.0013146384929617245
19/5000. loss: 0.002215956337749958
20/5000. loss: 0.0017124880105257034
21/5000. loss: 0.0015735048800706863
22/5000. loss: 0.001778071125348409
23/5000. loss: 0.0012910581814746063
24/5000. loss: 0.0015506517762939136
25/5000. loss: 0.001510181153813998
26/5000. loss: 0.0016583927596608798
27/5000. loss: 0.0020579624300201735
28/5000. loss: 0

In [ ]:
# MAML-跟车  version2   内循环使用adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
import copy


# ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 任务数据集划分
def split_train(data,data_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    if data_ratio < 0:
        # 如果小于零，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
        # 根据索引划分数据集
        data_indices =shuffled_indices[:data_size] # 测试 (1881)
    else:
        # 如果大于等于零，则data_ratio为获取数据集中的数量
        data_indices = shuffled_indices[:data_ratio]
    return data[data_indices]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150
his_horizon = 10

# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len):
        self.data = data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


# 定义模型
class nn_model(nn.Module):
    def __init__(self, input_size = 3):
        super(nn_model,self).__init__()
        self.net=nn.Sequential(OrderedDict([
            ('l1',nn.Linear(input_size,256)),
            ('relu1',nn.ReLU()),
            ('l2',nn.Linear(256,256)),
            ('relu2',nn.ReLU()),
            ('l3',nn.Linear(256,1))
        ]))
        self.state = list(self.net.parameters())
    # 用于元学习的内循环中更新权重的方法，用于计算元损失，然后重置下一个元任务的权重。
    def argforward(self,x,weights): 
        x=F.linear(x,weights[0],weights[1])  # (input,weight,bias)
        x=F.relu(x)
        x=F.linear(x,weights[2],weights[3])
        x=F.relu(x)
        x=F.linear(x,weights[4],weights[5])
        return x


# 元学习内循环训练过程
def inner_train(data_loader, net ,temp_weights, criterion):
    # 初始化变量
    train_loss_his = []  # 训练损失
    best_train_loss = None  # 最佳训练损失
    best_epoch = None  # 最佳验证损失时的轮次
    # 训练过程
    train_losses = []  # 记录每个epoch的训练损失
    net.train()
    # 遍历数据集
    for i, item in enumerate(data_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((20,149)，(20,149)，(20,149))  y_data(20,149)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(20,149)连接，转换成(3,20,149)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(149,20,3)，y->(149,20)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
#         print(T,B,d)  # 149  20  3
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,20)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,20)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = net.argforward(x, temp_weights).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        return loss


# 生成不同数据集的任务类
class DataTask():
    def __init__(self,data_range):
        self.data_range = data_range
  # 生成任务数据，默认生成一个数据点，可以通过指定size参数生成多个数据点
    def sample_data(self,size=1):
        if self.data_range == 1:
            meta_data = HighD_data
            Ts = 0.04
            max_len = 375
#             print('--HighD--')
        elif self.data_range == 2:
            meta_data = Lyft_data
            Ts = 0.1
            max_len = 150
#             print('--Lyft---')
        elif self.data_range == 3:
            meta_data = SPMD_DAS1_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS1')
        elif self.data_range == 4:
            meta_data = SPMD_DAS2_data
            Ts = 0.1
            max_len = 150
#             print('SPMD_DAS2')
        elif self.data_range == 5:
            meta_data = Waymo_data
            Ts = 0.1
            max_len = 150
#             print('--Waymo--')
        meta_data_set = split_train(meta_data, size)
#         print(data_set[0].shape)
        dataset_loader = ImitationCarFolData(meta_data_set, max_len = max_len)
        return dataset_loader


# 任务分布，及选择不同的数据
class DataDistribution():
    def __init__(self, data_range=5, n=3):
        self.data_range = data_range    # 元数据集的数量
        self.n = n  # 每次获取数据集的数量
    # 随机生成一个跟车数据集
    def sample_task(self):
        # 从1到5中随机选择3个不重复的数
        data_choice = np.random.choice(self.data_range, size=self.n, replace=False) + 1
        return data_choice


# MAML元学习训练过程
class MAML():
    def __init__(self, net, alpha, beta, tasks, k):  # (net,alpha=0.01,beta=0.001,tasks=data_tasks,k=100)
        # 初始化 MAML 算法的元参数
        self.net = net  # 元学习使用的神经网络模型
        self.weights = list(net.parameters())  # 获取神经网络的参数
        self.alpha = alpha  # 内循环学习率
        self.beta = beta  # 外循环学习率
        self.tasks = tasks  # 任务的分布
        self.k = k  # 内循环（任务采样）的数据集大小
        self.criterion = nn.MSELoss()  # 均方误差损失函数
        self.meta_optimiser = torch.optim.Adam(self.weights, self.beta)  # 外循环优化器（Adam）
        self.meta_losses = []  # 用于存储元学习的损失历史
        self.plot_every = 1  # 每隔多少次迭代记录一次损失，用于绘图
        self.print_every = 1  # 每隔多少次迭代输出一次损失信息，用于打印
        self.batch_size = self.k

    def inner_loop(self, task):
        # 内循环更新参数，用于计算元学习损失
#         temp_weights = [w.clone() for w in self.weights]  # 复制当前网络参数作为临时参数
        temp_weights = self.weights
        inner_optim = torch.optim.Adam(temp_weights, self.alpha)
        dataset_loader = task.sample_data(size=self.k)  # 从任务中采样数据集 D
        dataloader = DataLoader(
                dataset_loader,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        # 训练过程  第一步
        loss = inner_train(dataloader, self.net, temp_weights, self.criterion) / self.k
        # 优化过程
        inner_optim.zero_grad()
        grads = torch.autograd.grad(loss, temp_weights)  # 计算损失对参数的梯度
        for w, g in zip(temp_weights, grads):
            w.grad = g
        inner_optim.step()
#         temp_weights = [w - self.alpha * g for w, g in zip(temp_weights, grads)]  # 临时参数更新 梯度下降
        # 第二步
        metaloss = inner_train(dataloader, self.net, temp_weights, self.criterion) / self.k
        return metaloss

    def outer_loop(self, num_epochs):  # epoch 5000
        total_loss = 0
        for epoch in tqdm(range(1, num_epochs + 1)):
            metaloss_sum = 0
            tasks = self.tasks.sample_task()  # 从任务分布中采样一个元任务
            for i in tasks:
                task = DataTask(i)
                metaloss = self.inner_loop(task)  # 内循环更新参数，计算元学习损失
                metaloss_sum += metaloss  # mete_loss求和
            self.meta_optimiser.zero_grad()
            metagrads = torch.autograd.grad(metaloss_sum, self.weights)  # 计算元学习损失对参数的梯度
            # 重要步骤：使用元学习梯度更新网络参数
            for w, g in zip(self.weights, metagrads):
                w.grad = g
            self.meta_optimiser.step()  # 使用外循环优化器更新网络参数
            total_loss += metaloss_sum.item() / len(tasks)
            if epoch % self.print_every == 0:
                print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
            if epoch % self.plot_every == 0:
                self.meta_losses.append(total_loss / self.plot_every)
                total_loss = 0

dataset = 'NGSIM'
model_type = 'nn'
save = f'{model_type}_{dataset}.pt' # 保存模型文件
if model_type == 'nn':
    net = nn_model(input_size = his_horizon*3).to(device)
# elif model_type == 'lstm':
#     net = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm
data_tasks=DataDistribution(data_range=5, n=3)
maml=MAML(net,alpha=0.0001,beta=0.0001,tasks=data_tasks,k=150)


maml.outer_loop(num_epochs=5000)
plt.plot(maml.meta_losses)
plt.show()
print('----------------------')


# Train
Ts = 0.1
max_len = 150
og_net = maml.net.net
# 创建一个与原始网络结构相同的虚拟网络
if model_type == 'nn':
    dummy_net = nn.Sequential(OrderedDict([
        ('l1', nn.Linear(his_horizon*3,256)),
        ('relu1', nn.ReLU()),
        ('l2', nn.Linear(256,256)),
        ('relu2', nn.ReLU()),
        ('l3', nn.Linear(256,1))
    ]))
dummy_net=dummy_net.to(device)
# 加载原始网络的权重
dummy_net.load_state_dict(og_net.state_dict())
# 进行迭代，每次更新虚拟网络的参数
num_shots=10
lr = 0.01
loss_fn=nn.MSELoss()
optim=torch.optim.SGD
opt=optim(dummy_net.parameters(),lr=lr)


# 数据集划分
def split_data(data,data_ratio):
    if data_ratio < 0:
        # 如果小于零，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
    else:
        # 如果大于等于零，则data_ratio为获取数据集中的数量
        data_size = data_ratio
    return data[:data_size]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# 获取数据集的数量
K=40
dataset_train = split_data(NGSIM_train, K)
dataset_loader_train = ImitationCarFolData(dataset_train, max_len = max_len)
train_loader = DataLoader(
            dataset_loader_train,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_val = NGSIM_val
dataset_loader_val = ImitationCarFolData(dataset_val, max_len = max_len)
val_loader = DataLoader(
            dataset_loader_val,
            batch_size=10,
            shuffle=True,
            num_workers=1,
            drop_last=True)
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=10,
            shuffle=False,
            num_workers=1,
            drop_last=True)


# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失
best_epoch = None # 最佳验证损失时的轮次

# 训练过程
for epoch in tqdm(range(num_shots)):
    train_losses = [] # 记录每个epoch的训练损失
    validation_losses = [] # 记录每个epoch的验证损失
    jerk_val = 0
    dummy_net.train()
    # 遍历数据集
    for i, item in enumerate(train_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(32,374)连接，转换成(3,32,374)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(374,32,3)，y->(374,32)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            # 根据his_horizon个数据预测加速度
            acc_pre = dummy_net(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = loss_fn(y_pre, y_label)
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
        opt.step()

        train_losses.append(loss.item())
    # 计算本轮平均损失
    train_loss = np.mean(train_losses)

    train_loss_his.append(train_loss)
    print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

    # 验证过程
    dummy_net.eval()
    error_list = []
    for i, item in enumerate(val_loader):
        x_data, y_data = item['inputs'], item['label']
        # Put T into the first dimension, B, T, d -> T, B, d
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        # lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # save a copy of x_data
        x_data_orig = x_data.clone().detach()
        y_label = y_data[his_horizon:]
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
            if model_type == 'nn':
              x = x.transpose(0,1).reshape(B, -1) # flatten for history data
            acc_pre = dummy_net(x).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        validation_loss = loss_fn(y_pre, y_label)
        # jerk_val = np.mean(np.abs(np.diff(torch.tensor(y_pre))/Ts))/batch_size
        # print("Val jerk：", jerk_val)
        opt.zero_grad()
        torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25)
        opt.step()
        validation_losses.append(validation_loss.item())
    # 计算平均验证误差
    mean_validation_error = np.mean(validation_losses)
    train_val_error = (mean_validation_error + train_loss)/2
    if best_validation_loss is None or best_validation_loss > train_val_error:
        best_epoch = epoch + 1
        best_validation_loss = train_val_error
        # save the best model
    with open(save, 'wb') as f:
        torch.save(dummy_net, f)

    validation_error_his.append(mean_validation_error)
    print("Epoch: {0}| Validation error: {1:.7f}".format(epoch + 1, mean_validation_error))

# print(list(dummy_net.parameters()))
print("Epoch:{0}| Best Train+Val Loss：{1:.7f}".format(best_epoch, best_validation_loss))
plt.plot(train_loss_his, label = 'train_loss')
plt.plot(validation_error_his, label = 'val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss (MSE for acceleration prediction)')
plt.show()


# Testing, closed-loop prediction
# Load the best model saved
model = dummy_net
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()


error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device) # x(374,32,3)  y(374,32)
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10,32,3)
        if model_type == 'nn':
            x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测的车距
    spacing_pre = x_data[..., 0] # (374,32)
    # 真实的车距
    spacing_obs = x_data_orig[..., 0] # (374,32)

    error = loss_fn(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
# Ts = 0.1
# max_len = 150
dataset_test = NGSIM_test
dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len)
test_loader = DataLoader(
            dataset_loader_test,
            batch_size=1,
            shuffle=False,
            num_workers=1,
            drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
jerk_set = []
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d)
        if model_type == 'nn':
          x = x.transpose(0,1).reshape(B, -1) # flatten for history data

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
            # MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_) # 正常情况(139)
    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)  # 大小为测试数据总数减去发生了碰撞的事件

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = loss_fn(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)

    jerk_set.append(jerk)
# 输出指标
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)

In [1]:
# 原跟车训练测试过程
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from collections import OrderedDict
import os
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
import logging
from torch.utils.tensorboard import SummaryWriter
import learn2learn as l2l
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from PIL import Image
import warnings
# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=UserWarning)


SEED = 22
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# writer = SummaryWriter("/home/ubuntu/fedavg/FollowNet-car/SummaryWriter/to_img_log")

ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD_DAS2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# car_following_data = HighD_data
# the data shape format (number of car following event, 4 dimension data, lenth of each dimension)数据形状格式（跟车事件的车辆数、4 维数据、各维数据的长度）
# 4 dimension data= [spacing, subject_vehicle_speed, relative_speed, leading_vehicle_speed]4 维数据= [车距、后车车速、相对车速、前车车速］
# print(car_following_data)
# print(car_following_data.shape)   # HighD(12541, 4, 375)

# 保存日志
def get_logger(filename, verbosity=1, name=None):
    # 设置不同verbosity对应的日志级别
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    # 设置日志输出格式
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
    # 创建文件处理器，将日志写入文件
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    # 创建控制台处理器，将日志输出到控制台
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    return logger
# 日志保存路径
# logger = get_logger('/kaggle/working/to_img_main.log')


# split the date into train, validation, test 数据集划分
def split_train(data,test_ratio,val_ratio):
    # np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    # 根据输入百分比（test_ratio）计算测试集大小
    test_set_size=int(len(data)*test_ratio)
    # 根据输入百分比（val_ratio）计算验证集大小
    val_number=int(len(data)*(test_ratio+val_ratio))
    # 根据索引划分数据集
    test_indices =shuffled_indices[:test_set_size] # 测试 (1881)
    val_indices=shuffled_indices[test_set_size:val_number] # 验证 (1881)
    train_indices=shuffled_indices[val_number:] # 训练 (8779)
    return data[train_indices],data[test_indices],data[val_indices]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# Define NN and lstm models
# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150

class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, split: str, max_len = max_len):
        if split == 'train':
            self.data = train_data
        elif split == 'test':
            self.data = test_data
        elif split == 'validation':
            self.data = val_data
        self.max_len = max_len # Max length of a car following event.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / Ts  # (max_len - 1)
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


class CNN(nn.Module):
    def __init__(self, output_size, in_channels=3, hid_dim=60, embedding_size=None):
        super(CNN, self).__init__()

        if embedding_size is None:
            embedding_size = hid_dim

        self.encoder = l2l.vision.models.ConvBase(channels=in_channels, hidden=hid_dim, max_pool=True, layers=3,
                                                  max_pool_factor=1)
#         self.encoder = torch.nn.Sequential(
#             nn.Conv2d(3,8,3,1,1),
#             nn.BatchNorm2d(8),
#             nn.ReLU(),
#             nn.Dropout(0.3)
#         )

        self.features = torch.nn.Sequential(
            self.encoder,
            nn.Dropout(0.2),
            nn.Flatten(),
        )
        self.classifier = torch.nn.Linear(
            embedding_size,
            output_size,
            bias=True,
        )
#         self.classifier.weight.data.normal_()
#         self.classifier.bias.data.mul_(0.0)

        self.hidden_size = hid_dim

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Train
# 获取划分数据集
# (8779,4,375)，(1881,4,375)，(1881,4,375)
# train_data, test_data, val_data = split_train(car_following_data,0.15,0.15)
# train_data, test_data, val_data = split_train(car_following_data,0.8,0.1)
train_data = SPMD1_train
test_data = SPMD1_test
val_data = SPMD1_val
# train_data = NGSIM_train
# test_data = NGSIM_test
# val_data = NGSIM_val
print(train_data.shape, test_data.shape, val_data.shape)
dataset = 'SPMD1'
model_type = 'cnn'
batch_size = 32
total_epochs = 10
# 创建训练集 DataLoader
train_dataset = ImitationCarFolData(split = 'train', max_len = max_len)
train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建验证集 DataLoader
validation_dataset = ImitationCarFolData(split = 'validation', max_len = max_len)
validation_loader = DataLoader(
        validation_dataset ,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1,
        drop_last=True)
# 创建测试集 DataLoader
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=10,
        shuffle=False,
        num_workers=1,
        drop_last=True)

# 使用训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

his_horizon = 10 # number of time steps as history data
lr = 1e-3 # learning rate
# lr = 0.001
mtf = MarkovTransitionField(image_size=his_horizon)
gasf = GramianAngularField(method='summation')
gadf = GramianAngularField(method='difference')

# 定义保存文件的文件夹路径
save_folder = '/kaggle/input/model-state'
save = f'img_{model_type}_{dataset}.pt' # 保存模型文件
# 确保保存文件的文件夹存在，如果不存在，则创建
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
# 定义文件保存路径
save_path = os.path.join(save_folder, save)

# 根据名称定义模型
if model_type == 'cnn':
    model = CNN(output_size = 1).to(device)
model_state = list(model.parameters())
# print(model_state)
# 定义优化器和损失函数
model_optim = optim.Adam(model.parameters(), lr=lr, weight_decay=3e-4)
criterion = nn.MSELoss()

# 初始化变量
train_loss_his = [] # 训练损失
test_error_his = [] # 测试误差
validation_error_his = [] # 验证误差
best_train_loss = None # 最佳训练损失
best_validation_loss = None # 最佳验证损失
best_epoch = None # 最佳验证损失时的轮次

print("----")
# 训练过程
for epoch in tqdm(range(total_epochs)):
    train_losses = [] # 记录每个epoch的训练损失
    validation_losses = [] # 记录每个epoch的验证损失
    model.train()
    # 遍历数据集
    for i, item in enumerate(train_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(32,374)连接，转换成(3,32,374)
        x_data = torch.stack(x_data)
        if model_type == 'cnn':
            space = x_data[0].clone().detach() # (bitch_size,149)
            sv = x_data[1].clone().detach()
            relSpeed = x_data[2].clone().detach()
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(149,bs,3)，y->(149,bs)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
        
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,bs)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,bs)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, bs, 3)
            if model_type == 'cnn':
                a = space[:, frame-his_horizon:frame] # (bs, 10)
                b = sv[:, frame-his_horizon:frame]
                c = relSpeed[:, frame-his_horizon:frame]
                # 进行转换
                # space_change = torch.tensor(mtf.transform(a))
                # sv_change = torch.tensor(mtf.transform(b))
                # relS_change = torch.tensor(mtf.transform(c))
                # space_change = torch.tensor(gasf.transform(a))
                # sv_change = torch.tensor(gasf.transform(b))
                # relS_change = torch.tensor(gasf.transform(c))
                space_change = torch.tensor(gadf.transform(a)) # (bs, 10, 10)
                sv_change = torch.tensor(gadf.transform(b))
                relS_change = torch.tensor(gadf.transform(c))
                x = torch.stack([space_change, sv_change, relS_change], axis=1).float().to(device)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = model(x).squeeze() # (32)
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        model_optim.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
        model_optim.step()

        train_losses.append(loss.item())
    # 计算本轮平均损失
    train_loss = np.mean(train_losses)

    train_loss_his.append(train_loss)
    print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

    # 验证过程
    model.eval()
    error_list = []
    for i, item in enumerate(validation_loader):
        x_data, y_data = item['inputs'], item['label']
        # Put T into the first dimension, B, T, d -> T, B, d
        x_data = torch.stack(x_data)
        if model_type == 'cnn':
            space = x_data[0].clone().detach() # (bitch_size,149)
            sv = x_data[1].clone().detach()
            relSpeed = x_data[2].clone().detach()
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        # x->(149,bs,3)，y->(149,bs)
        x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
        T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
        # save a copy of x_data
        x_data_orig = x_data.clone().detach()
        y_label = y_data[his_horizon:]
        for frame in range(his_horizon, T):
            x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'cnn':
                a = space[:, frame-his_horizon:frame] # (bs, 10)
                b = sv[:, frame-his_horizon:frame]
                c = relSpeed[:, frame-his_horizon:frame]
                # 进行转换
                # space_change = torch.tensor(mtf.transform(a))
                # sv_change = torch.tensor(mtf.transform(b))
                # relS_change = torch.tensor(mtf.transform(c))
                # space_change = torch.tensor(gasf.transform(a))
                # sv_change = torch.tensor(gasf.transform(b))
                # relS_change = torch.tensor(gasf.transform(c))
                space_change = torch.tensor(gadf.transform(a)) # (bs, 10, 10)
                sv_change = torch.tensor(gadf.transform(b))
                relS_change = torch.tensor(gadf.transform(c))
                x = torch.stack([space_change, sv_change, relS_change], axis=1).float().to(device)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            acc_pre = model(x).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        validation_loss = criterion(y_pre, y_label)
        model_optim.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        model_optim.step()
        validation_losses.append(validation_loss.item())
    # 计算平均验证误差
    mean_validation_error = np.mean(validation_losses)
    val_error = mean_validation_error
    if best_validation_loss is None or best_validation_loss > val_error:
        best_epoch = epoch + 1
        best_validation_loss = val_error
        # save the best model
        with open(save, 'wb') as f:
            torch.save(model, f)

    validation_error_his.append(mean_validation_error)
    print("Epoch: {0}| Validation error: {1:.7f}".format(epoch + 1, mean_validation_error))

print("Epoch:{0}| Best Val Loss：{1:.7f}".format(best_epoch, best_validation_loss))
# plt_path = "/home/ubuntu/fedavg/FollowNet-car/test_metrics"
plt.plot(train_loss_his, label = 'train_loss')
plt.plot(validation_error_his, label = 'val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss (MSE for acceleration prediction)')
# plt.savefig(os.path.join(plt_path, 'img_train_val_loss.png'))
plt.show()
plt.close()


# 用该数据集训练得到的模型去对其他数据集进行测试
# Ts = 0.04
# max_len = 375
# # car_following_data = NGSIM_data
# # print(car_following_data.shape)
# # train_data, test_data, val_data = split_train(car_following_data,0.15,0.15)
# test_data = HighD_test
# # 创建测试集 DataLoader
# test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
# test_loader = DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=1,
#         drop_last=True)


# Testing, closed-loop prediction
# Load the best model saved
with open(f'{save}', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

error_list = []

for i, item in enumerate(test_loader):
    x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data) # (3,32,374)
    if model_type == 'cnn':
        space = x_data[0].clone().detach() # (bitch_size,149)
        sv = x_data[1].clone().detach()
        relSpeed = x_data[2].clone().detach()
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    # x->(149,bs,3)，y->(149,bs)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
    # print(T,B,d)  # 149  20  3
    # 获取前车速度
    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1) # (375,32)

    # save a copy of x_data
    x_data_orig = x_data.clone().detach() # (374,32,3)

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
        if model_type == 'cnn':
            a = space[:, frame-his_horizon:frame] # (bs, 10)
            b = sv[:, frame-his_horizon:frame]
            c = relSpeed[:, frame-his_horizon:frame]
            # 进行转换
            # space_change = torch.tensor(mtf.transform(a))
            # sv_change = torch.tensor(mtf.transform(b))
            # relS_change = torch.tensor(mtf.transform(c))
            # space_change = torch.tensor(gasf.transform(a))
            # sv_change = torch.tensor(gasf.transform(b))
            # relS_change = torch.tensor(gasf.transform(c))
            space_change = torch.tensor(gadf.transform(a)) # (bs, 10, 10)
            sv_change = torch.tensor(gadf.transform(b))
            relS_change = torch.tensor(gadf.transform(c))
            x = torch.stack([space_change, sv_change, relS_change], axis=1).float().to(device)
        if model_type == 'nn':
            x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)

        acc_pre = model(x).squeeze() # (32,1)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            # 根据当前速度和加速度计算下一时间速度
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts # (32)
            # 计算下一时间速度的相对速度
            delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
            # 该时刻真实相对速度
            delta_v = x_data[frame, :, -1] # (32)
            # 通过两车车距加上相对位移得到下一时间段车距 ？
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2 # (32)

            # update 根据计算得到的值，更新下一时间的值
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]

    error = criterion(spacing_pre, spacing_obs).item()
    error_list.append(error)
model.train()
# 计算MSE - 间距均方误差（Mean square error of spacing）
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)


# plot for one event
event_no = 2
plt.plot(spacing_obs[:, event_no].cpu().detach().numpy(), label = 'GT')
plt.plot(spacing_pre[:, event_no].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('Spacing (m)')
# plt.savefig(os.path.join(plt_path, 'img_Spacing.png'))
plt.show()
plt.close()

# speed part
plt.figure()
plt.plot(x_data_orig[:, event_no, 1].cpu().detach().numpy(), label = 'GT')
plt.plot(x_data[:, event_no, 1].cpu().detach().numpy(), '--', label = 'prediction')
plt.legend()
plt.xlabel(f'Time ({Ts}s)')
plt.ylabel('SV speed (m/s)')
# plt.savefig(os.path.join(plt_path, 'img_Speed.png'))
plt.show()
plt.close()


# Updated for the four key elvaluation metrics.  Note that: The loss design in FollowNet is for the reduction of the mse of spacing.
# Please note that The TTC results presented in the table of the paper indicate the average TTC value calculated.
# However, a drawback of this approach is that if the relative velocity in just one frame is extremely small, it will result in a significantly large computed TTC when reproducing paper.
# To address this issue, it may be beneficial to consider plotting the distribution of TTC as a way to present the results.
# 间距均方误差（Mean square error of spacing (MSE))
# 碰撞率（Collision rate）
# 驾驶舒适度指标(颠簸)（Driving comfort metric (jerk)）
# 碰撞时间(TTC)指标（Time-To-Collision (TTC) metric）
# for test
# 创建测试集 DataLoader
# Ts = 0.04
# max_len = 375
batch_size = 1
test_dataset = ImitationCarFolData(split = 'test', max_len = max_len)
test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=1,
        drop_last=True)
# 记录跟车事件数和发生碰撞事件数
count = 0
col = 0
min_error = 10000
min_index = 0
max_error = 0
max_index = 0
jerk_set = [] 
# TTC_set = []
minimum_ttc_set = []
model.eval()
# ttc = [1, 2, 3, 4, 5, 6, 7, 8, -1, -3, 7]
def calculate_safety(ttc):
    # Remove any negative values (assuming TTC cannot be negative)
    # ttc = [x for x in ttc if x >= 0]
    # 计算ttc中最小值
    minimum_ttc = min(ttc)
    # 计算ttc中第1百分位的值
    first_percentile_ttc = np.percentile(ttc, 1)
    # print(1111)
    return minimum_ttc, first_percentile_ttc

# # Example usage
# minimum_ttc, first_percentile_ttc = calculate_safety(ttc)
# print("Minimum TTC:", minimum_ttc)
# print("1st Percentile TTC:", first_percentile_ttc)

for i, item in tqdm(enumerate(test_loader)):

    # jerk
    # jerk = np.diff(sv)/Ts
    acc_single = []
    TTC_single = []

    count += 1
    x_data, y_data = item['inputs'], item['label']
    # Put T into the first dimension, B, T, d -> T, B, d
    x_data = torch.stack(x_data)
    if model_type == 'cnn':
        space = x_data[0].clone().detach() # (bitch_size,149)
        sv = x_data[1].clone().detach()
        relSpeed = x_data[2].clone().detach()
    # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
    # x->(149,bs,3)，y->(149,bs)
    x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
    T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
    # print(T,B,d)  # 149  20  3

    lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)

    # save a copy of x_data
    x_data_orig = x_data.clone().detach()

    for frame in range(his_horizon, T):
        x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
        if model_type == 'cnn':
            a = space[:, frame-his_horizon:frame] # (bs, 10)
            b = sv[:, frame-his_horizon:frame]
            c = relSpeed[:, frame-his_horizon:frame]
            # 进行转换
            # space_change = torch.tensor(mtf.transform(a))
            # sv_change = torch.tensor(mtf.transform(b))
            # relS_change = torch.tensor(mtf.transform(c))
            # space_change = torch.tensor(gasf.transform(a))
            # sv_change = torch.tensor(gasf.transform(b))
            # relS_change = torch.tensor(gasf.transform(c))
            space_change = torch.tensor(gadf.transform(a)) # (bs, 10, 10)
            sv_change = torch.tensor(gadf.transform(b))
            relS_change = torch.tensor(gadf.transform(c))
            x = torch.stack([space_change, sv_change, relS_change], axis=1).float().to(device)
        if model_type == 'nn':
            x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)

        acc_pre = model(x).squeeze()

        acc_single.append(acc_pre)

        # update next data
        # 根据前一次计算得到的值，计算后面的值
        if frame < T-1:
            sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts
            # print(sv_spd_)
            # MyDevice = torch.device('cuda:0')
            # 速度取值大于0.001的
            sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
            delta_v_ = lv_spd[frame + 1] - sv_spd_
            delta_v = x_data[frame, :, -1]
            spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2

            spacing = np.array(spacing_.cpu())
            # 如果spacing<=0，表示发生碰撞
            if float(np.array(spacing_.cpu())) <= 0:
                col += 1
                break
            next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
            x_data[frame + 1] = next_frame_data
        # 计算TTC
        TTC_single.append(-spacing_ / delta_v_) # 正常情况(139)
    # Mean absolute jerk
    jerk =np.mean(np.abs(np.diff(torch.tensor(acc_single))/Ts))

    # if np.sum(torch.tensor(TTC_single).cuda().cpu().numpy()>0) == 0:
    #     continue
    TTC_single = torch.tensor(TTC_single).to(device).cpu().numpy()
    TTC_single = [x for x in TTC_single if x >= 0]
    if len(TTC_single) > 0:
        minimum_ttc, _ = calculate_safety(TTC_single)
        minimum_ttc_set.append(minimum_ttc)  # 大小为测试数据总数减去发生了碰撞的事件

    # Calculating spacing error for the closed-loop simulation
    # 预测车距
    spacing_pre = x_data[..., 0]
    # 真实车距
    spacing_obs = x_data_orig[..., 0]
    # for index, (space_obs, space_pre) in enumerate(zip(spacing_obs[:, 0].cpu().detach().numpy(), spacing_pre[:, 0].cpu().detach().numpy())):
    #     writer.add_scalars('NGSIM:{}/Spacing'.format(count), {'GT': space_obs, 'prediction': space_pre}, global_step=index)
    # for index, (speed_obs, speed_pre) in enumerate(zip(x_data_orig[:, 0, 1].cpu().detach().numpy(), x_data[:, 0, 1].cpu().detach().numpy())):
    #     writer.add_scalars('NGSIM:{}/Speed'.format(count), {'GT': speed_obs, 'prediction': speed_pre}, global_step=index)
    # writer.close()

    # plt.plot(np.array(spacing_obs.cpu()), label = 'GT')
    # plt.plot(np.array(spacing_pre.cpu()), '--', label = 'Prediction')
    # plt.legend()
    # plt.xlabel(f'Time ({Ts}s)');
    # plt.ylabel('Spacing (m)');
    # plt.show()
    error = criterion(spacing_pre[:(frame+1)], spacing_obs[:(frame+1)]).item()
    error_list.append(error)
    if error > max_error:
        max_error = error
        max_index = count
    if error < min_error:
        min_error = error
        min_index = count

    jerk_set.append(jerk)
# 输出指标
print("index（{}）min_error：{}, index（{}）max_error：{}".format(min_index, min_error, max_index, max_error))
# logger.info("index（{}）min_error：{}, index（{}）max_error：{}".format(min_index, min_error, max_index, max_error))
print("count=",count,"col=",col,"rate",col/count,"jerk", np.mean(jerk_set), "miniumu_ttc", np.mean(minimum_ttc_set))
# logger.info("count={}，col={}，rate={:.5f}%".format(count, col, col/count*100))
# logger.info("jerk={:.5f}，miniumu_ttc={:.5f}".format(np.mean(jerk_set), np.mean(minimum_ttc_set)))
model.train()
mean_test_error = np.mean(error_list)
print('mean_test_error',mean_test_error)
# logger.info("mean_test_error：{:.5f}".format(mean_test_error))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'learn2learn'

In [ ]:
import numbers
from copy import copy

import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import random


def extract_top_level_dict(current_dict):
    """
    Builds a graph dictionary from the passed depth_keys, value pair. Useful for dynamically passing external params
    :param depth_keys: A list of strings making up the name of a variable. Used to make a graph for that params tree.
    :param value: Param value
    :param key_exists: If none then assume new dict, else load existing dict and add new key->value pairs to it.
    :return: A dictionary graph of the params already added to the graph.
    """
    output_dict = dict()
    for key in current_dict.keys():
        name = key.replace("layer_dict.", "")
        name = name.replace("layer_dict.", "")
        name = name.replace("block_dict.", "")
        name = name.replace("module-", "")
        top_level = name.split(".")[0]
        sub_level = ".".join(name.split(".")[1:])

        if top_level not in output_dict:
            if sub_level == "":
                output_dict[top_level] = current_dict[key]
            else:
                output_dict[top_level] = {sub_level: current_dict[key]}
        else:
            new_item = {key: value for key, value in output_dict[top_level].items()}
            new_item[sub_level] = current_dict[key]
            output_dict[top_level] = new_item

    #print(current_dict.keys(), output_dict.keys())
    return output_dict


class MetaLinearLayer(nn.Module):
    def __init__(self, input_shape, num_filters, use_bias):
        """
        A MetaLinear layer. Applies the same functionality of a standard linearlayer with the added functionality of
        being able to receive a parameter dictionary at the forward pass which allows the convolution to use external
        weights instead of the internal ones stored in the linear layer. Useful for inner loop optimization in the meta
        learning setting.
        :param input_shape: The shape of the input data, in the form (b, f)
        :param num_filters: Number of output filters
        :param use_bias: Whether to use biases or not.
        """
        super(MetaLinearLayer, self).__init__()
        b, c = input_shape

        self.use_bias = use_bias
        self.weights = nn.Parameter(torch.ones(num_filters, c))
        nn.init.xavier_uniform_(self.weights)
        if self.use_bias:
            self.bias = nn.Parameter(torch.zeros(num_filters))

    def forward(self, x, params=None):
        """
        Forward propagates by applying a linear function (Wx + b). If params are none then internal params are used.
        Otherwise passed params will be used to execute the function.
        :param x: Input data batch, in the form (b, f)
        :param params: A dictionary containing 'weights' and 'bias'. If params are none then internal params are used.
        Otherwise the external are used.
        :return: The result of the linear function.
        """
        if params is not None:
            params = extract_top_level_dict(current_dict=params)
            if self.use_bias:
                (weight, bias) = params["weights"], params["bias"]
            else:
                (weight) = params["weights"]
                bias = None
        else:
            pass
            #print('no inner loop params', self)

            if self.use_bias:
                weight, bias = self.weights, self.bias
            else:
                weight = self.weights
                bias = None
        # print(x.shape)
        out = F.linear(input=x, weight=weight, bias=bias)
        return out


class MetaStepLossNetwork(nn.Module):
    def __init__(self, input_dim, notspi, device):
        super(MetaStepLossNetwork, self).__init__()

        self.device = device
        # self.args = args
        self.input_dim = input_dim
        self.input_shape = (1, input_dim)

        self.build_network()
        # print("meta network params")
        # for name, param in self.named_parameters():
        #     print(name, param.shape)

    def build_network(self):
        """ 
        Builds the network before inference is required by creating some dummy inputs with the same input as the
        self.im_shape tuple. Then passes that through the network and dynamically computes input shapes and
        sets output shapes for each layer.
        """
        x = torch.zeros(self.input_shape)
        out = x

        self.linear1 = MetaLinearLayer(input_shape=self.input_shape,
                                                    num_filters=self.input_dim, use_bias=True)

        self.linear2 = MetaLinearLayer(input_shape=(1, self.input_dim),
                                                    num_filters=1, use_bias=True)


        out = self.linear1(out)
        out = F.relu_(out)
        out = self.linear2(out)

    def forward(self, x, params=None):
        """                                  
        Forward propages through the network. If any params are passed then they are used instead of stored params.
        :param x: Input image batch.
        :param num_step: The current inner loop step number
        :param params: If params are None then internal parameters are used. If params are a dictionary with keys the
         same as the layer names then they will be used instead.
        :param training: Whether this is training (True) or eval time.
        :param backup_running_statistics: Whether to backup the running statistics in their backup store. Which is
        then used to reset the stats back to a previous state (usually after an eval loop, when we want to throw away stored statistics)
        :return: Logits of shape b, num_output_classes.
        """ 

        linear1_params = None
        linear2_params = None

        if params is not None:
            params = extract_top_level_dict(current_dict=params)

            linear1_params = params['linear1']
            linear2_params = params['linear2']

        out = x
        
        out = self.linear1(out, linear1_params)
        out = F.relu_(out)
        out = self.linear2(out, linear2_params)

        return out

    def zero_grad(self, params=None):
        if params is None:
            for param in self.parameters():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
        else:
            for name, param in params.items():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
                            params[name].grad = None

    def restore_backup_stats(self):
        """
        Reset stored batch statistics from the stored backup.
        """
        for i in range(self.num_stages):
            self.layer_dict['conv{}'.format(i)].restore_backup_stats()


class MetaLossNetwork(nn.Module):
    def __init__(self, input_dim, notspi, device):
        """
        Builds a multilayer convolutional network. It also provides functionality for passing external parameters to be
        used at inference time. Enables inner loop optimization readily.
        :param im_shape: The input image batch shape.
        :param num_output_classes: The number of output classes of the network.
        :param args: A named tuple containing the system's hyperparameters.
        :param device: The device to run this on.
        :param meta_classifier: A flag indicating whether the system's meta-learning (inner-loop) functionalities should
        be enabled. 
        """
        super(MetaLossNetwork, self).__init__()
        
        self.device = device
        # self.args = args
        self.notspi = notspi
        self.input_dim = input_dim
        self.input_shape = (1, input_dim)

        self.num_steps = notspi # number of inner-loop steps

        self.build_network()
        # print("meta network params")
        # for name, param in self.named_parameters():
        #     print(name, param.shape)

    def build_network(self):
        """ 
        Builds the network before inference is required by creating some dummy inputs with the same input as the
        self.im_shape tuple. Then passes that through the network and dynamically computes input shapes and
        sets output shapes for each layer.
        """
        x = torch.zeros(self.input_shape)
        self.layer_dict = nn.ModuleDict()
        # ф
        for i in range(self.num_steps): 
            self.layer_dict['step{}'.format(i)] = MetaStepLossNetwork(self.input_dim, notspi=self.notspi, device=self.device)

            out = self.layer_dict['step{}'.format(i)](x)

    def forward(self, x, num_step, params=None):
        """                                  
        Forward propages through the network. If any params are passed then they are used instead of stored params.
        :param x: Input image batch.
        :param num_step: The current inner loop step number
        :param params: If params are None then internal parameters are used. If params are a dictionary with keys the
         same as the layer names then they will be used instead.
        :param training: Whether this is training (True) or eval time.
        :param backup_running_statistics: Whether to backup the running statistics in their backup store. Which is
        then used to reset the stats back to a previous state (usually after an eval loop, when we want to throw away stored statistics)
        :return: Logits of shape b, num_output_classes.
        """ 
        param_dict = dict()

        if params is not None: 
            # params = {key: value[0] for key, value in params.items()}
            param_dict = extract_top_level_dict(current_dict=params)
            
        for name, param in self.layer_dict.named_parameters():
            path_bits = name.split(".")
            layer_name = path_bits[0]
            if layer_name not in param_dict:
                param_dict[layer_name] = None

            
        out = x
        
        out = self.layer_dict['step{}'.format(num_step)](out, param_dict['step{}'.format(num_step)])

        return out

    def zero_grad(self, params=None):
        if params is None:
            for param in self.parameters():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
        else:
            for name, param in params.items():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
                            params[name].grad = None

    def restore_backup_stats(self):
        """
        Reset stored batch statistics from the stored backup.
        """
        for i in range(self.num_stages):
            self.layer_dict['conv{}'.format(i)].restore_backup_stats()


class StepLossAdapter(nn.Module):
    def __init__(self, input_dim, num_loss_net_layers, notspi, device):
        super(StepLossAdapter, self).__init__()

        self.device = device
        # self.args = args
        output_dim = num_loss_net_layers * 2 * 2 # 2 for weight and bias, another 2 for multiplier and offset

        self.linear1 = nn.Linear(input_dim, input_dim)
        self.activation = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(input_dim, output_dim)

        self.multiplier_bias = nn.Parameter(torch.zeros(output_dim // 2))
        self.offset_bias = nn.Parameter(torch.zeros(output_dim // 2))

    def forward(self, task_state, num_step, loss_params):
        # ψ
        out = self.linear1(task_state)
        out = F.relu_(out)
        out = self.linear2(out)

        generated_multiplier, generated_offset = torch.chunk(out, chunks=2, dim=-1)

        i = 0
        updated_loss_weights = dict()
        for key, val in loss_params.items():
            if 'step{}'.format(num_step) in key:
                updated_loss_weights[key] = (1 + self.multiplier_bias[i] * generated_multiplier[i]) * val + \
                                             self.offset_bias[i] * generated_offset[i]
                i+=1

        return updated_loss_weights


class LossAdapter(nn.Module):
    def __init__(self, input_dim, num_loss_net_layers, notspi, device):
        super(LossAdapter, self).__init__()

        self.device = device
        # self.args = args

        self.num_steps = notspi # number of inner-loop steps

        self.loss_adapter = nn.ModuleList()
        for i in range(self.num_steps): 
            self.loss_adapter.append(StepLossAdapter(input_dim, num_loss_net_layers, notspi=notspi, device=device))

    def forward(self, task_state, num_step, loss_params):
        return self.loss_adapter[num_step](task_state, num_step, loss_params)



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torch import _weight_norm
import numpy as np
from math import sqrt
import random

    
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, input_dim, num_heads=4):
        super(MultiHeadSelfAttention, self).__init__()
        self.dim_in = input_dim
        self.dim_k = input_dim//2
        self.dim_v = input_dim//2
        self.num_heads = num_heads
        self.linear_q = nn.Linear(input_dim, input_dim//2, bias=True)
        self.linear_k = nn.Linear(input_dim, input_dim//2, bias=True)
        self.linear_v = nn.Linear(input_dim, input_dim, bias=True)
        self._norm_fact = 1 / sqrt((input_dim//2) // num_heads)

    def forward(self, x, param=None):
        batch, n, dim_in = x.shape
        assert dim_in == self.dim_in

        nh = self.num_heads
        dk = self.dim_k // nh  # dim_k of each head
        dv = self.dim_v // nh  # dim_v of each head
        if param == None:
            q = self.linear_q(x).reshape(batch, n, nh, dk).transpose(1, 2)  # (batch, nh, n, dk)
            k = self.linear_k(x).reshape(batch, n, nh, dk).transpose(1, 2)  # (batch, nh, n, dk)
            v = self.linear_v(x).reshape(batch, n, nh, dv*2).transpose(1, 2)  # (batch, nh, n, dv)
        else:
            q = F.linear(x, weight=param[0], bias=param[1]).reshape(batch, n, nh, dk).transpose(1, 2)
            k = F.linear(x, weight=param[2], bias=param[3]).reshape(batch, n, nh, dk).transpose(1, 2)
            v = F.linear(x, weight=param[4], bias=param[5]).reshape(batch, n, nh, dv*2).transpose(1, 2)
        dist = torch.matmul(q, k.transpose(2, 3)) * self._norm_fact  # batch, nh, n, n
        dist = torch.softmax(dist, dim=-1)  # batch, nh, n, n

        att = torch.matmul(dist, v)  # batch, nh, n, dv
        out = att.transpose(1, 2).reshape(batch, n, self.dim_in)  # batch, n, dim_v
        
        # out = att.reshape(att.shape[0], -1)

        return out
    

class conv1d(nn.Module):
    def _init_(self):
        super(conv1d, self).__init__()
    
    def forward(self, x, weight, bias, stride, padding, dilation):
        return F.conv1d(x, weight, bias=bias, stride=stride, padding=padding, dilation=dilation)

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
        #                                    stride=stride, padding=padding, dilation=dilation)
        self.batchnorm1 = nn.BatchNorm1d(n_outputs)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        # self.relu1 = nn.Tanh()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
        #                                    stride=stride, padding=padding, dilation=dilation)
        self.batchnorm2 = nn.BatchNorm1d(n_outputs)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        # self.relu2 = nn.Tanh()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.batchnorm1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.batchnorm2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        # self.relu = nn.Tanh()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=5, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.num_inputs = num_inputs
        self.num_channels = num_channels
        layers = []
        # self.vars_bn = nn.ParameterList()
        num_levels = len(num_channels)
        # num_levels = num_channels
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.token = nn.Parameter(torch.ones(1, num_channels[-1]))
        # self.encoder_layer = nn.TransformerEncoderLayer(d_model=10, nhead= 5,\
        #                     dim_feedforward=10, batch_first=True)
        # self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        # self.features = nn.LSTM(input_size=num_channels[-1], hidden_size=16, num_layers=1, batch_first=True)
        # self.features = nn.GRU(input_size=num_channels[-1], hidden_size=16, num_layers=1, batch_first=True)
        # self.attention = TemporalAttention(num_channels[-1])
        self.attention = MultiHeadSelfAttention(num_channels[-1])
        self.mlp = nn.Sequential(
            nn.Linear(num_channels[-1], num_channels[-1]),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.ReLU(), 
            nn.Linear(num_channels[-1], 1)
        )
        # self.features = nn.Linear(num_channels[-1], num_channels[-1])
        # self.decoder = nn.Linear(num_channels[-1], 1)

        # encoder_layer = nn.TransformerEncoderLayer(d_model=num_channels[i-1], nhead= 4,\
        #                     dim_feedforward=num_channels[i-1], batch_first=False)
        # self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
        # self.tanh = nn.Tanh()

    def forward(self, x, weights=None):
        device =  x[0].device
        if weights == None:
            # transformer output:
            # x = self.transformer_encoder(x) + x
            
            output = self.network(x)
            output = output.transpose(1, 2)
            result = torch.cat([output, self.token.unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result)
            output = output[:, -1, :]
            # output = self.attention(output)
            # linear output:
            # output_shape = output.shape
            # output = output.reshape(output_shape[0], output_shape[1] * output_shape[2])
            # h_n = self.features(output)
            
            # LSTM output:
            # enc_x, (h_n, c_n) = self.features(output)
            
            # GRU output:
            # enc_x, h_n = self.features(output)

            # if len(h_n.shape) == 3:
            #     h_n = h_n[-1] # (32,16)
            
            # 通过线性层和激活函数得到最终输出
            # out = self.decoder(F.relu(h_n))
            # out = self.decoder(F.leaky_relu(h_n, negative_slope=0.01))
            out = self.mlp(output)
        else:
            # TemporalBlock0
            # 实际权重 = 模数 * 方向向量
            ks = self.kernel_size
            weight1 = _weight_norm(weights[3], weights[2], 0)
            conv1d1 = weight_norm(nn.Conv1d(self.num_inputs, self.num_channels[0], kernel_size=ks, stride=1, padding=(ks-1)*1, dilation=1))
            output1 = conv1d1._conv_forward(x, weight=weight1, bias=weights[1])
            # output1 = F.conv1d(x, weight=weights[1], bias=weights[2], stride=1, padding=(ks-1)*1, dilation=1)
            running_mean1 = nn.Parameter(torch.zeros(self.num_channels[0]), requires_grad=False).to(device)
            running_var1 = nn.Parameter(torch.ones(self.num_channels[0]), requires_grad=False).to(device)
            output1 = F.batch_norm(self.network[0].chomp1(output1), running_mean=running_mean1, running_var=running_var1, weight=weights[4], bias=weights[5])
            # output1 = F.tanh(output1)
            output1 = F.relu(output1)
            output1 = F.dropout(output1, self.dropout)
            weight2 = _weight_norm(weights[8], weights[7], 0)
            conv1d2 = weight_norm(nn.Conv1d(self.num_channels[0], self.num_channels[0], kernel_size=ks, stride=1, padding=(ks-1)*1, dilation=1))
            output1 = conv1d2._conv_forward(output1, weight=weight2, bias=weights[6])
            # output1 = F.conv1d(output1, weight=weights[5], bias=weights[6], stride=1, padding=(ks-1)*1, dilation=1)
            running_mean2 = nn.Parameter(torch.zeros(self.num_channels[0]), requires_grad=False).to(device)
            running_var2 = nn.Parameter(torch.ones(self.num_channels[0]), requires_grad=False).to(device)
            output1 = F.batch_norm(self.network[0].chomp2(output1), running_mean=running_mean2, running_var=running_var2, weight=weights[9], bias=weights[10])
            # output1 = F.tanh(output1)
            output1 = F.relu(output1)
            output1 = F.dropout(output1, self.dropout)
            res1 = F.conv1d(x, weights[11], bias=weights[12], stride=1)
            # output1 = F.tanh(output1 + res1)
            output1 = F.relu(output1 + res1)

            # TemporalBlock1
            weight3 = _weight_norm(weights[15], weights[14], 0)
            conv1d3 = weight_norm(nn.Conv1d(self.num_channels[0], self.num_channels[1], kernel_size=ks, stride=1, padding=(ks-1)*2, dilation=2))
            output2 = conv1d3._conv_forward(output1, weight=weight3, bias=weights[13])
            # output2 = F.conv1d(output1, weight=weights[11], bias=weights[12], stride=1, padding=(ks-1)*2, dilation=2)
            running_mean3 = nn.Parameter(torch.zeros(self.num_channels[1]), requires_grad=False).to(device)
            running_var3 = nn.Parameter(torch.ones(self.num_channels[1]), requires_grad=False).to(device)
            output2 = F.batch_norm(self.network[1].chomp1(output2), running_mean=running_mean3, running_var=running_var3, weight=weights[16], bias=weights[17])
            # output2 = F.tanh(output2)
            output2 = F.relu(output2)
            output2 = F.dropout(output2, self.dropout)
            weight4 = _weight_norm(weights[20], weights[19], 0)
            conv1d4 = weight_norm(nn.Conv1d(self.num_channels[1], self.num_channels[1], kernel_size=ks, stride=1, padding=(ks-1)*2, dilation=2))
            output2 = conv1d4._conv_forward(output2, weight=weight4, bias=weights[18])
            # output2 = F.conv1d(output2, weight=weights[15], bias=weights[16], stride=1, padding=(ks-1)*2, dilation=2)
            running_mean4 = nn.Parameter(torch.zeros(self.num_channels[1]), requires_grad=False).to(device)
            running_var4 = nn.Parameter(torch.ones(self.num_channels[1]), requires_grad=False).to(device)
            output2 = F.batch_norm(self.network[1].chomp2(output2), running_mean=running_mean4, running_var=running_var4, weight=weights[21], bias=weights[22])
            # output2 = F.tanh(output2)
            output2 = F.relu(output2)
            output2 = F.dropout(output2, self.dropout)
            res2 = F.conv1d(output1, weights[23], bias=weights[24], stride=1)
            # output2 = F.tanh(output2 + res2)
            output = F.relu(output2 + res2)

            output = output.transpose(1, 2)
            result = torch.cat([output, weights[0].unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result, weights[25:31])
            output = output[:, -1, :]

            # output = output.transpose(1, 2)
            # output_shape = output.shape
            # output = output.reshape(output_shape[0], output_shape[1] * output_shape[2])

            # out = F.leaky_relu(F.linear(output, weight=weights[31] , bias=weights[32]), negative_slope=0.01)
            out = F.relu(F.linear(output, weight=weights[31] , bias=weights[32]))
            out = F.linear(out, weight=weights[33], bias=weights[34])

        return out
    

# MAML-跟车  version1
# %matplotlib inline
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import partial
import copy
import random
import logging
from torch.utils.tensorboard import SummaryWriter
# from net.TCN import TemporalConvNet
# from net.TCN_wn_bn import TemporalConvNet
# from net.TCN_layer import TemporalConvNet
# from net.TCN import MultiHeadSelfAttention
# from net.cbrd import cbrd
# from net.meta_neural_network_architectures import MetaLossNetwork, LossAdapter

# 设置随机种子
SEED = 22
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# writer = SummaryWriter("/home/ubuntu/fedavg/FollowNet-car/SummaryWriter/MAML_alfa_log")

# ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# 使用训练设备
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
gpu = 0
device = torch.device('cuda:{}'.format(gpu) if torch.cuda.is_available() and gpu != -1 else 'cpu')
print(device)


# # 保存日志
# def get_logger(filename, verbosity=1, name=None):
#     # 设置不同verbosity对应的日志级别
#     level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
#     # 设置日志输出格式
#     formatter = logging.Formatter(
#         "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
#     )
#     logger = logging.getLogger(name)
#     logger.setLevel(level_dict[verbosity])
#     # 创建文件处理器，将日志写入文件
#     fh = logging.FileHandler(filename, "w")
#     fh.setFormatter(formatter)
#     logger.addHandler(fh)
#     # 创建控制台处理器，将日志输出到控制台
#     sh = logging.StreamHandler()
#     sh.setFormatter(formatter)
#     logger.addHandler(sh)
#     return logger
# # 日志保存路径
# logger = get_logger('/home/ubuntu/fedavg/FollowNet-car/log/MAML_alfa_oooollllldddd.log')

# 任务数据集划分
def split_train(data,data_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    if data_ratio > 0 and data_ratio <= 1:
        # 如果小于等于1，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
        data_num = int(data_size/2)
        # 根据索引划分数据集
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_size]   # query set
    else:
        # 如果大于1，则data_ratio为获取数据集中的数量
        data_num = int(data_ratio/2)
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_ratio]   # query set
    return data[data_indices1], data[data_indices2]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150
his_horizon = 10


# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len, Ts = 0.1):
        self.data = data
        self.max_len = max_len # Max length of a car following event.
        self.Ts = Ts

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / self.Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


# nn模型
class nn_model(nn.Module):
    def __init__(self, input_size = 3):
        super(nn_model,self).__init__()
        self.net=nn.Sequential(
                nn.Linear(input_size,256),
                nn.ReLU(),
                nn.Linear(256,256),
                nn.ReLU(),
                nn.Linear(256,1)
            )
        self.input_size = input_size
        self.state = list(self.net.parameters())
    # 用于元学习的内循环中更新权重的方法，用于计算元损失，然后重置下一个元任务的权重。
    def forward(self,x,weights=None):
        if weights == None:
            x = self.net(x)
        else:
            x=F.linear(x,weights[0],weights[1])  # (input,weight,bias)
            x=F.relu(x)
            x=F.linear(x,weights[2],weights[3])
            x=F.relu(x)
            x=F.linear(x,weights[4],weights[5])
        return x


# 计算ttc
def calculate_safety(ttc):
    minimum_ttc = min(ttc)
    return minimum_ttc
# 多数据同时验证
def model_evaluate(model,his_horizon,testdata):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    model.eval()

    jerk_set = torch.empty(0).to(device)
    error_set = torch.empty(0).to(device)
    minimum_ttc_set = torch.empty(0).to(device)

    criterion = nn.MSELoss()
    count = 0
    col = 0
    for i, item in enumerate(testdata):#每个样本挨个出来
        
        x_data, y_data = item['inputs'], item['label']
        x_data = torch.stack(x_data)#3x149
        if model_type == 'cnn1d' or model_type == 'tcn':
            # x->(bs,3,149)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
            B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
        else:
            # x->(149,bs,3)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
            T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
        lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        count += B
        x_data_orig = x_data.clone().detach()#提前克隆的副本
        
        col_list = torch.full((B,), -1).to(device)
        acc_batch = torch.empty(B,0).to(device)
        ttc_batch = torch.empty(B,0).to(device)

        
        for frame in range(his_horizon, T):#对32个样本检测
            if model_type == 'cnn1d' or model_type == 'tcn':
                x= x_data[:, :, frame-his_horizon:frame]   # (20, 3, 10)
            else:
                x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)

            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            #x = x.transpose(0,1).reshape(B, -1)
            acc_pre = model(x).squeeze()
            if acc_pre.dim() == 0:
                acc_pre = acc_pre.unsqueeze(0)
            acc_batch = torch.cat((acc_batch, acc_pre.unsqueeze(1)), dim=1)#记录所有acc     
            if model_type == 'cnn1d' or model_type == 'tcn':
                if frame < T-1:
                    # 根据当前速度和加速度计算下一时间速度
                    sv_spd_ = x_data[:, 1, frame] + acc_pre*Ts # (32)
                    sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
                    # 计算下一时间速度的相对速度
                    delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
                    # 该时刻真实相对速度
                    delta_v = x_data[:, -1, frame] # (32)
                    # 通过两车车距加上相对位移得到下一时间段车距 ？
                    spacing_ = x_data[:, 0, frame] + Ts*(delta_v + delta_v_)/2 # (32)
                    for i in range(len(spacing_)):  # 遍历每一行
                        if spacing_[i] < 0 and col_list[i] == -1:    # 如果这一行包含0
                            col_list[i] = frame+1
                    # update 根据计算得到的值，更新下一时间的值
                    next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
                    x_data[:, :, frame + 1] = next_frame_data
            else:
                if frame < T-1:#开环训练，不断依照预测值更新下一个时间节点的数据
                    sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts#新的自车速度
                    sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)#保证速度不小于等于0
                    delta_v_ = lv_spd[frame + 1] - sv_spd_#vt临时计算的
                    delta_v = x_data[frame, :, -1]#v0原有的
                    spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2#新的距离
                    for i in range(len(spacing_)):  # 遍历每一行
                        if spacing_[i] < 0 and col_list[i] == -1:    # 如果这一行包含0
                            col_list[i] = frame+1
                    next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
                    x_data[frame + 1] = next_frame_data
            if spacing_.dim() == 0:
                ttc_batch_ = (-spacing_ / delta_v_).unsqueeze(0)
            else:
                ttc_batch_ = (-spacing_ / delta_v_)
                
            ttc_batch = torch.cat((ttc_batch, ttc_batch_.unsqueeze(1)), dim=1)
        for i in range(B):#对32个样本统一处理
            if model_type == 'cnn1d' or model_type == 'tcn':
                spacing_obs = x_data_orig[i,0,...]
                if(col_list[i] != -1):
                    acc_batch_ = acc_batch[i,:col_list[i]-his_horizon]
                    ttc_batch_ = ttc_batch[i,:col_list[i]-his_horizon]#修改
                    spacing_pre_ = x_data[i, 0, :col_list[i]]
                    error_single = criterion(spacing_pre_[:col_list[i]], spacing_obs[:col_list[i]]).item()
                else:
                    acc_batch_ = acc_batch[i,:]
                    ttc_batch_ = ttc_batch[i,:]
                    spacing_pre_ = x_data[i, 0, :]
                    error_single = criterion(spacing_pre_[:], spacing_obs[:]).item()
            else:
                spacing_obs = x_data_orig[...,i,0]
                if(col_list[i] != -1):
                    acc_batch_ = acc_batch[i,:col_list[i]-his_horizon]
                    ttc_batch_ = ttc_batch[i,:col_list[i]-his_horizon]#修改
                    spacing_pre_ = x_data[:col_list[i],i, 0]
                    error_single = criterion(spacing_pre_[:col_list[i]], spacing_obs[:col_list[i]]).item()
                else:
                    acc_batch_ = acc_batch[i,:]
                    ttc_batch_ = ttc_batch[i,:]
                    spacing_pre_ = x_data[:,i, 0]
                    error_single = criterion(spacing_pre_[:], spacing_obs[:]).item()
            acc_batch_ = acc_batch_.cpu().detach().numpy()
            jerk_single = np.mean(np.abs(np.diff(acc_batch_)/Ts))
            TTC_single = [x for x in ttc_batch_ if x >= 0]#除去其中TTC小于0的
            if len(TTC_single) > 0:
                minimum_ttc_single = calculate_safety(TTC_single)#该样本的最小TTC
                minimum_ttc_set= torch.cat((minimum_ttc_set,torch.tensor(minimum_ttc_single).unsqueeze(0).to(device)), dim=0)      
            jerk_set= torch.cat((jerk_set,torch.tensor(jerk_single).unsqueeze(0).to(device)), dim=0)
            error_set= torch.cat((error_set,torch.tensor(error_single).unsqueeze(0).to(device)), dim=0)
        col = col + torch.sum(col_list != -1).item()
    if len(minimum_ttc_set) == 0:
        ttc = 0
    else:
        ttc = sum(minimum_ttc_set)/len(minimum_ttc_set)
    error = sum(error_set)/len(error_set)
    return error,col,count,sum(jerk_set)/len(jerk_set),ttc#还需修改（以MSE为基准）


def finetuning():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    # Train
    Ts = 0.1
    max_len = 150
    og_net = maml.net
    # 创建一个与原始网络结构相同的虚拟网络
    if model_type == 'nn':
        dummy_net = nn_model(input_size = his_horizon*3)
    elif model_type == 'tcn':
        dummy_net = TemporalConvNet(num_inputs=3, num_channels=(16,32))
#     elif model_type == 'cnn1d':
#         dummy_net = CNN1D()
        # dummy_net = cbrd(num_inputs=3, num_output=1)
    dummy_net=dummy_net.to(device)

    # 多gpu训练
    # if torch.cuda.device_count() > 1:   # 查看当前电脑的可用的gpu的数量，若gpu数量>1,就多gpu训练
    #     dummy_net = torch.nn.DataParallel(dummy_net)  # 自动选择gpu
    # dummy_net.to(device)

    # 加载原始网络的权重
    # weight_path = torch.load(state_path)
    # dummy_net.load_state_dict(weight_path.state_dict())
    dummy_net.load_state_dict(og_net.state_dict())
    # 进行迭代，每次更新虚拟网络的参数
    num_shots=5
    lr = 0.01
    loss_fn=nn.MSELoss()
    optim=torch.optim.Adam
    opt=optim(dummy_net.parameters(),lr=lr, weight_decay=3e-4)


    # 数据集划分
    def split_data(data,data_ratio):
        # random.seed(SEED)
        # np.random.seed(SEED)
        # torch.manual_seed(SEED)
        if data_ratio > 0 and data_ratio <= 1:
            # 如果小于等于1，根据输入百分比计算获取数据集的数量
            data_size=int(len(data)*data_ratio)
        else:
            # 如果大于1，则data_ratio为获取数据集中的数量
            data_size = data_ratio
        return data[:data_size]
    # np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

    # 获取数据集的数量
    K=40
    dataset_train = split_data(train_data, K)
    dataset_loader_train = ImitationCarFolData(dataset_train, max_len = max_len, Ts=Ts)
    train_loader = DataLoader(
                dataset_loader_train,
                batch_size=10,
                shuffle=True,
                num_workers=1,
                drop_last=True)
    dataset_test = test_data
    dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len, Ts=Ts)
    test_loader = DataLoader(
                dataset_loader_test,
                batch_size=64,
                shuffle=False,
                num_workers=1,
                drop_last=False)
    # dataset_val = NGSIM_val
    # dataset_loader_val = ImitationCarFolData(dataset_val, max_len = max_len, Ts=Ts)
    # val_loader = DataLoader(
    #             dataset_loader_val,
    #             batch_size=10,
    #             shuffle=True,
    #             num_workers=1,
    #             drop_last=True)


    # 初始化变量
    train_loss_his = [] # 训练损失
    test_error_his = [] # 测试误差
    best_train_loss = None # 最佳训练损失

    # 训练过程
    best_error = 10000
    for epoch in range(num_shots):
        train_losses = [] # 记录每个epoch的训练损失
        validation_losses = [] # 记录每个epoch的验证损失
        jerk_val = 0
        dummy_net.train()
        # 遍历数据集
        for i, item in enumerate(train_loader):
            # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
            # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
            x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
            # Put T into the first dimension, B, T, d -> T, B, d
            # 将x_data中3个(32,374)连接，转换成(3,32,374)
            x_data = torch.stack(x_data)
            # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
            if model_type == 'cnn1d' or model_type == 'tcn':
                # x->(bs,3,149)，y->(149,bs)
                x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
                B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
            else:
                # x->(149,bs,3)，y->(149,bs)
                x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
                T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
            # print(T,B,d)  # 149  20  3
                
            y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
            y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
            # 从历史数据时间步开始遍历
            for frame in range(his_horizon, T):
                if model_type == 'cnn1d' or model_type == 'tcn':
                    x= x_data[:, :, frame-his_horizon:frame]
                else:
                    x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
                if model_type == 'nn':
                    x = x.transpose(0,1).reshape(B, -1) # flatten for history data
                # 根据his_horizon个数据预测加速度
                acc_pre = dummy_net(x).squeeze() # (32)
                y_pre[frame - his_horizon] = acc_pre
            #计算损失并进行反传及优化
            loss = loss_fn(y_pre, y_label)
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
            opt.step()

            train_losses.append(loss.item())
        # 计算本轮平均损失
        train_loss = np.mean(train_losses)

        train_loss_his.append(train_loss)
        print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))
#         logger.info("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

        mean_spacing_error,col,count,jerk,miniumu_ttc = model_evaluate(dummy_net,his_horizon,test_loader)
        print("mean_spacing_error：{:.5f}，col={}，count={}，rate={:.5f}%，jerk={:.5f}，miniumu_ttc={:.5f}".format(mean_spacing_error, col, count, (col/count)*100, jerk, miniumu_ttc))
#         logger.info("mean_spacing_error：{:.5f}，col={}，count={}，rate={:.5f}%，jerk={:.5f}，miniumu_ttc={:.5f}".format(mean_spacing_error, col, count, (col/count)*100, jerk, miniumu_ttc))
        if mean_spacing_error < best_error:
            best_error = mean_spacing_error
            # save the best model
#             with open(save_path, 'wb') as f:
#                 torch.save(dummy_net, f)
    return best_error



# 元学习内循环训练过程
def inner_train(data_loader, net ,temp_weights, criterion, metal=None):
    net.train()
    # 遍历数据集
    for i, item in enumerate(data_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((20,149)，(20,149)，(20,149))  y_data(20,149)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(20,149)连接，转换成(3,20,149)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        if model_type == 'cnn1d' or model_type == 'tcn':
            # x->(bs,3,149)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
            B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
        else:
            # x->(149,bs,3)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
            T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
            
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,20)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,20)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            if model_type == 'cnn1d' or model_type == 'tcn':
                x= x_data[:, :, frame-his_horizon:frame]
            else:
                x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = net(x, temp_weights).squeeze() # (32)
            # acc_pre = net.module(x, temp_weights).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        if metal == None:
            return loss
        else:
            return loss, y_pre


# 生成不同数据集的任务类
class DataTask():
    def __init__(self,data_range):
        self.data_range = data_range
  # 生成任务数据，默认生成一个数据点，可以通过指定size参数生成多个数据点
    def sample_data(self,size=1):
        if self.data_range == 1:
            meta_data = HighD_data
            Ts = 0.04
            max_len = 375
            if adapt_tasks:
                task_net = HighD_net
            # print('--HighD--')
        elif self.data_range == 2:
            meta_data = Lyft_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = Lyft_net
            # print('--Lyft---')
        elif self.data_range == 3:
            meta_data = SPMD1_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = SPMD1_net
            # print('SPMD_DAS1')
        elif self.data_range == 4:
            meta_data = NGSIM_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = SPMD2_net
            # print('SPMD_DAS2')
        elif self.data_range == 5:
            meta_data = Waymo_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = Waymo_net
            # print('--Waymo--')
        meta_data_set1, meta_data_set2 = split_train(meta_data, size)
#         print(len(meta_data_set1),len(meta_data_set2))
        dataset_loader1 = ImitationCarFolData(meta_data_set1, max_len = max_len, Ts = Ts)
        dataset_loader2 = ImitationCarFolData(meta_data_set2, max_len = max_len, Ts = Ts)
        if adapt_tasks:
            return dataset_loader1, dataset_loader2, task_net
        else:
            return dataset_loader1, dataset_loader2


# 任务分布，及选择不同的数据
class DataDistribution():
    def __init__(self, data_range=5, n=3):
        self.data_range = data_range    # 获取数据集的数值
        self.n = n
    # 随机生成一个跟车数据集
    def sample_task(self, epoch):
        # 从1到5中随机选择3个不重复的数
        # 创建一个新的随机数生成器实例
        rng = np.random.RandomState(epoch)
        data_choice = rng.choice(self.data_range, size=self.n, replace=False) + 1
        # data_choice = np.random.choice(self.data_range, size=self.n, replace=False) + 1
        return data_choice


def attenuate_init(attenuator, panning, task_embeddings, names_weights_copy):
        ## Attenuate
        updated_names_weights_copy = list()
        
        # 生成衰减参数和偏移参数
        if attenuator == None:
            beta = panning(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(key + beta[i])
                i+=1
        elif panning == None:
            gamma = attenuator(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(gamma[i] * key)
                i+=1
        else:
            gamma = attenuator(task_embeddings)
            beta = panning(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(gamma[i] * key + beta[i])
                i+=1
            
        return updated_names_weights_copy


def get_inner_loop_parameter_dict(params):
    """
    Returns a dictionary with the parameters to use for inner loop updates.
    :param params: A dictionary of the network's parameters.
    :return: A dictionary of the parameters to use for the inner loop optimization process.
    """
    param_dict = dict()
    for name, param in params:
        if param.requires_grad:
            param_dict[name] = param.to(device)

    return param_dict

def get_per_step_loss_importance_vector(inner_step, current_epoch):
        """
        Generates a tensor of dimensionality (num_inner_loop_steps) indicating the importance of each step's target
        loss towards the optimization loss.
        :return: A tensor to be used to compute the weighted average of the loss, useful for
        the MSL (Multi Step Loss) mechanism.
        """
        multi_step_loss_num_epochs = 20
        loss_weights = np.ones(shape=(inner_step)) * (
                1.0 / inner_step)
        decay_rate = 1.0 / inner_step / multi_step_loss_num_epochs
        min_value_for_non_final_losses = 0.03 / inner_step
        for i in range(len(loss_weights) - 1):
            curr_value = np.maximum(loss_weights[i] - (current_epoch * decay_rate), min_value_for_non_final_losses)
            loss_weights[i] = curr_value

        curr_value = np.minimum(
            loss_weights[-1] + (current_epoch * (inner_step - 1) * decay_rate),
            1.0 - ((inner_step - 1) * min_value_for_non_final_losses))
        loss_weights[-1] = curr_value
        loss_weights = torch.Tensor(loss_weights).to(device)
        return loss_weights


# MAML元学习训练过程
class MAML():
    def __init__(self, net, alpha, beta, tasks, k, weight_decay, inner_step):  # (net,alpha=0.01,beta=0.0001,tasks=data_tasks,k=150)
        # 初始化 MAML 算法的元参数
        self.net = net  # 元学习使用的神经网络模型
        self.weights = list(net.parameters())  # 获取神经网络的参数
        self.alpha = alpha  # 内循环学习率
        self.beta = beta  # 外循环学习率
        self.inner_step = inner_step  # 内部循环更新的步数
        self.tasks = tasks  # 正弦任务的分布
        self.k = k  # 内循环（任务采样）的数据集大小
        self.criterion = nn.MSELoss()  # 均方误差损失函数
        self.meta_losses = []  # 用于存储元学习的损失历史
        self.meta_test = []
        self.finetuning_error = []
        self.plot_every = 10  # 每隔多少次迭代记录一次损失，用于绘图
        self.print_every = 1  # 每隔多少次迭代输出一次损失信息，用于打印
        self.first_add = 0  # 用于添加优化器优化的参数
        self.batch_size = int(self.k/2)
        # 设置权重衰减因子
        self.weight_decay = weight_decay
        self.num_layers = len(self.weights)
        params = [{'params': self.weights}]
        if MeTAL:
            base_learner_num_layers = len(self.weights)
            support_meta_loss_num_dim = base_learner_num_layers + 2
            support_adapter_num_dim = base_learner_num_layers + 1

            self.meta_loss = MetaLossNetwork(support_meta_loss_num_dim, notspi=self.inner_step, device=device).to(device=device)
            self.meta_loss_adapter = LossAdapter(support_adapter_num_dim, num_loss_net_layers=2, notspi=self.inner_step, device=device).to(device=device)
            new_params = {'params': self.meta_loss.parameters()}
            params.append(new_params)
            new_params = {'params': self.meta_loss_adapter.parameters()}
            params.append(new_params)

            if semi_supervised:
                query_num_dim = base_learner_num_layers + 1
                self.meta_query_loss = MetaLossNetwork(query_num_dim, notspi=self.inner_step, device=device).to(device=device)
                self.meta_query_loss_adapter = LossAdapter(query_num_dim, num_loss_net_layers=2, notspi=self.inner_step, device=device).to(device=device)
                new_params = {'params': self.meta_query_loss.parameters()}
                params.append(new_params)
                new_params = {'params': self.meta_query_loss_adapter.parameters()}
                params.append(new_params)

        if attenuate:  #L2F
            self.attenuator = nn.Sequential(
                nn.Linear(self.num_layers, self.num_layers),
                nn.ReLU(inplace=True),
                nn.Linear(self.num_layers, self.num_layers),
                nn.Sigmoid()
            ).to(device=device)
            new_params = {'params': self.attenuator.parameters()}
            params.append(new_params)
        if tapt:  #bias
            self.panning = nn.Sequential(
                nn.Linear(self.num_layers, self.num_layers),
                nn.ReLU(inplace=True),
                nn.Linear(self.num_layers, self.num_layers),
                nn.Tanh()
            ).to(device=device)
            new_params = {'params': self.panning.parameters()}
            params.append(new_params)
        if alfa:  # ALFA
            input_dim = self.num_layers*2
            self.regularizer = nn.Sequential(
                nn.Linear(input_dim, input_dim),
                nn.ReLU(inplace=True),
                nn.Linear(input_dim, input_dim)
            ).to(device=device)
            new_params = {'params': self.regularizer.parameters()}
            params.append(new_params)
        self.meta_optimiser = torch.optim.Adam(params,lr=self.beta, weight_decay=3e-4)

    def inner_loop(self, task):
        taskloss = []
        # 内循环更新参数，用于计算元学习损失
        temp_weights = [w.clone() for w in self.weights]  # 复制当前网络参数作为临时参数
        if adapt_tasks:
            dataset_loader1, dataset_loader2, task_net = task.sample_data(size=self.k)
            task_optimiser = torch.optim.Adam([{'params':task_net.parameters()}],lr=0.001, weight_decay=3e-4)
        else:
            dataset_loader1, dataset_loader2 = task.sample_data(size=self.k)  # 从任务中采样数据集 D 和 D'
        dataloader1 = DataLoader(
                dataset_loader1,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        for step in range(self.inner_step):
            if MeTAL:
                support_task_state = []
                support_loss, support_preds = inner_train(dataloader1, self.net, temp_weights, self.criterion, metal=1)
                support_loss /= self.batch_size
                support_task_state.append(support_loss)

                for v in temp_weights:
                    support_task_state.append(v.mean())

                support_task_state = torch.stack(support_task_state)
                adapt_support_task_state = (support_task_state - support_task_state.mean())/(support_task_state.std() + 1e-12)                                                 

                updated_meta_loss_weights = self.meta_loss_adapter(adapt_support_task_state, step, self.names_loss_weights_copy)

                # support_y = torch.zeros(support_preds.shape).to(device)
                # support_y[torch.arange(support_y.size(0)), y] = 1
                support_task_state = torch.cat((
                    support_task_state.view(1, -1).expand(support_preds.size(1), -1),
                    support_preds.transpose(0, 1).mean(dim=1, keepdim=True)
                    # support_y
                ), -1)

                support_task_state = (support_task_state - support_task_state.mean()) / (support_task_state.std() + 1e-12)
                meta_support_loss = self.meta_loss(support_task_state, step, params=updated_meta_loss_weights).mean().squeeze()
                if semi_supervised:
                    dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                    _, query_preds = inner_train(dataloader2, self.net, temp_weights, self.criterion, metal=1)
                    query_task_state = []
                    for v in temp_weights:
                        query_task_state.append(v.mean())
                    # out_prob = F.log_softmax(query_preds)
                    # instance_entropy = torch.sum(torch.exp(out_prob) * out_prob, dim=-1)
                    query_task_state = torch.stack(query_task_state)
                    query_task_state = torch.cat((
                                query_task_state.view(1, -1).expand(query_preds.size(1), -1), 
                                query_preds.transpose(0, 1).mean(dim=1, keepdim=True)
                                # instance_entropy.view(-1, 1)
                    ), -1)

                    query_task_state = (query_task_state - query_task_state.mean())/(query_task_state.std() + 1e-12)
                    updated_meta_query_loss_weights = self.meta_query_loss_adapter(query_task_state.mean(0), step, self.names_query_loss_weights_copy)

                    meta_query_loss = self.meta_query_loss(query_task_state, step, params=updated_meta_query_loss_weights).mean().squeeze()

                    loss = support_loss + meta_support_loss + meta_query_loss
                else:
                    loss = support_loss + meta_support_loss
                grads = torch.autograd.grad(loss, temp_weights, create_graph=True, retain_graph=True)  # 计算损失对参数的梯度

            else:
                # 训练过程
                loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
                grads = torch.autograd.grad(loss, temp_weights, create_graph=True, retain_graph=True)  # 计算损失对参数的梯度

            if step == 0:
                if adapt_tasks:     # every tasks
                    grads = tuple(grads[i] for i in range(len(grads)))
                    layerwise_mean_grads = []
                    for i in range(len(grads)):
                        layerwise_mean_grads.append(grads[i].mean())  # 每一层的梯度取平均值
                    layerwise_mean_grads = torch.stack(layerwise_mean_grads)
                    temp_weights = attenuate_init(task_net, None, layerwise_mean_grads, temp_weights)  # 得到新的权重
                    new_loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
                    grads = torch.autograd.grad(new_loss, temp_weights, create_graph=True, retain_graph=True)

                if attenuate or tapt:    #L2F
                    layerwise_mean_grads = []
                    for i in range(len(grads)):
                        layerwise_mean_grads.append(grads[i].mean())  # 每一层的梯度取平均值
                    layerwise_mean_grads = torch.stack(layerwise_mean_grads)
                    if attenuate and tapt:
                        temp_weights = attenuate_init(self.attenuator, self.panning, layerwise_mean_grads, temp_weights)  # 得到新的权重
                    elif attenuate and not tapt:
                        temp_weights = attenuate_init(self.attenuator, None, layerwise_mean_grads, temp_weights)
                    elif not attenuate and tapt:
                        temp_weights = attenuate_init(None, self.panning, layerwise_mean_grads, temp_weights)
                    new_loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
                    grads = torch.autograd.grad(new_loss, temp_weights, create_graph=True, retain_graph=True)

            if alfa:    #ALFA
                # 用于存储权重的平均值和梯度的平均值
                per_step_task_embedding = []
                for v in temp_weights:
                    per_step_task_embedding.append(v.mean())
            
                for i in range(len(grads)):
                    per_step_task_embedding.append(grads[i].mean())

                per_step_task_embedding = torch.stack(per_step_task_embedding)

                generated_params = self.regularizer(per_step_task_embedding)

                generated_alpha, generated_beta = torch.split(generated_params, split_size_or_sections=self.num_layers)
                generated_alpha_params = []
                generated_beta_params = []
                g = 0
                for key in temp_weights:
                    generated_alpha_params.append(generated_alpha[g])
                    generated_beta_params.append(generated_beta[g])
                    g+=1
                # 初始化ALFA自适应参数
                if self.first_add == 0:
                    self.first_add += 1

                    if random_init:
                        self.names_beta_dict_per_param = nn.ParameterDict()
                    self.names_alpha_dict = nn.ParameterDict()
                    self.names_beta_dict = nn.ParameterDict()
                    init_learning_rate = torch.ones(1) * self.alpha
                    init_learning_rate.to(device)
                    init_weight_decay = torch.ones(1)* self.weight_decay
                    init_weight_decay.to(device)

                    for idx, param in enumerate(temp_weights):

                        if random_init:
                        # per-param weight decay for random init
                            self.names_beta_dict_per_param[str(idx)] = nn.Parameter(
                                data=torch.ones(param.shape) * init_weight_decay * init_learning_rate,
                                requires_grad=use_learnable_learning_rates)

                            self.names_beta_dict[str(idx)] = nn.Parameter(
                                data=torch.ones(self.inner_step),
                                requires_grad=use_learnable_beta)
                            
                            self.names_beta_dict_per_param.to(device)
                        else:
                            # per-step per-layer meta-learnable weight decay bias term (for more stable training and better performance by 2~3%)
                            self.names_beta_dict[str(idx)] = nn.Parameter(
                                data=torch.ones(self.inner_step) * init_weight_decay * init_learning_rate,
                                requires_grad=use_learnable_beta)
                        
                        # per-step per-layer meta-learnable learning rate bias term (for more stable training and better performance by 2~3%)
                        self.names_alpha_dict[str(idx)] = nn.Parameter(
                            data=torch.ones(self.inner_step) * init_learning_rate,
                            requires_grad=use_learnable_alpha)
                        
                        self.names_beta_dict.to(device)
                        self.names_alpha_dict.to(device)
                    # 将额外的参数添加到优化器中
                    if random_init:
                        self.meta_optimiser.add_param_group({'params': self.names_beta_dict_per_param.parameters()})
                    self.meta_optimiser.add_param_group({'params': self.names_beta_dict.parameters()})
                    self.meta_optimiser.add_param_group({'params': self.names_alpha_dict.parameters()})
            else:
                # 使用可学习的参数
                if self.first_add == 0:
                    self.first_add += 1

                    init_learning_rate = torch.ones(1) * self.alpha
                    init_learning_rate.to(device)
                    init_weight_decay = torch.ones(1)* self.weight_decay
                    init_weight_decay.to(device)

                    self.names_learning_rates_dict = nn.ParameterDict()
                    for idx, param in enumerate(temp_weights):
                        self.names_learning_rates_dict[str(idx)] = nn.Parameter(
                            data=torch.ones(self.inner_step) * init_learning_rate,
                            requires_grad=use_learnable_alpha)
                    self.names_learning_rates_dict.to(device)
                    if use_learnable_alpha:
                        self.meta_optimiser.add_param_group({'params': self.names_learning_rates_dict.parameters()})
                        
                    if inner_update == "L2" and self.weight_decay != None:
                        self.names_weight_decay_dict = nn.ParameterDict()
                        for idx, param in enumerate(temp_weights):
                            self.names_weight_decay_dict[str(idx)] = nn.Parameter(
                                    data=torch.ones(self.inner_step) * init_weight_decay,
                                    requires_grad=use_learnable_beta)
                        self.names_weight_decay_dict.to(device)
                        if use_learnable_beta:
                            self.meta_optimiser.add_param_group({'params': self.names_weight_decay_dict.parameters()})

            if inner_update == "L2" and self.weight_decay != None:
                # temp_weights = [w - self.alpha * (g + (self.weight_decay * w)) for w, g in zip(temp_weights, grads)]
                # temp_weights = [((1 - self.alpha * self.weight_decay) * w) - (self.alpha * g) for w, g in zip(temp_weights, grads)]
                temp_weights = [((1 - self.names_learning_rates_dict[str(key)][step] * self.names_weight_decay_dict[str(key)][step]) * w) - (self.names_learning_rates_dict[str(key)][step] * g) for key, (w, g) in enumerate(zip(temp_weights, grads))]
            elif inner_update == "ALFA" and alfa:
                if random_init:
                    temp_weights = [(1 - beta * self.names_beta_dict[str(key)][step] * self.names_beta_dict_per_param[str(key)]) * w - (alpha * self.names_alpha_dict[str(key)][step] * g) for key, (w, g, beta, alpha) in enumerate(zip(temp_weights, grads, generated_beta_params, generated_alpha_params))]
                else:
                    temp_weights = [(1 - beta * self.names_beta_dict[str(key)][step]) * w - (alpha * self.names_alpha_dict[str(key)][step] * g) for key, (w, g, beta, alpha) in enumerate(zip(temp_weights, grads, generated_beta_params, generated_alpha_params))]
            else:
                temp_weights = [w - self.names_learning_rates_dict[str(key)][step] * g for key, (w, g) in enumerate(zip(temp_weights, grads))]  # 临时参数更新 梯度下降

            if use_multi_step_loss_optimization and self.epoch < 20:
                dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                metaloss = inner_train(dataloader2, self.net, temp_weights, self.criterion) / self.batch_size
                taskloss.append(self.per_step_loss_importance_vectors[step] * metaloss)
            else:
                if step == (self.inner_step - 1):
                    dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                    metaloss = inner_train(dataloader2, self.net, temp_weights, self.criterion) / self.batch_size
                    taskloss.append(metaloss)
        task_losses = torch.sum(torch.stack(taskloss))
        if adapt_tasks:
            task_optimiser.zero_grad()
            metaloss.backward(retain_graph=True)
            task_optimiser.step()
        return task_losses

    def outer_loop(self, num_epochs):  # epoch 5000
        best_loss = 10000
        best_index = 0
        for epoch in range(1, num_epochs + 1):
            self.epoch = epoch
            if MeTAL:
                self.names_loss_weights_copy = get_inner_loop_parameter_dict(self.meta_loss.named_parameters())
                if semi_supervised:
                    self.names_query_loss_weights_copy = get_inner_loop_parameter_dict(self.meta_query_loss.named_parameters())
            total_loss = 0
            metaloss_sum = 0
            tasks = self.tasks.sample_task(epoch)
            for i in tasks:
                if use_multi_step_loss_optimization:
                    self.per_step_loss_importance_vectors = get_per_step_loss_importance_vector(self.inner_step, epoch)
                task = DataTask(i)
                metaloss = self.inner_loop(task)  # 内循环更新参数，计算元学习损失
                metaloss_sum += metaloss  # mete_loss求和
            self.meta_optimiser.zero_grad()
            # metaloss_sum.backward(retain_graph=True)
            metagrads = torch.autograd.grad(metaloss_sum, self.weights, retain_graph=True)  # 计算元学习损失对参数的梯度
            # # 重要步骤：使用元学习梯度更新网络参数
            for w, g in zip(self.weights, metagrads):
                w.grad = g

            if MeTAL:
                meta_loss_grads = torch.autograd.grad(metaloss_sum, list(self.meta_loss.parameters()), retain_graph=True)
                for w, g in zip(self.meta_loss.parameters(), meta_loss_grads):
                    w.grad = g

                meta_loss_adapter_grads = torch.autograd.grad(metaloss_sum, list(self.meta_loss_adapter.parameters()), retain_graph=True)
                for w, g in zip(self.meta_loss_adapter.parameters(), meta_loss_adapter_grads):
                    w.grad = g

                if semi_supervised:
                    meta_query_loss_grads = torch.autograd.grad(metaloss_sum, list(self.meta_query_loss.parameters()), retain_graph=True)
                    for w, g in zip(self.meta_query_loss.parameters(), meta_query_loss_grads):
                        w.grad = g

                    meta_query_loss_adapter_grads = torch.autograd.grad(metaloss_sum, list(self.meta_query_loss_adapter.parameters()), retain_graph=True)
                    for w, g in zip(self.meta_query_loss_adapter.parameters(), meta_query_loss_adapter_grads):
                        w.grad = g

            if not alfa and use_learnable_alpha:
                learning_rates_grads = torch.autograd.grad(metaloss_sum, list(self.names_learning_rates_dict.parameters()), retain_graph=True)
                for w, g in zip(self.names_learning_rates_dict.parameters(), learning_rates_grads):
                    w.grad = g
                if inner_update == "L2" and self.weight_decay != None and use_learnable_beta:
                    weight_decay_grads = torch.autograd.grad(metaloss_sum, list(self.names_weight_decay_dict.parameters()), retain_graph=True)
                    for w, g in zip(self.names_weight_decay_dict.parameters(), weight_decay_grads):
                        w.grad = g

            if attenuate:
                attenuate_grads = torch.autograd.grad(metaloss_sum, list(self.attenuator.parameters()), retain_graph=True)
                for w, g in zip(self.attenuator.parameters(), attenuate_grads):
                    w.grad = g
            if tapt:
                tapt_grads = torch.autograd.grad(metaloss_sum, list(self.panning.parameters()), retain_graph=True)
                for w, g in zip(self.panning.parameters(), tapt_grads):
                    w.grad = g
            if alfa:
                alfa_grads = torch.autograd.grad(metaloss_sum, list(self.regularizer.parameters()), retain_graph=True)
                for w, g in zip(self.regularizer.parameters(), alfa_grads):
                    w.grad = g
                if use_learnable_beta:
                    names_beta_grads = torch.autograd.grad(metaloss_sum, list(self.names_beta_dict.parameters()), retain_graph=True)
                    for w, g in zip(self.names_beta_dict.parameters(), names_beta_grads):
                        w.grad = g
                if use_learnable_alpha:
                    names_alpha_grads = torch.autograd.grad(metaloss_sum, self.names_alpha_dict.parameters(), retain_graph=True)
                    for w, g in zip(self.names_alpha_dict.parameters(), names_alpha_grads):
                        w.grad = g
                if random_init:
                    random_init_grads = torch.autograd.grad(metaloss_sum, self.names_beta_dict_per_param.parameters(), retain_graph=True)
                    for w, g in zip(self.names_beta_dict_per_param.parameters(), random_init_grads):
                        w.grad = g
            
            self.meta_optimiser.step()  # 使用外循环优化器更新网络参数
            total_loss += metaloss_sum.item() / len(tasks)
            mserror,col,count,jerk,ttc = model_evaluate(self.net,his_horizon,meta_loader)
            if epoch % self.print_every == 0:
                print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
                print("{}/{}. mserror: {}  col: {}  count: {}  jerk: {}  ttc: {}".format(epoch, num_epochs, mserror, col, count, jerk, ttc))
#                 logger.info("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
#                 logger.info("{}/{}. mserror: {}  col: {}  count: {}  jerk: {}  ttc: {}".format(epoch, num_epochs, mserror, col, count, jerk, ttc))
                if mserror < best_loss:
                    best_loss = mserror
                    best_index = epoch
                    # with open(file_path, 'wb') as f:
                    #     torch.save(self.net, f)
                self.meta_losses.append(total_loss / self.print_every)
                self.meta_test.append(mserror.cpu().detach().numpy() / self.print_every)
            self.finetuning_error.append(finetuning())
            print("   ")
            # fine_error = finetuning()
            # self.finetuning_error.append(fine_error)
            # if epoch % self.plot_every == 0:
            #     epoch_path = f"/home/ubuntu/fedavg/FollowNet-car/MAML_state/maml_l2f_test/MAML_nn_a0.01_b0.0001_{epoch}E.pt" # 元初始化参数
            #     # weight_change = torch.load(file_path)
            #     with open(epoch_path, 'wb') as f:
            #         torch.save(self.net, f)
        print("(", best_index, ")","best_loss:", best_loss)

dataset = 'SPMD2'
model_type = 'tcn'
if dataset == 'SPMD1':
    train_data = SPMD1_train
    test_data = SPMD1_test
elif dataset == 'SPMD2':
    train_data = SPMD2_train
    test_data = SPMD2_test
elif dataset == 'Waymo':
    train_data = Waymo_train
    test_data = Waymo_test
elif dataset == 'NGSIM':
    train_data = NGSIM_train
    test_data = NGSIM_test
elif dataset == 'Lyft':
    train_data = Lyft_train
    test_data = Lyft_test
elif dataset == 'HighD':
    train_data = HighD_train
    test_data = HighD_test
# 定义保存文件的文件夹路径
# file_path = "/home/ubuntu/fedavg/FollowNet-car/MAML_state/MAML_alfa_nn_a0.01_b0.0001_k300_E50.pt" # 元初始化参数
# save_folder = '/home/ubuntu/fedavg/FollowNet-car/state'
# save = f'MAML_ALFA_{model_type}_{dataset}_oooollllldddd.pt' # 保存模型文件
# 确保保存文件的文件夹存在，如果不存在，则创建
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder)
# 定义文件保存路径
# save_path = os.path.join(save_folder, save)
meta_test = test_data
meta_loader_test = ImitationCarFolData(meta_test, max_len = max_len, Ts=Ts)
meta_loader = DataLoader(
            meta_loader_test,
            batch_size=96,
            shuffle=False,
            num_workers=1,
            drop_last=False)
# 模型初始化
if model_type == 'nn':
    net = nn_model(input_size = his_horizon*3).to(device)
elif model_type == 'tcn':
    net = TemporalConvNet(num_inputs=3, num_channels=(16,32)).to(device)
# elif model_type == 'cnn1d':
#     net = CNN1D().to(device)
    # net = cbrd(num_inputs=3, num_output=1).to(device)
# elif model_type == 'lstm':
#     net = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm

# model_state = list(net.parameters())
# print(model_state)

# 多gpu训练
# if torch.cuda.device_count() > 1:   # 查看当前电脑的可用的gpu的数量，若gpu数量>1,就多gpu训练
#     net = torch.nn.DataParallel(net)  # 自动选择gpu
# net.to(device)

# 内部循环更新策略
MeTAL = False   # 任务自适应loss函数
semi_supervised = False  # 使用查询集 作为半监督
adapt_tasks = False  # 权重初始化 每个任务用一个
attenuate = False  # 权重初始化 所有任务共用 权重衰减
tapt = False  # 权重初始化 所有任务共用 权重偏差
alfa = False        # 自适应学习优化超参数
random_init = False
use_learnable_learning_rates = random_init
use_learnable_beta = False  # 使优化参数beta可学习
use_learnable_alpha = False  # 使优化参数alpha可学习
use_multi_step_loss_optimization = False  # 内部循环多步时，使用最后一步的Loss，还是使用多步的Loss。
inner_update = 'ALFA'
if adapt_tasks:
    num_layers = len(list(net.parameters()))
    HighD_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    Lyft_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    SPMD1_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    SPMD2_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    Waymo_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
data_tasks=DataDistribution(data_range=5, n=3)
maml=MAML(net,alpha=0.01,beta=0.01,tasks=data_tasks,k=200,weight_decay=0.0001,inner_step=1)


maml.outer_loop(num_epochs=500)
plt_path = "/home/ubuntu/fedavg/FollowNet-car/test_metrics"
plt.plot(maml.meta_losses)
# plt.savefig(os.path.join(plt_path, 'MAML_alfa_oooollllldddd_Loss.png'))
plt.show()
plt.close()

plt.plot(maml.meta_test)
# plt.savefig(os.path.join(plt_path, 'MAML_alfa_oooollllldddd_mserror.png'))
plt.show()
plt.close()

plt.plot(maml.finetuning_error)
# plt.savefig(os.path.join(plt_path, 'MAML_alfa_oooollllldddd_finetuning_mserror.png'))
plt.show()
plt.close()
# # 将数据保存到文件中
np.save('/kaggle/working/kaggle_maml_meta_losses.npy', np.array(maml.meta_losses))
np.save('/kaggle/working/kaggle_maml_meta_test.npy', np.array(maml.meta_test))
np.save('/kaggle/working/kaggle_maml_finetuning_error.npy', np.array(maml.finetuning_error))


print('----------------------')
# logger.info("--------------------------")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


cuda:0


/tmp/ipykernel_26/3337634293.py:867: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  minimum_ttc_set= torch.cat((minimum_ttc_set,torch.tensor(minimum_ttc_single).unsqueeze(0).to(device)), dim=0)


1/500. loss: 0.0030535062154134116
1/500. mserror: 102.7873306274414  col: 1087  count: 3637  jerk: 0.00438933540135622  ttc: 36.295753479003906
Epoch: 1| Train Loss: 0.1696991
mean_spacing_error：97.53593，col=304，count=3637，rate=8.35854%，jerk=0.00040，miniumu_ttc=134.02794
Epoch: 2| Train Loss: 0.1518969
mean_spacing_error：96.88409，col=340，count=3637，rate=9.34836%，jerk=0.00327，miniumu_ttc=232.09081
Epoch: 3| Train Loss: 0.1512302
mean_spacing_error：95.04176，col=303，count=3637，rate=8.33104%，jerk=0.00330，miniumu_ttc=105.82082
Epoch: 4| Train Loss: 0.1518357
mean_spacing_error：84.95623，col=362，count=3637，rate=9.95326%，jerk=0.01029，miniumu_ttc=86.44263
Epoch: 5| Train Loss: 0.1476017
mean_spacing_error：63.69027，col=243，count=3637，rate=6.68133%，jerk=0.01057，miniumu_ttc=110.82883
   
2/500. loss: 0.003802728528777758
2/500. mserror: 76.08035278320312  col: 477  count: 3637  jerk: 0.0006114662974141538  ttc: 81.97216033935547
Epoch: 1| Train Loss: 0.1650520
mean_spacing_error：93.45181，col=313，

In [ ]:
import numbers
from copy import copy

import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import random


def extract_top_level_dict(current_dict):
    """
    Builds a graph dictionary from the passed depth_keys, value pair. Useful for dynamically passing external params
    :param depth_keys: A list of strings making up the name of a variable. Used to make a graph for that params tree.
    :param value: Param value
    :param key_exists: If none then assume new dict, else load existing dict and add new key->value pairs to it.
    :return: A dictionary graph of the params already added to the graph.
    """
    output_dict = dict()
    for key in current_dict.keys():
        name = key.replace("layer_dict.", "")
        name = name.replace("layer_dict.", "")
        name = name.replace("block_dict.", "")
        name = name.replace("module-", "")
        top_level = name.split(".")[0]
        sub_level = ".".join(name.split(".")[1:])

        if top_level not in output_dict:
            if sub_level == "":
                output_dict[top_level] = current_dict[key]
            else:
                output_dict[top_level] = {sub_level: current_dict[key]}
        else:
            new_item = {key: value for key, value in output_dict[top_level].items()}
            new_item[sub_level] = current_dict[key]
            output_dict[top_level] = new_item

    #print(current_dict.keys(), output_dict.keys())
    return output_dict


class MetaLinearLayer(nn.Module):
    def __init__(self, input_shape, num_filters, use_bias):
        """
        A MetaLinear layer. Applies the same functionality of a standard linearlayer with the added functionality of
        being able to receive a parameter dictionary at the forward pass which allows the convolution to use external
        weights instead of the internal ones stored in the linear layer. Useful for inner loop optimization in the meta
        learning setting.
        :param input_shape: The shape of the input data, in the form (b, f)
        :param num_filters: Number of output filters
        :param use_bias: Whether to use biases or not.
        """
        super(MetaLinearLayer, self).__init__()
        b, c = input_shape

        self.use_bias = use_bias
        self.weights = nn.Parameter(torch.ones(num_filters, c))
        nn.init.xavier_uniform_(self.weights)
        if self.use_bias:
            self.bias = nn.Parameter(torch.zeros(num_filters))

    def forward(self, x, params=None):
        """
        Forward propagates by applying a linear function (Wx + b). If params are none then internal params are used.
        Otherwise passed params will be used to execute the function.
        :param x: Input data batch, in the form (b, f)
        :param params: A dictionary containing 'weights' and 'bias'. If params are none then internal params are used.
        Otherwise the external are used.
        :return: The result of the linear function.
        """
        if params is not None:
            params = extract_top_level_dict(current_dict=params)
            if self.use_bias:
                (weight, bias) = params["weights"], params["bias"]
            else:
                (weight) = params["weights"]
                bias = None
        else:
            pass
            #print('no inner loop params', self)

            if self.use_bias:
                weight, bias = self.weights, self.bias
            else:
                weight = self.weights
                bias = None
        # print(x.shape)
        out = F.linear(input=x, weight=weight, bias=bias)
        return out


class MetaStepLossNetwork(nn.Module):
    def __init__(self, input_dim, notspi, device):
        super(MetaStepLossNetwork, self).__init__()

        self.device = device
        # self.args = args
        self.input_dim = input_dim
        self.input_shape = (1, input_dim)

        self.build_network()
        # print("meta network params")
        # for name, param in self.named_parameters():
        #     print(name, param.shape)

    def build_network(self):
        """ 
        Builds the network before inference is required by creating some dummy inputs with the same input as the
        self.im_shape tuple. Then passes that through the network and dynamically computes input shapes and
        sets output shapes for each layer.
        """
        x = torch.zeros(self.input_shape)
        out = x

        self.linear1 = MetaLinearLayer(input_shape=self.input_shape,
                                                    num_filters=self.input_dim, use_bias=True)

        self.linear2 = MetaLinearLayer(input_shape=(1, self.input_dim),
                                                    num_filters=1, use_bias=True)


        out = self.linear1(out)
        out = F.relu_(out)
        out = self.linear2(out)

    def forward(self, x, params=None):
        """                                  
        Forward propages through the network. If any params are passed then they are used instead of stored params.
        :param x: Input image batch.
        :param num_step: The current inner loop step number
        :param params: If params are None then internal parameters are used. If params are a dictionary with keys the
         same as the layer names then they will be used instead.
        :param training: Whether this is training (True) or eval time.
        :param backup_running_statistics: Whether to backup the running statistics in their backup store. Which is
        then used to reset the stats back to a previous state (usually after an eval loop, when we want to throw away stored statistics)
        :return: Logits of shape b, num_output_classes.
        """ 

        linear1_params = None
        linear2_params = None

        if params is not None:
            params = extract_top_level_dict(current_dict=params)

            linear1_params = params['linear1']
            linear2_params = params['linear2']

        out = x
        
        out = self.linear1(out, linear1_params)
        out = F.relu_(out)
        out = self.linear2(out, linear2_params)

        return out

    def zero_grad(self, params=None):
        if params is None:
            for param in self.parameters():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
        else:
            for name, param in params.items():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
                            params[name].grad = None

    def restore_backup_stats(self):
        """
        Reset stored batch statistics from the stored backup.
        """
        for i in range(self.num_stages):
            self.layer_dict['conv{}'.format(i)].restore_backup_stats()


class MetaLossNetwork(nn.Module):
    def __init__(self, input_dim, notspi, device):
        """
        Builds a multilayer convolutional network. It also provides functionality for passing external parameters to be
        used at inference time. Enables inner loop optimization readily.
        :param im_shape: The input image batch shape.
        :param num_output_classes: The number of output classes of the network.
        :param args: A named tuple containing the system's hyperparameters.
        :param device: The device to run this on.
        :param meta_classifier: A flag indicating whether the system's meta-learning (inner-loop) functionalities should
        be enabled. 
        """
        super(MetaLossNetwork, self).__init__()
        
        self.device = device
        # self.args = args
        self.notspi = notspi
        self.input_dim = input_dim
        self.input_shape = (1, input_dim)

        self.num_steps = notspi # number of inner-loop steps

        self.build_network()
        # print("meta network params")
        # for name, param in self.named_parameters():
        #     print(name, param.shape)

    def build_network(self):
        """ 
        Builds the network before inference is required by creating some dummy inputs with the same input as the
        self.im_shape tuple. Then passes that through the network and dynamically computes input shapes and
        sets output shapes for each layer.
        """
        x = torch.zeros(self.input_shape)
        self.layer_dict = nn.ModuleDict()
        # ф
        for i in range(self.num_steps): 
            self.layer_dict['step{}'.format(i)] = MetaStepLossNetwork(self.input_dim, notspi=self.notspi, device=self.device)

            out = self.layer_dict['step{}'.format(i)](x)

    def forward(self, x, num_step, params=None):
        """                                  
        Forward propages through the network. If any params are passed then they are used instead of stored params.
        :param x: Input image batch.
        :param num_step: The current inner loop step number
        :param params: If params are None then internal parameters are used. If params are a dictionary with keys the
         same as the layer names then they will be used instead.
        :param training: Whether this is training (True) or eval time.
        :param backup_running_statistics: Whether to backup the running statistics in their backup store. Which is
        then used to reset the stats back to a previous state (usually after an eval loop, when we want to throw away stored statistics)
        :return: Logits of shape b, num_output_classes.
        """ 
        param_dict = dict()

        if params is not None: 
            # params = {key: value[0] for key, value in params.items()}
            param_dict = extract_top_level_dict(current_dict=params)
            
        for name, param in self.layer_dict.named_parameters():
            path_bits = name.split(".")
            layer_name = path_bits[0]
            if layer_name not in param_dict:
                param_dict[layer_name] = None

            
        out = x
        
        out = self.layer_dict['step{}'.format(num_step)](out, param_dict['step{}'.format(num_step)])

        return out

    def zero_grad(self, params=None):
        if params is None:
            for param in self.parameters():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
        else:
            for name, param in params.items():
                if param.requires_grad == True:
                    if param.grad is not None:
                        if torch.sum(param.grad) > 0:
                            # print(param.grad)
                            param.grad.zero_()
                            params[name].grad = None

    def restore_backup_stats(self):
        """
        Reset stored batch statistics from the stored backup.
        """
        for i in range(self.num_stages):
            self.layer_dict['conv{}'.format(i)].restore_backup_stats()


class StepLossAdapter(nn.Module):
    def __init__(self, input_dim, num_loss_net_layers, notspi, device):
        super(StepLossAdapter, self).__init__()

        self.device = device
        # self.args = args
        output_dim = num_loss_net_layers * 2 * 2 # 2 for weight and bias, another 2 for multiplier and offset

        self.linear1 = nn.Linear(input_dim, input_dim)
        self.activation = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(input_dim, output_dim)

        self.multiplier_bias = nn.Parameter(torch.zeros(output_dim // 2))
        self.offset_bias = nn.Parameter(torch.zeros(output_dim // 2))

    def forward(self, task_state, num_step, loss_params):
        # ψ
        out = self.linear1(task_state)
        out = F.relu_(out)
        out = self.linear2(out)

        generated_multiplier, generated_offset = torch.chunk(out, chunks=2, dim=-1)

        i = 0
        updated_loss_weights = dict()
        for key, val in loss_params.items():
            if 'step{}'.format(num_step) in key:
                updated_loss_weights[key] = (1 + self.multiplier_bias[i] * generated_multiplier[i]) * val + \
                                             self.offset_bias[i] * generated_offset[i]
                i+=1

        return updated_loss_weights


class LossAdapter(nn.Module):
    def __init__(self, input_dim, num_loss_net_layers, notspi, device):
        super(LossAdapter, self).__init__()

        self.device = device
        # self.args = args

        self.num_steps = notspi # number of inner-loop steps

        self.loss_adapter = nn.ModuleList()
        for i in range(self.num_steps): 
            self.loss_adapter.append(StepLossAdapter(input_dim, num_loss_net_layers, notspi=notspi, device=device))

    def forward(self, task_state, num_step, loss_params):
        return self.loss_adapter[num_step](task_state, num_step, loss_params)



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torch import _weight_norm
import numpy as np
from math import sqrt
import random

    
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, input_dim, num_heads=4):
        super(MultiHeadSelfAttention, self).__init__()
        self.dim_in = input_dim
        self.dim_k = input_dim//2
        self.dim_v = input_dim//2
        self.num_heads = num_heads
        self.linear_q = nn.Linear(input_dim, input_dim//2, bias=True)
        self.linear_k = nn.Linear(input_dim, input_dim//2, bias=True)
        self.linear_v = nn.Linear(input_dim, input_dim, bias=True)
        self._norm_fact = 1 / sqrt((input_dim//2) // num_heads)

    def forward(self, x, param=None):
        batch, n, dim_in = x.shape
        assert dim_in == self.dim_in

        nh = self.num_heads
        dk = self.dim_k // nh  # dim_k of each head
        dv = self.dim_v // nh  # dim_v of each head
        if param == None:
            q = self.linear_q(x).reshape(batch, n, nh, dk).transpose(1, 2)  # (batch, nh, n, dk)
            k = self.linear_k(x).reshape(batch, n, nh, dk).transpose(1, 2)  # (batch, nh, n, dk)
            v = self.linear_v(x).reshape(batch, n, nh, dv*2).transpose(1, 2)  # (batch, nh, n, dv)
        else:
            q = F.linear(x, weight=param[0], bias=param[1]).reshape(batch, n, nh, dk).transpose(1, 2)
            k = F.linear(x, weight=param[2], bias=param[3]).reshape(batch, n, nh, dk).transpose(1, 2)
            v = F.linear(x, weight=param[4], bias=param[5]).reshape(batch, n, nh, dv*2).transpose(1, 2)
        dist = torch.matmul(q, k.transpose(2, 3)) * self._norm_fact  # batch, nh, n, n
        dist = torch.softmax(dist, dim=-1)  # batch, nh, n, n

        att = torch.matmul(dist, v)  # batch, nh, n, dv
        out = att.transpose(1, 2).reshape(batch, n, self.dim_in)  # batch, n, dim_v
        
        # out = att.reshape(att.shape[0], -1)

        return out
    

class conv1d(nn.Module):
    def _init_(self):
        super(conv1d, self).__init__()
    
    def forward(self, x, weight, bias, stride, padding, dilation):
        return F.conv1d(x, weight, bias=bias, stride=stride, padding=padding, dilation=dilation)

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
        #                                    stride=stride, padding=padding, dilation=dilation)
        self.batchnorm1 = nn.BatchNorm1d(n_outputs)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        # self.relu1 = nn.Tanh()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
        #                                    stride=stride, padding=padding, dilation=dilation)
        self.batchnorm2 = nn.BatchNorm1d(n_outputs)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        # self.relu2 = nn.Tanh()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.batchnorm1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.batchnorm2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        # self.relu = nn.Tanh()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=5, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.num_inputs = num_inputs
        self.num_channels = num_channels
        layers = []
        # self.vars_bn = nn.ParameterList()
        num_levels = len(num_channels)
        # num_levels = num_channels
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.token = nn.Parameter(torch.ones(1, num_channels[-1]))
        # self.encoder_layer = nn.TransformerEncoderLayer(d_model=10, nhead= 5,\
        #                     dim_feedforward=10, batch_first=True)
        # self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        # self.features = nn.LSTM(input_size=num_channels[-1], hidden_size=16, num_layers=1, batch_first=True)
        # self.features = nn.GRU(input_size=num_channels[-1], hidden_size=16, num_layers=1, batch_first=True)
        # self.attention = TemporalAttention(num_channels[-1])
        self.attention = MultiHeadSelfAttention(num_channels[-1])
        self.mlp = nn.Sequential(
            nn.Linear(num_channels[-1], num_channels[-1]),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.ReLU(), 
            nn.Linear(num_channels[-1], 1)
        )
        # self.features = nn.Linear(num_channels[-1], num_channels[-1])
        # self.decoder = nn.Linear(num_channels[-1], 1)

        # encoder_layer = nn.TransformerEncoderLayer(d_model=num_channels[i-1], nhead= 4,\
        #                     dim_feedforward=num_channels[i-1], batch_first=False)
        # self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
        # self.tanh = nn.Tanh()

    def forward(self, x, weights=None):
        device =  x[0].device
        if weights == None:
            # transformer output:
            # x = self.transformer_encoder(x) + x
            
            output = self.network(x)
            output = output.transpose(1, 2)
            result = torch.cat([output, self.token.unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result)
            output = output[:, -1, :]
            # output = self.attention(output)
            # linear output:
            # output_shape = output.shape
            # output = output.reshape(output_shape[0], output_shape[1] * output_shape[2])
            # h_n = self.features(output)
            
            # LSTM output:
            # enc_x, (h_n, c_n) = self.features(output)
            
            # GRU output:
            # enc_x, h_n = self.features(output)

            # if len(h_n.shape) == 3:
            #     h_n = h_n[-1] # (32,16)
            
            # 通过线性层和激活函数得到最终输出
            # out = self.decoder(F.relu(h_n))
            # out = self.decoder(F.leaky_relu(h_n, negative_slope=0.01))
            out = self.mlp(output)
        else:
            # TemporalBlock0
            # 实际权重 = 模数 * 方向向量
            ks = self.kernel_size
            weight1 = _weight_norm(weights[3], weights[2], 0)
            conv1d1 = weight_norm(nn.Conv1d(self.num_inputs, self.num_channels[0], kernel_size=ks, stride=1, padding=(ks-1)*1, dilation=1))
            output1 = conv1d1._conv_forward(x, weight=weight1, bias=weights[1])
            # output1 = F.conv1d(x, weight=weights[1], bias=weights[2], stride=1, padding=(ks-1)*1, dilation=1)
            running_mean1 = nn.Parameter(torch.zeros(self.num_channels[0]), requires_grad=False).to(device)
            running_var1 = nn.Parameter(torch.ones(self.num_channels[0]), requires_grad=False).to(device)
            output1 = F.batch_norm(self.network[0].chomp1(output1), running_mean=running_mean1, running_var=running_var1, weight=weights[4], bias=weights[5])
            # output1 = F.tanh(output1)
            output1 = F.relu(output1)
            output1 = F.dropout(output1, self.dropout)
            weight2 = _weight_norm(weights[8], weights[7], 0)
            conv1d2 = weight_norm(nn.Conv1d(self.num_channels[0], self.num_channels[0], kernel_size=ks, stride=1, padding=(ks-1)*1, dilation=1))
            output1 = conv1d2._conv_forward(output1, weight=weight2, bias=weights[6])
            # output1 = F.conv1d(output1, weight=weights[5], bias=weights[6], stride=1, padding=(ks-1)*1, dilation=1)
            running_mean2 = nn.Parameter(torch.zeros(self.num_channels[0]), requires_grad=False).to(device)
            running_var2 = nn.Parameter(torch.ones(self.num_channels[0]), requires_grad=False).to(device)
            output1 = F.batch_norm(self.network[0].chomp2(output1), running_mean=running_mean2, running_var=running_var2, weight=weights[9], bias=weights[10])
            # output1 = F.tanh(output1)
            output1 = F.relu(output1)
            output1 = F.dropout(output1, self.dropout)
            res1 = F.conv1d(x, weights[11], bias=weights[12], stride=1)
            # output1 = F.tanh(output1 + res1)
            output1 = F.relu(output1 + res1)

            # TemporalBlock1
            weight3 = _weight_norm(weights[15], weights[14], 0)
            conv1d3 = weight_norm(nn.Conv1d(self.num_channels[0], self.num_channels[1], kernel_size=ks, stride=1, padding=(ks-1)*2, dilation=2))
            output2 = conv1d3._conv_forward(output1, weight=weight3, bias=weights[13])
            # output2 = F.conv1d(output1, weight=weights[11], bias=weights[12], stride=1, padding=(ks-1)*2, dilation=2)
            running_mean3 = nn.Parameter(torch.zeros(self.num_channels[1]), requires_grad=False).to(device)
            running_var3 = nn.Parameter(torch.ones(self.num_channels[1]), requires_grad=False).to(device)
            output2 = F.batch_norm(self.network[1].chomp1(output2), running_mean=running_mean3, running_var=running_var3, weight=weights[16], bias=weights[17])
            # output2 = F.tanh(output2)
            output2 = F.relu(output2)
            output2 = F.dropout(output2, self.dropout)
            weight4 = _weight_norm(weights[20], weights[19], 0)
            conv1d4 = weight_norm(nn.Conv1d(self.num_channels[1], self.num_channels[1], kernel_size=ks, stride=1, padding=(ks-1)*2, dilation=2))
            output2 = conv1d4._conv_forward(output2, weight=weight4, bias=weights[18])
            # output2 = F.conv1d(output2, weight=weights[15], bias=weights[16], stride=1, padding=(ks-1)*2, dilation=2)
            running_mean4 = nn.Parameter(torch.zeros(self.num_channels[1]), requires_grad=False).to(device)
            running_var4 = nn.Parameter(torch.ones(self.num_channels[1]), requires_grad=False).to(device)
            output2 = F.batch_norm(self.network[1].chomp2(output2), running_mean=running_mean4, running_var=running_var4, weight=weights[21], bias=weights[22])
            # output2 = F.tanh(output2)
            output2 = F.relu(output2)
            output2 = F.dropout(output2, self.dropout)
            res2 = F.conv1d(output1, weights[23], bias=weights[24], stride=1)
            # output2 = F.tanh(output2 + res2)
            output = F.relu(output2 + res2)

            output = output.transpose(1, 2)
            result = torch.cat([output, weights[0].unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result, weights[25:31])
            output = output[:, -1, :]

            # output = output.transpose(1, 2)
            # output_shape = output.shape
            # output = output.reshape(output_shape[0], output_shape[1] * output_shape[2])

            # out = F.leaky_relu(F.linear(output, weight=weights[31] , bias=weights[32]), negative_slope=0.01)
            out = F.relu(F.linear(output, weight=weights[31] , bias=weights[32]))
            out = F.linear(out, weight=weights[33], bias=weights[34])

        return out
    


# MAML-跟车  version1
# %matplotlib inline
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import partial
import copy
import random
import logging
from torch.utils.tensorboard import SummaryWriter
# from net.TCN import TemporalConvNet
# from net.TCN_wn_bn import TemporalConvNet
# from net.TCN_layer import TemporalConvNet
# from net.TCN import MultiHeadSelfAttention
# from net.meta_neural_network_architectures import MetaLossNetwork, LossAdapter

# 设置随机种子
SEED = 22
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# writer = SummaryWriter("/home/ubuntu/fedavg/FollowNet-car/SummaryWriter/MAML_MeTAL_log")

# ACC_LIMIT = 3 # the limit of acceleration, this can be calibrated based on the data 加速度极限
# Ts = 0.04 # time interval for data sampling for HighD is 0.04 for other datasets are 0.1 数据采样时间间隔 15/375s
Ts = 0.1

# load data    加载包含Python对象的.npy或.npz文件，需要显式地设置allow_pickle参数为True
HighD_data = np.load('/kaggle/input/follow-car-datasets/data/HighD_car_fol_event_set_dhw.npy', allow_pickle=True)
NGSIM_data = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_car_fol_event_set.npy', allow_pickle=True)
SPMD1_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das1_car_fol_event_set.npy', allow_pickle=True)
SPMD2_data = np.load('/kaggle/input/follow-car-datasets/data/SPMD_das2_car_fol_event_set.npy', allow_pickle=True)
Waymo_data = np.load('/kaggle/input/follow-car-datasets/data/waymo_filter_carfo_info_150.npy', allow_pickle=True)
Lyft_data = np.load('/kaggle/input/lyft-data/Lyft_car_fol_event_set.npy', allow_pickle=True)
Lyft_test = np.load('/kaggle/input/follow-car-datasets/data/Lyft_test_data.npy', allow_pickle=True)
Lyft_train = np.load('/kaggle/input/follow-car-datasets/data/Lyft_train_data.npy', allow_pickle=True)
Lyft_val = np.load('/kaggle/input/follow-car-datasets/data/Lyft_val_data.npy', allow_pickle=True)
HighD_test = np.load('/kaggle/input/follow-car-datasets/data/HighD_test_data.npy', allow_pickle=True)
HighD_train = np.load('/kaggle/input/follow-car-datasets/data/HighD_train_data.npy', allow_pickle=True)
HighD_val = np.load('/kaggle/input/follow-car-datasets/data/HighD_val_data.npy', allow_pickle=True)
NGSIM_test = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_test_data.npy', allow_pickle=True)
NGSIM_train = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_train_data.npy', allow_pickle=True)
NGSIM_val = np.load('/kaggle/input/follow-car-datasets/data/NGSIM_I_80_val_data.npy', allow_pickle=True)
SPMD1_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_test_data.npy', allow_pickle=True)
SPMD1_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_train_data.npy', allow_pickle=True)
SPMD1_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD1_val_data.npy', allow_pickle=True)
SPMD2_test = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_test_data.npy', allow_pickle=True)
SPMD2_train = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_train_data.npy', allow_pickle=True)
SPMD2_val = np.load('/kaggle/input/follow-car-datasets/data/SPMD2_val_data.npy', allow_pickle=True)
Waymo_test = np.load('/kaggle/input/follow-car-datasets/data/Waymo_test_data.npy', allow_pickle=True)
Waymo_train = np.load('/kaggle/input/follow-car-datasets/data/Waymo_train_data.npy', allow_pickle=True)
Waymo_val = np.load('/kaggle/input/follow-car-datasets/data/Waymo_val_data.npy', allow_pickle=True)
# 使用训练设备
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
gpu = 0
device = torch.device('cuda:{}'.format(gpu) if torch.cuda.is_available() and gpu != -1 else 'cpu')
print(device)


# 保存日志
def get_logger(filename, verbosity=1, name=None):
    # 设置不同verbosity对应的日志级别
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    # 设置日志输出格式
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
    # 创建文件处理器，将日志写入文件
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    # 创建控制台处理器，将日志输出到控制台
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    return logger
# 日志保存路径
# dataset = 'NGSIM'
logger = get_logger(f'/kaggle/working/MeTAL_Waymo_{device}_tcnks5_1632_a0.001_b0.01_step1_k200_K40_Mese_acc_ht.log')

# 任务数据集划分
def split_train(data,data_ratio):
#     np.random.seed(43)
    # 随机排列数据集的索引
    shuffled_indices=np.random.permutation(len(data))
    if data_ratio > 0 and data_ratio <= 1:
        # 如果小于等于1，根据输入百分比计算获取数据集的数量
        data_size=int(len(data)*data_ratio)
        data_num = int(data_size/2)
        # 根据索引划分数据集
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_size]   # query set
    else:
        # 如果大于1，则data_ratio为获取数据集中的数量
        data_num = int(data_ratio/2)
        data_indices1 = shuffled_indices[:data_num]             # support set
        data_indices2 = shuffled_indices[data_num:data_ratio]   # query set
    return data[data_indices1], data[data_indices2]
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


# max_len = 375 # for HighD dataset is 375 for others are 150
max_len = 150
his_horizon = 10


# 将数据集划分成样本
class ImitationCarFolData(torch.utils.data.Dataset):
    """
    Dataset class for imitation learning (state -> action fitting) based car-following models.
    """
    def __init__(self, data, max_len = max_len, Ts = 0.1):
        self.data = data
        self.max_len = max_len # Max length of a car following event.
        self.Ts = Ts

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 获取数据集中指定索引的车辆跟驰事件
        event = self.data[idx]
        # 截取事件中的max_len个时间步数据
        sv = event[1][:self.max_len] # 后车速度
        lv = event[3][:self.max_len] # 前车速度
        spacing = event[0][:self.max_len] # 车距
        relSpeed = event[2][:self.max_len] # 相对速度
        # 构建输入数据，包括 spacing, sv, relSpeed，截取除最后一项的全部数据
        inputs = [spacing[:-1], sv[:-1], relSpeed[:-1]]
        # 计算加速度标签（当前速度减去上一个时间步的速度）
        acc_label = np.diff(sv) / self.Ts
        # 保存 lv 作为速度标签
        lv_spd = lv
        return {'inputs': inputs, 'label': acc_label, 'lv_spd': lv_spd}


# nn模型
class nn_model(nn.Module):
    def __init__(self, input_size = 3):
        super(nn_model,self).__init__()
        self.net=nn.Sequential(
                nn.Linear(input_size,256),
                nn.ReLU(),
                nn.Linear(256,256),
                nn.ReLU(),
                nn.Linear(256,1)
            )
        self.input_size = input_size
        self.state = list(self.net.parameters())
    # 用于元学习的内循环中更新权重的方法，用于计算元损失，然后重置下一个元任务的权重。
    def forward(self,x,weights=None):
        if weights == None:
            x = self.net(x)
        else:
            x=F.linear(x,weights[0],weights[1])  # (input,weight,bias)
            x=F.relu(x)
            x=F.linear(x,weights[2],weights[3])
            x=F.relu(x)
            x=F.linear(x,weights[4],weights[5])
        return x

# 1dcnn模型
class CNN1D(nn.Module):
    def __init__(self, input_size=3, num_classes=1, dropout=0.2):
        super(CNN1D, self).__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout))
        self.attention = MultiHeadSelfAttention(32)
        self.mlp = nn.Sequential(
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
        self.token = nn.Parameter(torch.ones(1, 32))
        self.input_size = input_size
        self.num_classes = num_classes
        self.dropout = dropout

    def forward(self, x, weights=None): # input x(150, 3, 10)
        if weights == None:
            output = self.net(x)
            output = output.transpose(1, 2)
            result = torch.cat([output, self.token.unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result)
            output = output[:, -1, :]

            out = self.mlp(output)
        else:
            # bn = nn.BatchNorm1d(32)
            # print(bn.running_mean, bn.running_var, bn.weight, bn.bias)
            running_mean1 = nn.Parameter(torch.zeros(8), requires_grad=False).to(device)
            running_var1 = nn.Parameter(torch.ones(8), requires_grad=False).to(device)
            output = F.conv1d(x, weights[1],weights[2], stride=1, padding=1)   # (150, 32, 10)
            output = F.batch_norm(output, running_mean=running_mean1, running_var=running_var1, weight=weights[3], bias=weights[4])
            output = F.relu(output)   # (150, 32, 10)
            output = F.dropout(output, p=self.dropout)

            running_mean2 = nn.Parameter(torch.zeros(16), requires_grad=False).to(device)
            running_var2 = nn.Parameter(torch.ones(16), requires_grad=False).to(device)
            output = F.conv1d(output, weights[5],weights[6], stride=1, padding=1)   # (150, 64, 10)
            output = F.batch_norm(output, running_mean=running_mean2, running_var=running_var2, weight=weights[7], bias=weights[8])
            output = F.relu(output)   # (150, 64, 10)
            output = F.dropout(output, p=self.dropout)

            running_mean3 = nn.Parameter(torch.zeros(32), requires_grad=False).to(device)
            running_var3 = nn.Parameter(torch.ones(32), requires_grad=False).to(device)
            output = F.conv1d(output, weights[9],weights[10], stride=1, padding=1)   # (150, 128, 10)
            output = F.batch_norm(output, running_mean=running_mean3, running_var=running_var3, weight=weights[11], bias=weights[12])
            output = F.relu(output)   # (150, 128, 10)
            output = F.dropout(output, p=self.dropout)

            output = output.transpose(1, 2)
            result = torch.cat([output, weights[0].unsqueeze(0).expand(x.shape[0], -1, -1)], dim=1)
            output = self.attention(result, weights[13:19])
            output = output[:, -1, :]

            out = F.linear(output, weights[19], weights[20]) # (150, 128)
            out = F.relu(out)   # (150, 128)
            out = F.linear(out, weights[21], weights[22]) # (150, 1)
        return out

# 计算ttc
def calculate_safety(ttc):
    minimum_ttc = min(ttc)
    return minimum_ttc
# 多数据同时验证
def model_evaluate(model,his_horizon,testdata):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    model.eval()

    jerk_set = torch.empty(0).to(device)
    error_set = torch.empty(0).to(device)
    minimum_ttc_set = torch.empty(0).to(device)

    criterion = nn.MSELoss()
    count = 0
    col = 0
    for i, item in tqdm(enumerate(testdata)):#每个样本挨个出来
        
        x_data, y_data = item['inputs'], item['label']
        x_data = torch.stack(x_data)#3x149
        if model_type == 'cnn1d' or model_type == 'tcn':
            # x->(bs,3,149)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
            B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
        else:
            # x->(149,bs,3)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
            T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
        lv_spd = item['lv_spd'].float().to(device).transpose(0, 1)
        count += B
        x_data_orig = x_data.clone().detach()#提前克隆的副本
        
        col_list = torch.full((B,), -1).to(device)
        acc_batch = torch.empty(B,0).to(device)
        ttc_batch = torch.empty(B,0).to(device)

        
        for frame in range(his_horizon, T):#对32个样本检测
            if model_type == 'cnn1d' or model_type == 'tcn':
                x= x_data[:, :, frame-his_horizon:frame]   # (20, 3, 10)
            else:
                x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)

            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            #x = x.transpose(0,1).reshape(B, -1)
            acc_pre = model(x).squeeze()
            if acc_pre.dim() == 0:
                acc_pre = acc_pre.unsqueeze(0)
            acc_batch = torch.cat((acc_batch, acc_pre.unsqueeze(1)), dim=1)#记录所有acc     
            if model_type == 'cnn1d' or model_type == 'tcn':
                if frame < T-1:
                    # 根据当前速度和加速度计算下一时间速度
                    sv_spd_ = x_data[:, 1, frame] + acc_pre*Ts # (32)
                    sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)
                    # 计算下一时间速度的相对速度
                    delta_v_ = lv_spd[frame + 1] - sv_spd_ # (32)
                    # 该时刻真实相对速度
                    delta_v = x_data[:, -1, frame] # (32)
                    # 通过两车车距加上相对位移得到下一时间段车距 ？
                    spacing_ = x_data[:, 0, frame] + Ts*(delta_v + delta_v_)/2 # (32)
                    for i in range(len(spacing_)):  # 遍历每一行
                        if spacing_[i] < 0 and col_list[i] == -1:    # 如果这一行包含0
                            col_list[i] = frame+1
                    # update 根据计算得到的值，更新下一时间的值
                    next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
                    x_data[:, :, frame + 1] = next_frame_data
            else:
                if frame < T-1:#开环训练，不断依照预测值更新下一个时间节点的数据
                    sv_spd_ = x_data[frame, :, 1] + acc_pre*Ts#新的自车速度
                    sv_spd_ = torch.tensor(np.maximum(np.array(sv_spd_.detach().cpu()),0.001), device=device)#保证速度不小于等于0
                    delta_v_ = lv_spd[frame + 1] - sv_spd_#vt临时计算的
                    delta_v = x_data[frame, :, -1]#v0原有的
                    spacing_ = x_data[frame, :, 0] + Ts*(delta_v + delta_v_)/2#新的距离
                    for i in range(len(spacing_)):  # 遍历每一行
                        if spacing_[i] < 0 and col_list[i] == -1:    # 如果这一行包含0
                            col_list[i] = frame+1
                    next_frame_data = torch.stack((spacing_, sv_spd_, delta_v_)).transpose(0, 1) # B, 3
                    x_data[frame + 1] = next_frame_data
            if spacing_.dim() == 0:
                ttc_batch_ = (-spacing_ / delta_v_).unsqueeze(0)
            else:
                ttc_batch_ = (-spacing_ / delta_v_)
                
            ttc_batch = torch.cat((ttc_batch, ttc_batch_.unsqueeze(1)), dim=1)
        for i in range(B):#对32个样本统一处理
            if model_type == 'cnn1d' or model_type == 'tcn':
                spacing_obs = x_data_orig[i,0,...]
                if(col_list[i] != -1):
                    acc_batch_ = acc_batch[i,:col_list[i]-his_horizon]
                    ttc_batch_ = ttc_batch[i,:col_list[i]-his_horizon]#修改
                    spacing_pre_ = x_data[i, 0, :col_list[i]]
                    error_single = criterion(spacing_pre_[:col_list[i]], spacing_obs[:col_list[i]]).item()
                else:
                    acc_batch_ = acc_batch[i,:]
                    ttc_batch_ = ttc_batch[i,:]
                    spacing_pre_ = x_data[i, 0, :]
                    error_single = criterion(spacing_pre_[:], spacing_obs[:]).item()
            else:
                spacing_obs = x_data_orig[...,i,0]
                if(col_list[i] != -1):
                    acc_batch_ = acc_batch[i,:col_list[i]-his_horizon]
                    ttc_batch_ = ttc_batch[i,:col_list[i]-his_horizon]#修改
                    spacing_pre_ = x_data[:col_list[i],i, 0]
                    error_single = criterion(spacing_pre_[:col_list[i]], spacing_obs[:col_list[i]]).item()
                else:
                    acc_batch_ = acc_batch[i,:]
                    ttc_batch_ = ttc_batch[i,:]
                    spacing_pre_ = x_data[:,i, 0]
                    error_single = criterion(spacing_pre_[:], spacing_obs[:]).item()
            acc_batch_ = acc_batch_.cpu().detach().numpy()
            jerk_single = np.mean(np.abs(np.diff(acc_batch_)/Ts))
            TTC_single = [x for x in ttc_batch_ if x >= 0]#除去其中TTC小于0的
            if len(TTC_single) > 0:
                minimum_ttc_single = calculate_safety(TTC_single)#该样本的最小TTC
                minimum_ttc_set= torch.cat((minimum_ttc_set,torch.tensor(minimum_ttc_single).unsqueeze(0).to(device)), dim=0)      
            jerk_set= torch.cat((jerk_set,torch.tensor(jerk_single).unsqueeze(0).to(device)), dim=0)
            error_set= torch.cat((error_set,torch.tensor(error_single).unsqueeze(0).to(device)), dim=0)
        col = col + torch.sum(col_list != -1).item()
    if len(minimum_ttc_set) == 0:
        ttc = 0
    else:
        ttc = sum(minimum_ttc_set)/len(minimum_ttc_set)
    error = sum(error_set)/len(error_set)
    return error,col,count,sum(jerk_set)/len(jerk_set),ttc#还需修改（以MSE为基准）


def finetuning(best_mse_state):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    # Train
    Ts = 0.1
    max_len = 150
    og_net = maml.net
    # 创建一个与原始网络结构相同的虚拟网络
    if model_type == 'nn':
        dummy_net = nn_model(input_size = his_horizon*3)
    elif model_type == 'tcn':
        dummy_net = TemporalConvNet(num_inputs=3, num_channels=(16,32))
    elif model_type == 'cnn1d':
        dummy_net = CNN1D()
        # dummy_net = cbrd(num_inputs=3, num_output=1)
    dummy_net=dummy_net.to(device)

    # 多gpu训练
    # if torch.cuda.device_count() > 1:   # 查看当前电脑的可用的gpu的数量，若gpu数量>1,就多gpu训练
    #     dummy_net = torch.nn.DataParallel(dummy_net)  # 自动选择gpu
    # dummy_net.to(device)

    # 加载原始网络的权重
    # weight_path = torch.load(state_path)
    # dummy_net.load_state_dict(weight_path.state_dict())
    dummy_net.load_state_dict(og_net.state_dict())
    # 进行迭代，每次更新虚拟网络的参数
    num_shots=5
    lr = 0.01
    loss_fn=nn.MSELoss()
    optim=torch.optim.Adam
    opt=optim(dummy_net.parameters(),lr=lr, weight_decay=3e-4)


    # 数据集划分
    def split_data(data,data_ratio):
        # random.seed(SEED)
        # np.random.seed(SEED)
        # torch.manual_seed(SEED)
        if data_ratio > 0 and data_ratio <= 1:
            # 如果小于等于1，根据输入百分比计算获取数据集的数量
            data_size=int(len(data)*data_ratio)
        else:
            # 如果大于1，则data_ratio为获取数据集中的数量
            data_size = data_ratio
        return data[:data_size]
    # np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

    # 获取数据集的数量
    K=40
    dataset_train = split_data(train_data, K)
    dataset_loader_train = ImitationCarFolData(dataset_train, max_len = max_len, Ts = Ts)
    train_loader = DataLoader(
                dataset_loader_train,
                batch_size=10,
                shuffle=True,
                num_workers=1,
                drop_last=True)
    dataset_test = test_data
    dataset_loader_test = ImitationCarFolData(dataset_test, max_len = max_len, Ts = Ts)
    test_loader = DataLoader(
                dataset_loader_test,
                batch_size=96,
                shuffle=False,
                num_workers=1,
                drop_last=False)
    # dataset_val = NGSIM_val
    # dataset_loader_val = ImitationCarFolData(dataset_val, max_len = max_len, Ts = Ts)
    # val_loader = DataLoader(
    #             dataset_loader_val,
    #             batch_size=10,
    #             shuffle=True,
    #             num_workers=1,
    #             drop_last=True)


    # 初始化变量
    train_loss_his = [] # 训练损失
    test_error_his = [] # 测试误差
    best_train_loss = None # 最佳训练损失

    # 训练过程
    best_error = 10000
    for epoch in tqdm(range(num_shots)):
        train_losses = [] # 记录每个epoch的训练损失
        validation_losses = [] # 记录每个epoch的验证损失
        jerk_val = 0
        dummy_net.train()
        # 遍历数据集
        for i, item in enumerate(train_loader):
            # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
            # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
            x_data, y_data = item['inputs'], item['label'] # x_data((32,374)，(32,374)，(32,374))  y_data(32,374)
            # Put T into the first dimension, B, T, d -> T, B, d
            # 将x_data中3个(32,374)连接，转换成(3,32,374)
            x_data = torch.stack(x_data)
            # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
            if model_type == 'cnn1d' or model_type == 'tcn':
                # x->(bs,3,149)，y->(149,bs)
                x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
                B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
            else:
                # x->(149,bs,3)，y->(149,bs)
                x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
                T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
            # print(T,B,d)  # 149  20  3
                
            y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (364,32)
            y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (364,32)
            # 从历史数据时间步开始遍历
            for frame in range(his_horizon, T):
                if model_type == 'cnn1d' or model_type == 'tcn':
                    x= x_data[:, :, frame-his_horizon:frame]
                else:
                    x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
                if model_type == 'nn':
                    x = x.transpose(0,1).reshape(B, -1) # flatten for history data
                # 根据his_horizon个数据预测加速度
                acc_pre = dummy_net(x).squeeze() # (32)
                y_pre[frame - his_horizon] = acc_pre
            #计算损失并进行反传及优化
            loss = loss_fn(y_pre, y_label)
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(dummy_net.parameters(), 0.25) # 对模型梯度进行裁剪，避免梯度爆炸
            opt.step()

            train_losses.append(loss.item())
        # 计算本轮平均损失
        train_loss = np.mean(train_losses)

        train_loss_his.append(train_loss)
#         print("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))
        logger.info("Epoch: {0}| Train Loss: {1:.7f}".format(epoch + 1, train_loss))

        mean_spacing_error,col,count,jerk,miniumu_ttc = model_evaluate(dummy_net,his_horizon,test_loader)
        logger.info("mean_spacing_error：{:.5f}，col={}，count={}，rate={:.5f}%，jerk={:.5f}，miniumu_ttc={:.5f}".format(mean_spacing_error, col, count, (col/count)*100, jerk, miniumu_ttc))
        if epoch == 0:
            first_step_error = mean_spacing_error
        if mean_spacing_error < best_error:
            best_error = mean_spacing_error
        if mean_spacing_error < best_mse_state:
            best_mse_state = mean_spacing_error
            # save the best model
            with open(save_path, 'wb') as f:
                torch.save(dummy_net, f)
    return best_error, first_step_error, best_mse_state



# 元学习内循环训练过程
def inner_train(data_loader, net ,temp_weights, criterion, metal=None):
    net.train()
    # 遍历数据集
    for i, item in enumerate(data_loader):
        # x_data, y_data = item['inputs'].float().to(device), item['label'].float().to(device)
        # 获取输入数据及标签，input包括[距离，后车速度，相对速度]，label为加速度
        x_data, y_data = item['inputs'], item['label'] # x_data((20,149)，(20,149)，(20,149))  y_data(20,149)
        # Put T into the first dimension, B, T, d -> T, B, d
        # 将x_data中3个(20,149)连接，转换成(3,20,149)
        x_data = torch.stack(x_data)
        # x_data, y_data = x_data.transpose(0, 1), y_data.transpose(0, 1)
        if model_type == 'cnn1d' or model_type == 'tcn':
            # x->(bs,3,149)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 1).float().to(device), y_data.transpose(0, 1).float().to(device)
            B, d, T = x_data.shape # (total steps, batch_size, d) as the shape of the data
        else:
            # x->(149,bs,3)，y->(149,bs)
            x_data, y_data = x_data.transpose(0, 2).float().to(device), y_data.transpose(0, 1).float().to(device)
            T, B, d = x_data.shape # (total steps, batch_size, d) as the shape of the data
        # print(T,B,d)  # 149  20  3
            
        y_pre = torch.zeros(T - his_horizon, B).to(device) # 存储预测加速度 (139,20)
        y_label = y_data[his_horizon:] # 存储从his_horizon开始的真实加速度 (139,20)
        # 从历史数据时间步开始遍历
        for frame in range(his_horizon, T):
            if model_type == 'cnn1d' or model_type == 'tcn':
                x= x_data[:, :, frame-his_horizon:frame]
            else:
                x = x_data[frame-his_horizon:frame] # (his_horizon, B, d) (10, 20, 3)
            if model_type == 'nn':
                x = x.transpose(0,1).reshape(B, -1) # flatten for history data (20, 30)
            # 根据his_horizon个数据预测加速度
            acc_pre = net(x, temp_weights).squeeze() # (32)
            # acc_pre = net.module(x, temp_weights).squeeze()
            y_pre[frame - his_horizon] = acc_pre
        #计算损失并进行反传及优化
        loss = criterion(y_pre, y_label)
        if metal == None:
            return loss
        else:
            return loss, y_pre


# 生成不同数据集的任务类
class DataTask():
    def __init__(self,data_range):
        self.data_range = data_range
  # 生成任务数据，默认生成一个数据点，可以通过指定size参数生成多个数据点
    def sample_data(self,size=1):
        if self.data_range == 1:
            meta_data = HighD_data
            Ts = 0.04
            max_len = 375
            if adapt_tasks:
                task_net = HighD_net
            # print('--HighD--')
        elif self.data_range == 2:
            meta_data = Lyft_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = Lyft_net
            # print('--Lyft---')
        elif self.data_range == 3:
            meta_data = SPMD1_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = SPMD1_net
            # print('SPMD_DAS1')
        elif self.data_range == 4:
            meta_data = SPMD2_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = SPMD2_net
            # print('SPMD_DAS2')
        elif self.data_range == 5:
            meta_data = NGSIM_data
            Ts = 0.1
            max_len = 150
            if adapt_tasks:
                task_net = Waymo_net
            # print('--Waymo--')
        meta_data_set1, meta_data_set2 = split_train(meta_data, size)
#         print(len(meta_data_set1),len(meta_data_set2))
        dataset_loader1 = ImitationCarFolData(meta_data_set1, max_len = max_len, Ts = Ts)
        dataset_loader2 = ImitationCarFolData(meta_data_set2, max_len = max_len, Ts = Ts)
        if adapt_tasks:
            return dataset_loader1, dataset_loader2, task_net
        else:
            return dataset_loader1, dataset_loader2


# 任务分布，及选择不同的数据
class DataDistribution():
    def __init__(self, data_range=5, n=3):
        self.data_range = data_range    # 获取数据集的数值
        self.n = n
    # 随机生成一个跟车数据集
    def sample_task(self, epoch):
        # 从1到5中随机选择3个不重复的数
        # 创建一个新的随机数生成器实例
        rng = np.random.RandomState(epoch)
        data_choice = rng.choice(self.data_range, size=self.n, replace=False) + 1
        # data_choice = np.random.choice(self.data_range, size=self.n, replace=False) + 1
        return data_choice


def attenuate_init(attenuator, panning, task_embeddings, names_weights_copy):
        ## Attenuate
        updated_names_weights_copy = list()
        
        # 生成衰减参数和偏移参数
        if attenuator == None:
            beta = panning(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(key + beta[i])
                i+=1
        elif panning == None:
            gamma = attenuator(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(gamma[i] * key)
                i+=1
        else:
            gamma = attenuator(task_embeddings)
            beta = panning(task_embeddings)
            i = 0
            for key in names_weights_copy:
                updated_names_weights_copy.append(gamma[i] * key + beta[i])
                i+=1
            
        return updated_names_weights_copy


def get_inner_loop_parameter_dict(params):
    """
    Returns a dictionary with the parameters to use for inner loop updates.
    :param params: A dictionary of the network's parameters.
    :return: A dictionary of the parameters to use for the inner loop optimization process.
    """
    param_dict = dict()
    for name, param in params:
        if param.requires_grad:
            param_dict[name] = param.to(device)

    return param_dict

def get_per_step_loss_importance_vector(inner_step, current_epoch):
        """
        Generates a tensor of dimensionality (num_inner_loop_steps) indicating the importance of each step's target
        loss towards the optimization loss.
        :return: A tensor to be used to compute the weighted average of the loss, useful for
        the MSL (Multi Step Loss) mechanism.
        """
        multi_step_loss_num_epochs = 20
        loss_weights = np.ones(shape=(inner_step)) * (
                1.0 / inner_step)
        decay_rate = 1.0 / inner_step / multi_step_loss_num_epochs
        min_value_for_non_final_losses = 0.03 / inner_step
        for i in range(len(loss_weights) - 1):
            curr_value = np.maximum(loss_weights[i] - (current_epoch * decay_rate), min_value_for_non_final_losses)
            loss_weights[i] = curr_value

        curr_value = np.minimum(
            loss_weights[-1] + (current_epoch * (inner_step - 1) * decay_rate),
            1.0 - ((inner_step - 1) * min_value_for_non_final_losses))
        loss_weights[-1] = curr_value
        loss_weights = torch.Tensor(loss_weights).to(device)
        return loss_weights


# MAML元学习训练过程
class MAML():
    def __init__(self, net, alpha, beta, tasks, k, weight_decay, inner_step):  # (net,alpha=0.01,beta=0.0001,tasks=data_tasks,k=150)
        # 初始化 MAML 算法的元参数
        self.net = net  # 元学习使用的神经网络模型
        self.weights = list(net.parameters())  # 获取神经网络的参数
        self.alpha = alpha  # 内循环学习率
        self.beta = beta  # 外循环学习率
        self.inner_step = inner_step  # 内部循环更新的步数
        self.tasks = tasks  # 正弦任务的分布
        self.k = k  # 内循环（任务采样）的数据集大小
        self.criterion = nn.MSELoss()  # 均方误差损失函数
        self.meta_losses = []  # 用于存储元学习的损失历史
        self.meta_test = []
        self.finetuning_error = []
        self.plot_every = 10  # 每隔多少次迭代记录一次损失，用于绘图
        self.print_every = 1  # 每隔多少次迭代输出一次损失信息，用于打印
        self.first_add = 0  # 用于添加优化器优化的参数
        self.all_best_state = 10000 # 用于存储微调后最好mse时的模型权重
        self.batch_size = int(self.k/2)
        # 设置权重衰减因子
        self.weight_decay = weight_decay
        self.num_layers = len(self.weights)
        params = [{'params': self.weights}]
        if MeTAL:
            base_learner_num_layers = len(self.weights)
            support_meta_loss_num_dim = base_learner_num_layers + 2 + 1
            support_adapter_num_dim = base_learner_num_layers + 1

            self.meta_loss = MetaLossNetwork(support_meta_loss_num_dim, notspi=self.inner_step, device=device).to(device=device)
            self.meta_loss_adapter = LossAdapter(support_adapter_num_dim, num_loss_net_layers=2, notspi=self.inner_step, device=device).to(device=device)
            new_params = {'params': self.meta_loss.parameters()}
            params.append(new_params)
            new_params = {'params': self.meta_loss_adapter.parameters()}
            params.append(new_params)

            if semi_supervised:
                query_num_dim = base_learner_num_layers + 1 + 1
                self.meta_query_loss = MetaLossNetwork(query_num_dim, notspi=self.inner_step, device=device).to(device=device)
                self.meta_query_loss_adapter = LossAdapter(query_num_dim, num_loss_net_layers=2, notspi=self.inner_step, device=device).to(device=device)
                new_params = {'params': self.meta_query_loss.parameters()}
                params.append(new_params)
                new_params = {'params': self.meta_query_loss_adapter.parameters()}
                params.append(new_params)

        if attenuate:  #L2F
            self.attenuator = nn.Sequential(
                nn.Linear(self.num_layers, self.num_layers),
                nn.ReLU(inplace=True),
                nn.Linear(self.num_layers, self.num_layers),
                nn.Sigmoid()
            ).to(device=device)
            new_params = {'params': self.attenuator.parameters()}
            params.append(new_params)
        if tapt:  #bias
            self.panning = nn.Sequential(
                nn.Linear(self.num_layers, self.num_layers),
                nn.ReLU(inplace=True),
                nn.Linear(self.num_layers, self.num_layers),
                nn.Tanh()
            ).to(device=device)
            new_params = {'params': self.panning.parameters()}
            params.append(new_params)
        if alfa:  # ALFA
            input_dim = self.num_layers*2
            self.regularizer = nn.Sequential(
                nn.Linear(input_dim, input_dim),
                nn.ReLU(inplace=True),
                nn.Linear(input_dim, input_dim)
            ).to(device=device)
            new_params = {'params': self.regularizer.parameters()}
            params.append(new_params)
        self.meta_optimiser = torch.optim.Adam(params,lr=self.beta, weight_decay=3e-4)

    def inner_loop(self, task):
        taskloss = []
        # 内循环更新参数，用于计算元学习损失
        temp_weights = [w.clone() for w in self.weights]  # 复制当前网络参数作为临时参数
        if adapt_tasks:
            dataset_loader1, dataset_loader2, task_net = task.sample_data(size=self.k)
            task_optimiser = torch.optim.Adam([{'params':task_net.parameters()}],lr=0.001, weight_decay=3e-4)
        else:
            dataset_loader1, dataset_loader2 = task.sample_data(size=self.k)  # 从任务中采样数据集 D 和 D'
        dataloader1 = DataLoader(
                dataset_loader1,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=1,
                drop_last=False)
        for step in range(self.inner_step):
            if step == 0 and adapt_tasks or attenuate or tapt:
                loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
                grads = torch.autograd.grad(loss, temp_weights, create_graph=True, retain_graph=True)  # 计算损失对参数的梯度
                if adapt_tasks:     # every tasks
                    grads = tuple(grads[i] for i in range(len(grads)))
                    layerwise_mean_grads = []
                    for i in range(len(grads)):
                        layerwise_mean_grads.append(grads[i].mean())  # 每一层的梯度取平均值
                    layerwise_mean_grads = torch.stack(layerwise_mean_grads)
                    temp_weights = attenuate_init(task_net, None, layerwise_mean_grads, temp_weights)  # 得到新的权重

                if attenuate or tapt:    #L2F
                    layerwise_mean_grads = []
                    for i in range(len(grads)):
                        layerwise_mean_grads.append(grads[i].mean())  # 每一层的梯度取平均值
                    layerwise_mean_grads = torch.stack(layerwise_mean_grads)
                    if attenuate and tapt:
                        temp_weights = attenuate_init(self.attenuator, self.panning, layerwise_mean_grads, temp_weights)  # 得到新的权重
                    elif attenuate and not tapt:
                        temp_weights = attenuate_init(self.attenuator, None, layerwise_mean_grads, temp_weights)
                    elif not attenuate and tapt:
                        temp_weights = attenuate_init(None, self.panning, layerwise_mean_grads, temp_weights)

            if MeTAL:
                support_task_state = []
                # 获取支持集 Loss 和 预测值
                support_loss, support_preds = inner_train(dataloader1, self.net, temp_weights, self.criterion, metal=1)
                support_loss /= int(self.batch_size)
                support_task_state.append(support_loss)

                # 将每层模型权重的平均值添加到task_state中
                for v in temp_weights:
                    support_task_state.append(v.mean())

                support_task_state = torch.stack(support_task_state)
                # 归一化
                adapt_support_task_state = (support_task_state - support_task_state.mean())/(support_task_state.std() + 1e-12)                                                 
                # 损失函数网络参数更新
                updated_meta_loss_weights = self.meta_loss_adapter(adapt_support_task_state, step, self.names_loss_weights_copy)
                for i, item in enumerate(dataloader1):
                    acceleration = item['label']
                # 将预测值添加到task_state
                support_task_state = torch.cat((
                    support_task_state.view(1, -1).expand(support_preds.size(1), -1),
                    support_preds.transpose(0, 1).mean(dim=1, keepdim=True),
                    acceleration.mean(dim=1, keepdim=True).float().to(device)
                ), -1)

                support_task_state = (support_task_state - support_task_state.mean()) / (support_task_state.std() + 1e-12)
                # 计算support_loss
                meta_support_loss = self.meta_loss(support_task_state, step, params=updated_meta_loss_weights).mean().squeeze()
                if semi_supervised:
                    dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                    query_loss, query_preds = inner_train(dataloader2, self.net, temp_weights, self.criterion, metal=1)
                    query_loss /= int(self.batch_size)
                    query_task_state = []
                    
                    for i, item in enumerate(dataloader2):
                        acceleration = item['label']
                    # query_task_state.append(query_loss)

                    for v in temp_weights:
                        query_task_state.append(v.mean())
                    query_task_state = torch.stack(query_task_state)
                    query_task_state = torch.cat((
                                query_task_state.view(1, -1).expand(query_preds.size(1), -1),
                                query_preds.transpose(0, 1).mean(dim=1, keepdim=True),
                                acceleration.mean(dim=1, keepdim=True).float().to(device)
                    ), -1)

                    query_task_state = (query_task_state - query_task_state.mean())/(query_task_state.std() + 1e-12)
                    updated_meta_query_loss_weights = self.meta_query_loss_adapter(query_task_state.mean(0), step, self.names_query_loss_weights_copy)

                    meta_query_loss = self.meta_query_loss(query_task_state, step, params=updated_meta_query_loss_weights).mean().squeeze()

                    loss = support_loss + meta_support_loss + meta_query_loss
                else:
                    loss = support_loss + meta_support_loss
            else:
                # 训练过程
                loss = inner_train(dataloader1, self.net, temp_weights, self.criterion) / self.batch_size
            grads = torch.autograd.grad(loss, temp_weights, create_graph=True, retain_graph=True)  # 计算损失对参数的梯度

            if alfa:    #ALFA
                # 用于存储权重的平均值和梯度的平均值
                per_step_task_embedding = []
                for v in temp_weights:
                    per_step_task_embedding.append(v.mean())
            
                for i in range(len(grads)):
                    per_step_task_embedding.append(grads[i].mean())

                per_step_task_embedding = torch.stack(per_step_task_embedding)

                generated_params = self.regularizer(per_step_task_embedding)

                generated_alpha, generated_beta = torch.split(generated_params, split_size_or_sections=self.num_layers)
                generated_alpha_params = []
                generated_beta_params = []
                g = 0
                for key in temp_weights:
                    generated_alpha_params.append(generated_alpha[g])
                    generated_beta_params.append(generated_beta[g])
                    g+=1
                # 初始化ALFA自适应参数
                if self.first_add == 0:
                    self.first_add += 1

                    if random_init:
                        self.names_beta_dict_per_param = nn.ParameterDict()
                    self.names_alpha_dict = nn.ParameterDict()
                    self.names_beta_dict = nn.ParameterDict()
                    init_learning_rate = torch.ones(1) * self.alpha
                    init_learning_rate.to(device)
                    init_weight_decay = torch.ones(1)* self.weight_decay
                    init_weight_decay.to(device)

                    for idx, param in enumerate(temp_weights):

                        if random_init:
                        # per-param weight decay for random init
                            self.names_beta_dict_per_param[str(idx)] = nn.Parameter(
                                data=torch.ones(param.shape) * init_weight_decay * init_learning_rate,
                                requires_grad=use_learnable_learning_rates)

                            self.names_beta_dict[str(idx)] = nn.Parameter(
                                data=torch.ones(self.inner_step),
                                requires_grad=use_learnable_beta)
                            
                            self.names_beta_dict_per_param.to(device)
                        else:
                            # per-step per-layer meta-learnable weight decay bias term (for more stable training and better performance by 2~3%)
                            self.names_beta_dict[str(idx)] = nn.Parameter(
                                data=torch.ones(self.inner_step) * init_weight_decay * init_learning_rate,
                                requires_grad=use_learnable_beta)
                        
                        # per-step per-layer meta-learnable learning rate bias term (for more stable training and better performance by 2~3%)
                        self.names_alpha_dict[str(idx)] = nn.Parameter(
                            data=torch.ones(self.inner_step) * init_learning_rate,
                            requires_grad=use_learnable_alpha)
                        
                        self.names_beta_dict.to(device)
                        self.names_alpha_dict.to(device)
                    # 将额外的参数添加到优化器中
                    if random_init:
                        self.meta_optimiser.add_param_group({'params': self.names_beta_dict_per_param.parameters()})
                    self.meta_optimiser.add_param_group({'params': self.names_beta_dict.parameters()})
                    self.meta_optimiser.add_param_group({'params': self.names_alpha_dict.parameters()})
            else:
                # 使用可学习的参数
                if self.first_add == 0:
                    self.first_add += 1

                    init_learning_rate = torch.ones(1) * self.alpha
                    init_learning_rate.to(device)
                    init_weight_decay = torch.ones(1)* self.weight_decay
                    init_weight_decay.to(device)

                    self.names_learning_rates_dict = nn.ParameterDict()
                    for idx, param in enumerate(temp_weights):
                        self.names_learning_rates_dict[str(idx)] = nn.Parameter(
                            data=torch.ones(self.inner_step) * init_learning_rate,
                            requires_grad=use_learnable_alpha)
                    self.names_learning_rates_dict.to(device)
                    if use_learnable_alpha:
                        self.meta_optimiser.add_param_group({'params': self.names_learning_rates_dict.parameters()})
                        
                    if inner_update == "L2" and self.weight_decay != None:
                        self.names_weight_decay_dict = nn.ParameterDict()
                        for idx, param in enumerate(temp_weights):
                            self.names_weight_decay_dict[str(idx)] = nn.Parameter(
                                    data=torch.ones(self.inner_step) * init_weight_decay,
                                    requires_grad=use_learnable_beta)
                        self.names_weight_decay_dict.to(device)
                        if use_learnable_beta:
                            self.meta_optimiser.add_param_group({'params': self.names_weight_decay_dict.parameters()})

            if inner_update == "L2" and self.weight_decay != None:
                # temp_weights = [w - self.alpha * (g + (self.weight_decay * w)) for w, g in zip(temp_weights, grads)]
                # temp_weights = [((1 - self.alpha * self.weight_decay) * w) - (self.alpha * g) for w, g in zip(temp_weights, grads)]
                temp_weights = [((1 - self.names_learning_rates_dict[str(key)][step] * self.names_weight_decay_dict[str(key)][step]) * w) - (self.names_learning_rates_dict[str(key)][step] * g) for key, (w, g) in enumerate(zip(temp_weights, grads))]
            elif inner_update == "ALFA" and alfa:
                if random_init:
                    temp_weights = [(1 - beta * self.names_beta_dict[str(key)][step] * self.names_beta_dict_per_param[str(key)]) * w - (alpha * self.names_alpha_dict[str(key)][step] * g) for key, (w, g, beta, alpha) in enumerate(zip(temp_weights, grads, generated_beta_params, generated_alpha_params))]
                else:
                    temp_weights = [(1 - beta * self.names_beta_dict[str(key)][step]) * w - (alpha * self.names_alpha_dict[str(key)][step] * g) for key, (w, g, beta, alpha) in enumerate(zip(temp_weights, grads, generated_beta_params, generated_alpha_params))]
            else:
                temp_weights = [w - self.names_learning_rates_dict[str(key)][step] * g for key, (w, g) in enumerate(zip(temp_weights, grads))]  # 临时参数更新 梯度下降

            if use_multi_step_loss_optimization and self.epoch < 20:
                dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                metaloss = inner_train(dataloader2, self.net, temp_weights, self.criterion) / self.batch_size
                taskloss.append(self.per_step_loss_importance_vectors[step] * metaloss)
            else:
                if step == (self.inner_step - 1):
                    dataloader2 = DataLoader(
                            dataset_loader2,
                            batch_size=self.batch_size,
                            shuffle=True,
                            num_workers=1,
                            drop_last=False)
                    metaloss = inner_train(dataloader2, self.net, temp_weights, self.criterion) / self.batch_size
                    taskloss.append(metaloss)
        task_losses = torch.sum(torch.stack(taskloss))
        if adapt_tasks:
            task_optimiser.zero_grad()
            metaloss.backward(retain_graph=True)
            task_optimiser.step()
        return task_losses

    def outer_loop(self, num_epochs):  # epoch 5000
        best_loss = 10000
        best_first_step_error = 10000
        best_index = 0
        for epoch in tqdm(range(1, num_epochs + 1)):
            self.epoch = epoch
            if MeTAL:
                self.names_loss_weights_copy = get_inner_loop_parameter_dict(self.meta_loss.named_parameters())
                if semi_supervised:
                    self.names_query_loss_weights_copy = get_inner_loop_parameter_dict(self.meta_query_loss.named_parameters())
            total_loss = 0
            metaloss_sum = 0
            tasks = self.tasks.sample_task(epoch)
            for i in tasks:
                if use_multi_step_loss_optimization:
                    self.per_step_loss_importance_vectors = get_per_step_loss_importance_vector(self.inner_step, epoch)
                task = DataTask(i)
                metaloss = self.inner_loop(task)  # 内循环更新参数，计算元学习损失
                metaloss_sum += metaloss  # mete_loss求和
            self.meta_optimiser.zero_grad()
            # metaloss_sum.backward(retain_graph=True)
            metagrads = torch.autograd.grad(metaloss_sum, self.weights, retain_graph=True)  # 计算元学习损失对参数的梯度
            # # 重要步骤：使用元学习梯度更新网络参数
            for w, g in zip(self.weights, metagrads):
                w.grad = g

            if MeTAL:
                meta_loss_grads = torch.autograd.grad(metaloss_sum, list(self.meta_loss.parameters()), retain_graph=True)
                for w, g in zip(self.meta_loss.parameters(), meta_loss_grads):
                    w.grad = g

                meta_loss_adapter_grads = torch.autograd.grad(metaloss_sum, list(self.meta_loss_adapter.parameters()), retain_graph=True)
                for w, g in zip(self.meta_loss_adapter.parameters(), meta_loss_adapter_grads):
                    w.grad = g

                if semi_supervised:
                    meta_query_loss_grads = torch.autograd.grad(metaloss_sum, list(self.meta_query_loss.parameters()), retain_graph=True)
                    for w, g in zip(self.meta_query_loss.parameters(), meta_query_loss_grads):
                        w.grad = g

                    meta_query_loss_adapter_grads = torch.autograd.grad(metaloss_sum, list(self.meta_query_loss_adapter.parameters()), retain_graph=True)
                    for w, g in zip(self.meta_query_loss_adapter.parameters(), meta_query_loss_adapter_grads):
                        w.grad = g

            if not alfa and use_learnable_alpha:
                learning_rates_grads = torch.autograd.grad(metaloss_sum, list(self.names_learning_rates_dict.parameters()), retain_graph=True)
                for w, g in zip(self.names_learning_rates_dict.parameters(), learning_rates_grads):
                    w.grad = g
                if inner_update == "L2" and self.weight_decay != None and use_learnable_beta:
                    weight_decay_grads = torch.autograd.grad(metaloss_sum, list(self.names_weight_decay_dict.parameters()), retain_graph=True)
                    for w, g in zip(self.names_weight_decay_dict.parameters(), weight_decay_grads):
                        w.grad = g

            if attenuate:
                attenuate_grads = torch.autograd.grad(metaloss_sum, list(self.attenuator.parameters()), retain_graph=True)
                for w, g in zip(self.attenuator.parameters(), attenuate_grads):
                    w.grad = g
            if tapt:
                tapt_grads = torch.autograd.grad(metaloss_sum, list(self.panning.parameters()), retain_graph=True)
                for w, g in zip(self.panning.parameters(), tapt_grads):
                    w.grad = g
            if alfa:
                alfa_grads = torch.autograd.grad(metaloss_sum, list(self.regularizer.parameters()), retain_graph=True)
                for w, g in zip(self.regularizer.parameters(), alfa_grads):
                    w.grad = g
                if use_learnable_beta:
                    names_beta_grads = torch.autograd.grad(metaloss_sum, list(self.names_beta_dict.parameters()), retain_graph=True)
                    for w, g in zip(self.names_beta_dict.parameters(), names_beta_grads):
                        w.grad = g
                if use_learnable_alpha:
                    names_alpha_grads = torch.autograd.grad(metaloss_sum, self.names_alpha_dict.parameters(), retain_graph=True)
                    for w, g in zip(self.names_alpha_dict.parameters(), names_alpha_grads):
                        w.grad = g
                if random_init:
                    random_init_grads = torch.autograd.grad(metaloss_sum, self.names_beta_dict_per_param.parameters(), retain_graph=True)
                    for w, g in zip(self.names_beta_dict_per_param.parameters(), random_init_grads):
                        w.grad = g
            
            self.meta_optimiser.step()  # 使用外循环优化器更新网络参数
            total_loss += metaloss_sum.item() / len(tasks)
            mserror,col,count,jerk,ttc = model_evaluate(self.net,his_horizon,meta_loader)
            if epoch % self.print_every == 0:
                # print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
                logger.info("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / self.print_every))
                logger.info("{}/{}. mserror: {}  col: {}  count: {}  jerk: {}  ttc: {}".format(epoch, num_epochs, mserror, col, count, jerk, ttc))
                if mserror < best_loss:
                    best_loss = mserror
                    best_index = epoch
                    # with open(file_path, 'wb') as f:
                    #     torch.save(self.net, f)
                self.meta_losses.append(total_loss / self.print_every)
                self.meta_test.append(mserror.cpu().detach().numpy() / self.print_every)
            # finetuning()
            fine_error, first_step_error, best_mse_state = finetuning(self.all_best_state)
            self.all_best_state = best_mse_state
            if first_step_error < best_first_step_error:
                best_first_step_epoch = epoch
                best_first_step_error = first_step_error
            self.finetuning_error.append(fine_error.cpu().detach().numpy())
            # if epoch % self.plot_every == 0:
            #     epoch_path = f"/home/ubuntu/fedavg/FollowNet-car/MAML_state/maml_l2f_test/MAML_nn_a0.01_b0.0001_{epoch}E.pt" # 元初始化参数
            #     # weight_change = torch.load(file_path)
            #     with open(epoch_path, 'wb') as f:
            #         torch.save(self.net, f)
        logger.info("(", best_index, ")","best_loss:", best_loss)
        logger.info("(", best_first_step_epoch, ")", "best_first_step_error:",best_first_step_error)

dataset = 'Waymo'
model_type = 'tcn'
if dataset == 'SPMD1':
    train_data = SPMD1_train
    test_data = SPMD1_test
elif dataset == 'SPMD2':
    train_data = SPMD2_train
    test_data = SPMD2_test
elif dataset == 'Waymo':
    train_data = Waymo_train
    test_data = Waymo_test
elif dataset == 'NGSIM':
    train_data = NGSIM_train
    test_data = NGSIM_test
elif dataset == 'Lyft':
    train_data = Lyft_train
    test_data = Lyft_test
elif dataset == 'HighD':
    train_data = HighD_train
    test_data = HighD_test
# 定义保存文件的文件夹路径
# file_path = "/home/ubuntu/fedavg/FollowNet-car/MAML_state/MAML_MeTAL_nn_a0.01_b0.0001_k300_E50.pt" # 元初始化参数
save_folder = '/kaggle/working/'
save = f'MeTAL_Waymo_{model_type}_{dataset}_{device}_tcnks5_1632_a0.001_b0.01_step1_k200_K40_Mese_acc.pt' # 保存模型文件
# 确保保存文件的文件夹存在，如果不存在，则创建
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
# 定义文件保存路径
save_path = os.path.join(save_folder, save)
meta_test = test_data
meta_loader_test = ImitationCarFolData(meta_test, max_len = max_len, Ts = Ts)
meta_loader = DataLoader(
            meta_loader_test,
            batch_size=96,
            shuffle=False,
            num_workers=1,
            drop_last=False)
# 模型初始化
if model_type == 'nn':
    net = nn_model(input_size = his_horizon*3).to(device)
elif model_type == 'tcn':
    net = TemporalConvNet(num_inputs=3, num_channels=(16,32)).to(device)
elif model_type == 'cnn1d':
    net = CNN1D().to(device)
    # net = cbrd(num_inputs=3, num_output=1).to(device)
# elif model_type == 'lstm':
#     net = lstm_model(input_size = 3, lstm_layers = 1).to(device) # single layer lstm

# model_state = list(net.parameters())
# print(model_state)

# 多gpu训练
# if torch.cuda.device_count() > 1:   # 查看当前电脑的可用的gpu的数量，若gpu数量>1,就多gpu训练
#     net = torch.nn.DataParallel(net)  # 自动选择gpu
# net.to(device)

# 内部循环更新策略
MeTAL = True   # 任务自适应loss函数
semi_supervised = True  # 使用查询集 作为半监督
adapt_tasks = False  # 权重初始化 每个任务用一个
attenuate = False  # 权重初始化 所有任务共用 权重衰减
tapt = False  # 权重初始化 所有任务共用 权重偏差
alfa = False        # 自适应学习优化超参数
random_init = False
use_learnable_learning_rates = random_init
use_learnable_beta = False  # 使优化参数beta可学习
use_learnable_alpha = False  # 使优化参数alpha可学习
use_multi_step_loss_optimization = False  # 内部循环多步时，使用最后一步的Loss，还是使用多步的Loss
inner_update = 'ALFA'
if adapt_tasks:
    num_layers = len(list(net.parameters()))
    HighD_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    Lyft_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    SPMD1_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    SPMD2_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
    Waymo_net = nn.Sequential(
        nn.Linear(num_layers, num_layers),
        nn.ReLU(inplace=True),
        nn.Linear(num_layers, num_layers),
        nn.Sigmoid()
    ).to(device=device)
data_tasks=DataDistribution(data_range=5, n=3)
maml=MAML(net,alpha=0.001,beta=0.01,tasks=data_tasks,k=200,weight_decay=0.0001,inner_step=1)
maml.outer_loop(num_epochs=2)

# plt_path = "/home/ubuntu/fedavg/FollowNet-car/test_metrics"
plt.plot(maml.meta_losses)
# plt.savefig(os.path.join(plt_path, f'MeTAL_Waymo_{device}_tcnks5_1632_a0.001_b0.01_step1_k200_K40_Mese_acc_Loss.png'))
plt.show()
plt.close()

plt.plot(maml.meta_test)
# plt.savefig(os.path.join(plt_path, f'MeTAL_Waymo_{device}_tcnks5_1632_a0.001_b0.01_step1_k200_K40_Mese_acc_mserror.png'))
plt.show()
plt.close()

plt.plot(maml.finetuning_error)
# plt.savefig(os.path.join(plt_path, f'MeTAL_Waymo_{device}_tcnks5_1632_a0.001_b0.01_step1_k200_K40_Mese_acc_finetuning_mserror.png'))
plt.show()
plt.close()
# 将数据保存到文件中
np.save('/kaggle/working/kaggle_maml_meta_losses.npy', np.array(maml.meta_losses))
np.save('/kaggle/working/kaggle_maml_meta_test.npy', np.array(maml.meta_test))
np.save('/kaggle/working/kaggle_maml_finetuning_error.npy', np.array(maml.finetuning_error))

print('----------------------')
logger.info("----------------------")
